In [8]:
# https://catboost.ai/en/docs/concepts/python-quickstart
import catboost as cb
from catboost import CatBoostClassifier


print ('catboost version', cb.__version__)

catboost version 1.0.6


# K-Fold CV for Boosting Methods Using SKLearn 

In [1]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random
import statistics

In [2]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

C:\Users\santo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [4]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:55<00:00,  2.23s/it]


In [5]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [6]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation',
       'Rhythm', 'Subject'],
      dtype='object')


# CatBoosting

In [33]:
# Short to Short

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['StoS']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6656347	total: 6.66ms	remaining: 659ms
1:	learn: 0.6431669	total: 12.7ms	remaining: 621ms
2:	learn: 0.6245068	total: 19.2ms	remaining: 621ms
3:	learn: 0.6094613	total: 24.9ms	remaining: 598ms
4:	learn: 0.5961784	total: 32ms	remaining: 609ms
5:	learn: 0.5856882	total: 37.4ms	remaining: 586ms
6:	learn: 0.5766558	total: 43.8ms	remaining: 582ms
7:	learn: 0.5686886	total: 50.5ms	remaining: 581ms
8:	learn: 0.5623363	total: 57ms	remaining: 576ms
9:	learn: 0.5571214	total: 62.7ms	remaining: 565ms
10:	learn: 0.5526487	total: 69.1ms	remaining: 559ms
11:	learn: 0.5487590	total: 75.3ms	remaining: 552ms
12:	learn: 0.5454489	total: 81.9ms	remaining: 548ms
13:	learn: 0.5425919	total: 89.4ms	remaining: 549ms
14:	learn: 0.5402393	total: 96ms	remaining: 544ms
15:	learn: 0.5385127	total: 103ms	remaining: 540ms
16:	learn: 0.5368418	total: 109ms	remaining: 532ms
17:	learn: 0.5353984	total: 115ms	remaining: 526ms
18:	learn: 0.5341114	total: 122ms	remaining: 520ms
19:	learn: 0.5331108	total: 128m

87:	learn: 0.4993977	total: 540ms	remaining: 73.6ms
88:	learn: 0.4993989	total: 544ms	remaining: 67.3ms
89:	learn: 0.4993937	total: 550ms	remaining: 61.2ms
90:	learn: 0.4993969	total: 557ms	remaining: 55.1ms
91:	learn: 0.4993921	total: 563ms	remaining: 49ms
92:	learn: 0.4993881	total: 570ms	remaining: 42.9ms
93:	learn: 0.4993916	total: 576ms	remaining: 36.8ms
94:	learn: 0.4993862	total: 582ms	remaining: 30.6ms
95:	learn: 0.4993894	total: 587ms	remaining: 24.5ms
96:	learn: 0.4993874	total: 592ms	remaining: 18.3ms
97:	learn: 0.4993843	total: 598ms	remaining: 12.2ms
98:	learn: 0.4993852	total: 603ms	remaining: 6.09ms
99:	learn: 0.4993875	total: 609ms	remaining: 0us
0:	learn: 0.6661007	total: 14.2ms	remaining: 1.4s
1:	learn: 0.6440246	total: 26.9ms	remaining: 1.32s
2:	learn: 0.6256520	total: 56.2ms	remaining: 1.82s
3:	learn: 0.6108790	total: 92.3ms	remaining: 2.21s
4:	learn: 0.5978107	total: 108ms	remaining: 2.06s
5:	learn: 0.5875038	total: 122ms	remaining: 1.92s
6:	learn: 0.5786064	total:

48:	learn: 0.5268883	total: 384ms	remaining: 400ms
49:	learn: 0.5268809	total: 388ms	remaining: 388ms
50:	learn: 0.5268692	total: 392ms	remaining: 376ms
51:	learn: 0.5268458	total: 397ms	remaining: 366ms
52:	learn: 0.5268296	total: 402ms	remaining: 356ms
53:	learn: 0.5268167	total: 406ms	remaining: 346ms
54:	learn: 0.5267963	total: 411ms	remaining: 336ms
55:	learn: 0.5267882	total: 415ms	remaining: 326ms
56:	learn: 0.5267803	total: 419ms	remaining: 316ms
57:	learn: 0.5267770	total: 423ms	remaining: 306ms
58:	learn: 0.5267744	total: 427ms	remaining: 297ms
59:	learn: 0.5267722	total: 432ms	remaining: 288ms
60:	learn: 0.5267551	total: 436ms	remaining: 279ms
61:	learn: 0.5267447	total: 441ms	remaining: 270ms
62:	learn: 0.5267376	total: 445ms	remaining: 261ms
63:	learn: 0.5267391	total: 449ms	remaining: 252ms
64:	learn: 0.5267246	total: 453ms	remaining: 244ms
65:	learn: 0.5267254	total: 457ms	remaining: 235ms
66:	learn: 0.5267246	total: 461ms	remaining: 227ms
67:	learn: 0.5267191	total: 465

21:	learn: 0.4951109	total: 194ms	remaining: 687ms
22:	learn: 0.4944000	total: 199ms	remaining: 667ms
23:	learn: 0.4938017	total: 206ms	remaining: 652ms
24:	learn: 0.4933629	total: 213ms	remaining: 638ms
25:	learn: 0.4929625	total: 218ms	remaining: 621ms
26:	learn: 0.4925967	total: 225ms	remaining: 607ms
27:	learn: 0.4923238	total: 231ms	remaining: 594ms
28:	learn: 0.4920859	total: 236ms	remaining: 578ms
29:	learn: 0.4918507	total: 242ms	remaining: 565ms
30:	learn: 0.4916979	total: 248ms	remaining: 552ms
31:	learn: 0.4915627	total: 253ms	remaining: 538ms
32:	learn: 0.4914169	total: 259ms	remaining: 527ms
33:	learn: 0.4913202	total: 264ms	remaining: 513ms
34:	learn: 0.4912224	total: 270ms	remaining: 501ms
35:	learn: 0.4911341	total: 275ms	remaining: 490ms
36:	learn: 0.4910816	total: 280ms	remaining: 476ms
37:	learn: 0.4910205	total: 286ms	remaining: 466ms
38:	learn: 0.4909533	total: 291ms	remaining: 456ms
39:	learn: 0.4908822	total: 297ms	remaining: 446ms
40:	learn: 0.4908330	total: 303

88:	learn: 0.5017873	total: 551ms	remaining: 68.1ms
89:	learn: 0.5017901	total: 557ms	remaining: 61.9ms
90:	learn: 0.5017906	total: 564ms	remaining: 55.8ms
91:	learn: 0.5017884	total: 571ms	remaining: 49.6ms
92:	learn: 0.5017887	total: 577ms	remaining: 43.5ms
93:	learn: 0.5017901	total: 585ms	remaining: 37.3ms
94:	learn: 0.5017885	total: 591ms	remaining: 31.1ms
95:	learn: 0.5017891	total: 596ms	remaining: 24.9ms
96:	learn: 0.5017901	total: 602ms	remaining: 18.6ms
97:	learn: 0.5017847	total: 609ms	remaining: 12.4ms
98:	learn: 0.5017863	total: 614ms	remaining: 6.2ms
99:	learn: 0.5017840	total: 618ms	remaining: 0us
0:	learn: 0.6659362	total: 10.2ms	remaining: 1.01s
1:	learn: 0.6437307	total: 20.8ms	remaining: 1.02s
2:	learn: 0.6252676	total: 31.1ms	remaining: 1s
3:	learn: 0.6104024	total: 39.1ms	remaining: 939ms
4:	learn: 0.5972720	total: 49.4ms	remaining: 939ms
5:	learn: 0.5868970	total: 57.8ms	remaining: 905ms
6:	learn: 0.5779604	total: 67.7ms	remaining: 899ms
7:	learn: 0.5700830	total:

...

58:	learn: 0.5259314	total: 366ms	remaining: 255ms
59:	learn: 0.5259262	total: 373ms	remaining: 249ms
60:	learn: 0.5259271	total: 379ms	remaining: 242ms
61:	learn: 0.5259197	total: 386ms	remaining: 236ms
62:	learn: 0.5259203	total: 391ms	remaining: 229ms
63:	learn: 0.5259144	total: 397ms	remaining: 223ms
64:	learn: 0.5259129	total: 403ms	remaining: 217ms
65:	learn: 0.5259139	total: 407ms	remaining: 210ms
66:	learn: 0.5259097	total: 411ms	remaining: 202ms
67:	learn: 0.5259016	total: 416ms	remaining: 196ms
68:	learn: 0.5258983	total: 421ms	remaining: 189ms
69:	learn: 0.5258930	total: 426ms	remaining: 182ms
70:	learn: 0.5258848	total: 430ms	remaining: 176ms
71:	learn: 0.5258826	total: 436ms	remaining: 169ms
72:	learn: 0.5258801	total: 440ms	remaining: 163ms
73:	learn: 0.5258778	total: 443ms	remaining: 156ms
74:	learn: 0.5258788	total: 447ms	remaining: 149ms
75:	learn: 0.5258775	total: 451ms	remaining: 142ms
76:	learn: 0.5258712	total: 456ms	remaining: 136ms
77:	learn: 0.5258731	total: 461

26:	learn: 0.4911141	total: 183ms	remaining: 493ms
27:	learn: 0.4907023	total: 190ms	remaining: 488ms
28:	learn: 0.4903188	total: 197ms	remaining: 483ms
29:	learn: 0.4900118	total: 205ms	remaining: 478ms
30:	learn: 0.4897349	total: 213ms	remaining: 474ms
31:	learn: 0.4895294	total: 219ms	remaining: 466ms
32:	learn: 0.4893345	total: 227ms	remaining: 461ms
33:	learn: 0.4891689	total: 234ms	remaining: 454ms
34:	learn: 0.4890169	total: 241ms	remaining: 448ms
35:	learn: 0.4889090	total: 246ms	remaining: 438ms
36:	learn: 0.4887938	total: 253ms	remaining: 431ms
37:	learn: 0.4887295	total: 258ms	remaining: 421ms
38:	learn: 0.4886900	total: 264ms	remaining: 412ms
39:	learn: 0.4886321	total: 270ms	remaining: 406ms
40:	learn: 0.4885786	total: 276ms	remaining: 397ms
41:	learn: 0.4885097	total: 282ms	remaining: 390ms
42:	learn: 0.4884541	total: 290ms	remaining: 384ms
43:	learn: 0.4883873	total: 297ms	remaining: 377ms
44:	learn: 0.4883526	total: 303ms	remaining: 370ms
45:	learn: 0.4883370	total: 308

96:	learn: 0.4972364	total: 542ms	remaining: 16.8ms
97:	learn: 0.4972354	total: 546ms	remaining: 11.1ms
98:	learn: 0.4972399	total: 550ms	remaining: 5.56ms
99:	learn: 0.4972350	total: 555ms	remaining: 0us
0:	learn: 0.6644148	total: 7.44ms	remaining: 736ms
1:	learn: 0.6409824	total: 16.2ms	remaining: 793ms
2:	learn: 0.6215326	total: 24.3ms	remaining: 785ms
3:	learn: 0.6057746	total: 30.7ms	remaining: 737ms
4:	learn: 0.5918831	total: 38.6ms	remaining: 733ms
5:	learn: 0.5808591	total: 45.3ms	remaining: 710ms
6:	learn: 0.5713717	total: 52.8ms	remaining: 702ms
7:	learn: 0.5630067	total: 61ms	remaining: 701ms
8:	learn: 0.5563016	total: 68.5ms	remaining: 693ms
9:	learn: 0.5507771	total: 75.2ms	remaining: 677ms
10:	learn: 0.5461635	total: 82.9ms	remaining: 670ms
11:	learn: 0.5419364	total: 90.5ms	remaining: 664ms
12:	learn: 0.5383785	total: 98.6ms	remaining: 660ms
13:	learn: 0.5353633	total: 107ms	remaining: 655ms
14:	learn: 0.5332750	total: 114ms	remaining: 645ms
15:	learn: 0.5311196	total: 1

75:	learn: 0.5252194	total: 555ms	remaining: 175ms
76:	learn: 0.5252156	total: 562ms	remaining: 168ms
77:	learn: 0.5252130	total: 570ms	remaining: 161ms
78:	learn: 0.5252117	total: 576ms	remaining: 153ms
79:	learn: 0.5252104	total: 587ms	remaining: 147ms
80:	learn: 0.5252056	total: 595ms	remaining: 140ms
81:	learn: 0.5252023	total: 601ms	remaining: 132ms
82:	learn: 0.5252041	total: 609ms	remaining: 125ms
83:	learn: 0.5252045	total: 617ms	remaining: 117ms
84:	learn: 0.5252003	total: 624ms	remaining: 110ms
85:	learn: 0.5251998	total: 631ms	remaining: 103ms
86:	learn: 0.5251966	total: 637ms	remaining: 95.2ms
87:	learn: 0.5251962	total: 644ms	remaining: 87.8ms
88:	learn: 0.5251969	total: 651ms	remaining: 80.4ms
89:	learn: 0.5251925	total: 658ms	remaining: 73.2ms
90:	learn: 0.5251940	total: 666ms	remaining: 65.9ms
91:	learn: 0.5251912	total: 674ms	remaining: 58.6ms
92:	learn: 0.5251887	total: 681ms	remaining: 51.3ms
93:	learn: 0.5251900	total: 690ms	remaining: 44ms
94:	learn: 0.5251860	tota

60:	learn: 0.5265942	total: 369ms	remaining: 236ms
61:	learn: 0.5265844	total: 375ms	remaining: 230ms
62:	learn: 0.5265813	total: 381ms	remaining: 224ms
63:	learn: 0.5265801	total: 387ms	remaining: 217ms
64:	learn: 0.5265707	total: 393ms	remaining: 212ms
65:	learn: 0.5265723	total: 421ms	remaining: 217ms
66:	learn: 0.5265653	total: 430ms	remaining: 212ms
67:	learn: 0.5265594	total: 438ms	remaining: 206ms
68:	learn: 0.5265557	total: 444ms	remaining: 199ms
69:	learn: 0.5265530	total: 450ms	remaining: 193ms
70:	learn: 0.5265582	total: 455ms	remaining: 186ms
71:	learn: 0.5265526	total: 461ms	remaining: 179ms
72:	learn: 0.5265512	total: 467ms	remaining: 173ms
73:	learn: 0.5265484	total: 473ms	remaining: 166ms
74:	learn: 0.5265419	total: 478ms	remaining: 159ms
75:	learn: 0.5265440	total: 484ms	remaining: 153ms
76:	learn: 0.5265422	total: 489ms	remaining: 146ms
77:	learn: 0.5265431	total: 495ms	remaining: 140ms
78:	learn: 0.5265398	total: 501ms	remaining: 133ms
79:	learn: 0.5265371	total: 507

In [34]:
# Short to Regular

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['StoR']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6313605	total: 6.88ms	remaining: 681ms
1:	learn: 0.5809475	total: 14.3ms	remaining: 701ms
2:	learn: 0.5393044	total: 21.6ms	remaining: 700ms
3:	learn: 0.5045491	total: 28ms	remaining: 673ms
4:	learn: 0.4747995	total: 34.5ms	remaining: 655ms
5:	learn: 0.4494429	total: 41.4ms	remaining: 648ms
6:	learn: 0.4284476	total: 47.2ms	remaining: 627ms
7:	learn: 0.4100604	total: 54ms	remaining: 621ms
8:	learn: 0.3946656	total: 64.3ms	remaining: 650ms
9:	learn: 0.3810969	total: 73.6ms	remaining: 662ms
10:	learn: 0.3694459	total: 81.9ms	remaining: 663ms
11:	learn: 0.3596859	total: 88.9ms	remaining: 652ms
12:	learn: 0.3512809	total: 96.8ms	remaining: 648ms
13:	learn: 0.3436523	total: 104ms	remaining: 640ms
14:	learn: 0.3370306	total: 111ms	remaining: 629ms
15:	learn: 0.3317848	total: 116ms	remaining: 609ms
16:	learn: 0.3271275	total: 122ms	remaining: 597ms
17:	learn: 0.3226674	total: 128ms	remaining: 584ms
18:	learn: 0.3187209	total: 135ms	remaining: 574ms
19:	learn: 0.3154339	total: 140m

72:	learn: 0.2957469	total: 539ms	remaining: 199ms
73:	learn: 0.2957420	total: 546ms	remaining: 192ms
74:	learn: 0.2957407	total: 551ms	remaining: 184ms
75:	learn: 0.2957415	total: 559ms	remaining: 177ms
76:	learn: 0.2957355	total: 565ms	remaining: 169ms
77:	learn: 0.2957296	total: 573ms	remaining: 162ms
78:	learn: 0.2957283	total: 579ms	remaining: 154ms
79:	learn: 0.2957277	total: 585ms	remaining: 146ms
80:	learn: 0.2957247	total: 591ms	remaining: 139ms
81:	learn: 0.2957234	total: 602ms	remaining: 132ms
82:	learn: 0.2957232	total: 609ms	remaining: 125ms
83:	learn: 0.2957198	total: 615ms	remaining: 117ms
84:	learn: 0.2957183	total: 622ms	remaining: 110ms
85:	learn: 0.2957162	total: 628ms	remaining: 102ms
86:	learn: 0.2957175	total: 636ms	remaining: 95.1ms
87:	learn: 0.2957158	total: 642ms	remaining: 87.5ms
88:	learn: 0.2957141	total: 649ms	remaining: 80.3ms
89:	learn: 0.2957142	total: 654ms	remaining: 72.7ms
90:	learn: 0.2957129	total: 660ms	remaining: 65.3ms
91:	learn: 0.2957119	total

57:	learn: 0.2897140	total: 359ms	remaining: 260ms
58:	learn: 0.2897048	total: 366ms	remaining: 254ms
59:	learn: 0.2896764	total: 371ms	remaining: 247ms
60:	learn: 0.2896717	total: 376ms	remaining: 240ms
61:	learn: 0.2896679	total: 380ms	remaining: 233ms
62:	learn: 0.2896465	total: 384ms	remaining: 226ms
63:	learn: 0.2896226	total: 390ms	remaining: 220ms
64:	learn: 0.2896091	total: 395ms	remaining: 213ms
65:	learn: 0.2896026	total: 401ms	remaining: 207ms
66:	learn: 0.2895864	total: 407ms	remaining: 201ms
67:	learn: 0.2895744	total: 412ms	remaining: 194ms
68:	learn: 0.2895733	total: 418ms	remaining: 188ms
69:	learn: 0.2895714	total: 424ms	remaining: 182ms
70:	learn: 0.2895707	total: 430ms	remaining: 175ms
71:	learn: 0.2895612	total: 435ms	remaining: 169ms
72:	learn: 0.2895610	total: 439ms	remaining: 162ms
73:	learn: 0.2895605	total: 445ms	remaining: 156ms
74:	learn: 0.2895535	total: 449ms	remaining: 150ms
75:	learn: 0.2895409	total: 455ms	remaining: 144ms
76:	learn: 0.2895370	total: 460

28:	learn: 0.3022336	total: 184ms	remaining: 451ms
29:	learn: 0.3012613	total: 192ms	remaining: 447ms
30:	learn: 0.3005251	total: 198ms	remaining: 441ms
31:	learn: 0.2997920	total: 206ms	remaining: 437ms
32:	learn: 0.2994400	total: 213ms	remaining: 432ms
33:	learn: 0.2991455	total: 219ms	remaining: 426ms
34:	learn: 0.2985975	total: 226ms	remaining: 419ms
35:	learn: 0.2981918	total: 230ms	remaining: 409ms
36:	learn: 0.2977712	total: 235ms	remaining: 401ms
37:	learn: 0.2974150	total: 241ms	remaining: 394ms
38:	learn: 0.2971153	total: 247ms	remaining: 386ms
39:	learn: 0.2968390	total: 252ms	remaining: 378ms
40:	learn: 0.2965847	total: 258ms	remaining: 372ms
41:	learn: 0.2963676	total: 264ms	remaining: 365ms
42:	learn: 0.2961794	total: 270ms	remaining: 359ms
43:	learn: 0.2961252	total: 277ms	remaining: 352ms
44:	learn: 0.2959740	total: 283ms	remaining: 346ms
45:	learn: 0.2959488	total: 287ms	remaining: 337ms
46:	learn: 0.2959161	total: 293ms	remaining: 331ms
47:	learn: 0.2958050	total: 300

93:	learn: 0.2896824	total: 748ms	remaining: 47.8ms
94:	learn: 0.2896801	total: 757ms	remaining: 39.9ms
95:	learn: 0.2896794	total: 766ms	remaining: 31.9ms
96:	learn: 0.2896807	total: 774ms	remaining: 23.9ms
97:	learn: 0.2896771	total: 783ms	remaining: 16ms
98:	learn: 0.2896759	total: 793ms	remaining: 8.01ms
99:	learn: 0.2896725	total: 801ms	remaining: 0us
0:	learn: 0.6325403	total: 6.92ms	remaining: 686ms
1:	learn: 0.5830849	total: 13.5ms	remaining: 661ms
2:	learn: 0.5422351	total: 20.2ms	remaining: 652ms
3:	learn: 0.5081786	total: 25.9ms	remaining: 621ms
4:	learn: 0.4790546	total: 32ms	remaining: 608ms
5:	learn: 0.4542540	total: 39ms	remaining: 611ms
6:	learn: 0.4337092	total: 44.3ms	remaining: 588ms
7:	learn: 0.4157429	total: 50.2ms	remaining: 577ms
8:	learn: 0.4007001	total: 56.9ms	remaining: 575ms
9:	learn: 0.3874674	total: 63.8ms	remaining: 574ms
10:	learn: 0.3761478	total: 70.5ms	remaining: 570ms
11:	learn: 0.3671394	total: 76.7ms	remaining: 562ms
12:	learn: 0.3584310	total: 84.

...

56:	learn: 0.2927994	total: 362ms	remaining: 273ms
57:	learn: 0.2927864	total: 369ms	remaining: 267ms
58:	learn: 0.2927732	total: 375ms	remaining: 261ms
59:	learn: 0.2927413	total: 381ms	remaining: 254ms
60:	learn: 0.2927394	total: 388ms	remaining: 248ms
61:	learn: 0.2927297	total: 394ms	remaining: 242ms
62:	learn: 0.2927292	total: 398ms	remaining: 234ms
63:	learn: 0.2926997	total: 404ms	remaining: 227ms
64:	learn: 0.2926998	total: 408ms	remaining: 220ms
65:	learn: 0.2926910	total: 414ms	remaining: 213ms
66:	learn: 0.2926675	total: 420ms	remaining: 207ms
67:	learn: 0.2926484	total: 425ms	remaining: 200ms
68:	learn: 0.2926493	total: 431ms	remaining: 194ms
69:	learn: 0.2926341	total: 435ms	remaining: 186ms
70:	learn: 0.2926323	total: 440ms	remaining: 180ms
71:	learn: 0.2926182	total: 447ms	remaining: 174ms
72:	learn: 0.2926068	total: 453ms	remaining: 167ms
73:	learn: 0.2925950	total: 458ms	remaining: 161ms
74:	learn: 0.2925951	total: 462ms	remaining: 154ms
75:	learn: 0.2925936	total: 468

29:	learn: 0.2978547	total: 190ms	remaining: 443ms
30:	learn: 0.2970994	total: 197ms	remaining: 439ms
31:	learn: 0.2963655	total: 204ms	remaining: 434ms
32:	learn: 0.2960001	total: 211ms	remaining: 429ms
33:	learn: 0.2956827	total: 219ms	remaining: 424ms
34:	learn: 0.2951279	total: 226ms	remaining: 419ms
35:	learn: 0.2947105	total: 232ms	remaining: 412ms
36:	learn: 0.2942842	total: 240ms	remaining: 408ms
37:	learn: 0.2939126	total: 247ms	remaining: 403ms
38:	learn: 0.2936004	total: 253ms	remaining: 395ms
39:	learn: 0.2933098	total: 259ms	remaining: 388ms
40:	learn: 0.2930415	total: 265ms	remaining: 381ms
41:	learn: 0.2928077	total: 272ms	remaining: 375ms
42:	learn: 0.2926091	total: 278ms	remaining: 368ms
43:	learn: 0.2925482	total: 284ms	remaining: 361ms
44:	learn: 0.2923847	total: 290ms	remaining: 354ms
45:	learn: 0.2923553	total: 294ms	remaining: 345ms
46:	learn: 0.2922262	total: 300ms	remaining: 338ms
47:	learn: 0.2921201	total: 306ms	remaining: 331ms
48:	learn: 0.2920217	total: 311

92:	learn: 0.2686241	total: 541ms	remaining: 40.7ms
93:	learn: 0.2686205	total: 547ms	remaining: 34.9ms
94:	learn: 0.2686195	total: 553ms	remaining: 29.1ms
95:	learn: 0.2686193	total: 559ms	remaining: 23.3ms
96:	learn: 0.2686184	total: 564ms	remaining: 17.5ms
97:	learn: 0.2686187	total: 571ms	remaining: 11.6ms
98:	learn: 0.2686198	total: 577ms	remaining: 5.83ms
99:	learn: 0.2686176	total: 583ms	remaining: 0us
0:	learn: 0.6320929	total: 10.3ms	remaining: 1.01s
1:	learn: 0.5822795	total: 19.3ms	remaining: 947ms
2:	learn: 0.5411066	total: 28.2ms	remaining: 912ms
3:	learn: 0.5067826	total: 36.9ms	remaining: 885ms
4:	learn: 0.4774057	total: 44.9ms	remaining: 854ms
5:	learn: 0.4523608	total: 54.3ms	remaining: 850ms
6:	learn: 0.4316466	total: 62.1ms	remaining: 824ms
7:	learn: 0.4134866	total: 70.4ms	remaining: 809ms
8:	learn: 0.3982870	total: 79.1ms	remaining: 800ms
9:	learn: 0.3848905	total: 88.4ms	remaining: 796ms
10:	learn: 0.3733967	total: 96.7ms	remaining: 783ms
11:	learn: 0.3637353	tota

59:	learn: 0.2984211	total: 363ms	remaining: 242ms
60:	learn: 0.2984162	total: 371ms	remaining: 237ms
61:	learn: 0.2984096	total: 378ms	remaining: 232ms
62:	learn: 0.2984068	total: 382ms	remaining: 225ms
63:	learn: 0.2983760	total: 389ms	remaining: 219ms
64:	learn: 0.2983732	total: 394ms	remaining: 212ms
65:	learn: 0.2983639	total: 400ms	remaining: 206ms
66:	learn: 0.2983385	total: 407ms	remaining: 200ms
67:	learn: 0.2983160	total: 411ms	remaining: 194ms
68:	learn: 0.2983131	total: 416ms	remaining: 187ms
69:	learn: 0.2982946	total: 422ms	remaining: 181ms
70:	learn: 0.2982905	total: 429ms	remaining: 175ms
71:	learn: 0.2982723	total: 435ms	remaining: 169ms
72:	learn: 0.2982592	total: 442ms	remaining: 163ms
73:	learn: 0.2982475	total: 448ms	remaining: 157ms
74:	learn: 0.2982463	total: 452ms	remaining: 151ms
75:	learn: 0.2982429	total: 458ms	remaining: 145ms
76:	learn: 0.2982419	total: 464ms	remaining: 139ms
77:	learn: 0.2982328	total: 469ms	remaining: 132ms
78:	learn: 0.2982238	total: 475

38:	learn: 0.3009494	total: 373ms	remaining: 584ms
39:	learn: 0.3006764	total: 382ms	remaining: 573ms
40:	learn: 0.3004246	total: 392ms	remaining: 564ms
41:	learn: 0.3002062	total: 402ms	remaining: 556ms
42:	learn: 0.3000138	total: 412ms	remaining: 546ms
43:	learn: 0.2999570	total: 423ms	remaining: 538ms
44:	learn: 0.2998072	total: 432ms	remaining: 528ms
45:	learn: 0.2997748	total: 439ms	remaining: 515ms
46:	learn: 0.2996589	total: 447ms	remaining: 504ms
47:	learn: 0.2995618	total: 456ms	remaining: 494ms
48:	learn: 0.2994720	total: 466ms	remaining: 485ms
49:	learn: 0.2993977	total: 475ms	remaining: 475ms
50:	learn: 0.2993374	total: 483ms	remaining: 464ms
51:	learn: 0.2993260	total: 491ms	remaining: 453ms
52:	learn: 0.2992659	total: 499ms	remaining: 443ms
53:	learn: 0.2992204	total: 508ms	remaining: 433ms
54:	learn: 0.2991829	total: 515ms	remaining: 422ms
55:	learn: 0.2991432	total: 524ms	remaining: 412ms
56:	learn: 0.2991198	total: 533ms	remaining: 402ms
57:	learn: 0.2991079	total: 543

In [35]:
# Short to Long

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['StoL']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6770762	total: 6.44ms	remaining: 638ms
1:	learn: 0.6639136	total: 15.9ms	remaining: 778ms
2:	learn: 0.6533399	total: 21.1ms	remaining: 682ms
3:	learn: 0.6448035	total: 26.2ms	remaining: 629ms
4:	learn: 0.6377233	total: 32.1ms	remaining: 609ms
5:	learn: 0.6318412	total: 38.2ms	remaining: 599ms
6:	learn: 0.6270368	total: 43.3ms	remaining: 575ms
7:	learn: 0.6230218	total: 49.5ms	remaining: 569ms
8:	learn: 0.6197956	total: 55.8ms	remaining: 564ms
9:	learn: 0.6171253	total: 62.7ms	remaining: 565ms
10:	learn: 0.6148960	total: 69.5ms	remaining: 562ms
11:	learn: 0.6131547	total: 76.1ms	remaining: 558ms
12:	learn: 0.6116294	total: 82.7ms	remaining: 554ms
13:	learn: 0.6104023	total: 88.6ms	remaining: 544ms
14:	learn: 0.6094182	total: 94.3ms	remaining: 534ms
15:	learn: 0.6085463	total: 101ms	remaining: 530ms
16:	learn: 0.6078770	total: 107ms	remaining: 521ms
17:	learn: 0.6072821	total: 113ms	remaining: 516ms
18:	learn: 0.6068142	total: 119ms	remaining: 508ms
19:	learn: 0.6064232	total

80:	learn: 0.6060920	total: 570ms	remaining: 134ms
81:	learn: 0.6060880	total: 576ms	remaining: 127ms
82:	learn: 0.6060935	total: 583ms	remaining: 119ms
83:	learn: 0.6060959	total: 589ms	remaining: 112ms
84:	learn: 0.6060907	total: 595ms	remaining: 105ms
85:	learn: 0.6060907	total: 602ms	remaining: 98ms
86:	learn: 0.6060963	total: 606ms	remaining: 90.6ms
87:	learn: 0.6060955	total: 612ms	remaining: 83.4ms
88:	learn: 0.6060829	total: 617ms	remaining: 76.3ms
89:	learn: 0.6060937	total: 624ms	remaining: 69.3ms
90:	learn: 0.6060854	total: 629ms	remaining: 62.2ms
91:	learn: 0.6060849	total: 633ms	remaining: 55.1ms
92:	learn: 0.6060879	total: 639ms	remaining: 48.1ms
93:	learn: 0.6060882	total: 644ms	remaining: 41.1ms
94:	learn: 0.6060874	total: 648ms	remaining: 34.1ms
95:	learn: 0.6060816	total: 654ms	remaining: 27.2ms
96:	learn: 0.6060818	total: 658ms	remaining: 20.3ms
97:	learn: 0.6060807	total: 662ms	remaining: 13.5ms
98:	learn: 0.6060821	total: 668ms	remaining: 6.74ms
99:	learn: 0.606083

62:	learn: 0.5975579	total: 371ms	remaining: 218ms
63:	learn: 0.5975594	total: 376ms	remaining: 212ms
64:	learn: 0.5975640	total: 383ms	remaining: 206ms
65:	learn: 0.5975571	total: 388ms	remaining: 200ms
66:	learn: 0.5975557	total: 395ms	remaining: 195ms
67:	learn: 0.5975557	total: 403ms	remaining: 190ms
68:	learn: 0.5975522	total: 410ms	remaining: 184ms
69:	learn: 0.5975514	total: 417ms	remaining: 179ms
70:	learn: 0.5975482	total: 424ms	remaining: 173ms
71:	learn: 0.5975552	total: 431ms	remaining: 167ms
72:	learn: 0.5975514	total: 439ms	remaining: 162ms
73:	learn: 0.5975485	total: 445ms	remaining: 156ms
74:	learn: 0.5975484	total: 451ms	remaining: 150ms
75:	learn: 0.5975478	total: 457ms	remaining: 144ms
76:	learn: 0.5975454	total: 463ms	remaining: 138ms
77:	learn: 0.5975411	total: 471ms	remaining: 133ms
78:	learn: 0.5975370	total: 477ms	remaining: 127ms
79:	learn: 0.5975379	total: 484ms	remaining: 121ms
80:	learn: 0.5975456	total: 489ms	remaining: 115ms
81:	learn: 0.5975462	total: 494

30:	learn: 0.6062122	total: 192ms	remaining: 428ms
31:	learn: 0.6061722	total: 198ms	remaining: 420ms
32:	learn: 0.6061173	total: 203ms	remaining: 412ms
33:	learn: 0.6060789	total: 208ms	remaining: 403ms
34:	learn: 0.6060493	total: 214ms	remaining: 397ms
35:	learn: 0.6060155	total: 218ms	remaining: 388ms
36:	learn: 0.6059908	total: 223ms	remaining: 380ms
37:	learn: 0.6059629	total: 230ms	remaining: 375ms
38:	learn: 0.6059524	total: 235ms	remaining: 368ms
39:	learn: 0.6059288	total: 242ms	remaining: 363ms
40:	learn: 0.6059003	total: 249ms	remaining: 358ms
41:	learn: 0.6059002	total: 255ms	remaining: 352ms
42:	learn: 0.6058723	total: 262ms	remaining: 347ms
43:	learn: 0.6058557	total: 268ms	remaining: 341ms
44:	learn: 0.6058510	total: 273ms	remaining: 334ms
45:	learn: 0.6058526	total: 279ms	remaining: 327ms
46:	learn: 0.6058386	total: 285ms	remaining: 321ms
47:	learn: 0.6058250	total: 291ms	remaining: 316ms
48:	learn: 0.6058228	total: 298ms	remaining: 310ms
49:	learn: 0.6058133	total: 304

0:	learn: 0.6771913	total: 9.88ms	remaining: 978ms
1:	learn: 0.6641585	total: 20ms	remaining: 978ms
2:	learn: 0.6536213	total: 30ms	remaining: 969ms
3:	learn: 0.6451553	total: 38.9ms	remaining: 934ms
4:	learn: 0.6381344	total: 49.9ms	remaining: 948ms
5:	learn: 0.6323201	total: 60.5ms	remaining: 948ms
6:	learn: 0.6275768	total: 71.4ms	remaining: 949ms
7:	learn: 0.6236203	total: 83.1ms	remaining: 955ms
8:	learn: 0.6204814	total: 93ms	remaining: 940ms
9:	learn: 0.6178469	total: 104ms	remaining: 933ms
10:	learn: 0.6156672	total: 115ms	remaining: 927ms
11:	learn: 0.6139359	total: 126ms	remaining: 921ms
12:	learn: 0.6124318	total: 138ms	remaining: 927ms
13:	learn: 0.6112303	total: 151ms	remaining: 926ms
14:	learn: 0.6102724	total: 161ms	remaining: 911ms
15:	learn: 0.6094223	total: 171ms	remaining: 896ms
16:	learn: 0.6087697	total: 179ms	remaining: 874ms
17:	learn: 0.6081975	total: 189ms	remaining: 861ms
18:	learn: 0.6077526	total: 201ms	remaining: 858ms
19:	learn: 0.6073753	total: 213ms	rema

...

66:	learn: 0.5962514	total: 539ms	remaining: 266ms
67:	learn: 0.5962484	total: 547ms	remaining: 258ms
68:	learn: 0.5962464	total: 553ms	remaining: 248ms
69:	learn: 0.5962422	total: 562ms	remaining: 241ms
70:	learn: 0.5962376	total: 569ms	remaining: 232ms
71:	learn: 0.5962426	total: 578ms	remaining: 225ms
72:	learn: 0.5962421	total: 584ms	remaining: 216ms
73:	learn: 0.5962399	total: 589ms	remaining: 207ms
74:	learn: 0.5962420	total: 595ms	remaining: 198ms
75:	learn: 0.5962362	total: 602ms	remaining: 190ms
76:	learn: 0.5962377	total: 609ms	remaining: 182ms
77:	learn: 0.5962407	total: 617ms	remaining: 174ms
78:	learn: 0.5962350	total: 625ms	remaining: 166ms
79:	learn: 0.5962356	total: 632ms	remaining: 158ms
80:	learn: 0.5962336	total: 640ms	remaining: 150ms
81:	learn: 0.5962292	total: 646ms	remaining: 142ms
82:	learn: 0.5962282	total: 654ms	remaining: 134ms
83:	learn: 0.5962307	total: 662ms	remaining: 126ms
84:	learn: 0.5962333	total: 669ms	remaining: 118ms
85:	learn: 0.5962287	total: 676

31:	learn: 0.5961368	total: 193ms	remaining: 409ms
32:	learn: 0.5960831	total: 198ms	remaining: 403ms
33:	learn: 0.5960352	total: 206ms	remaining: 399ms
34:	learn: 0.5959904	total: 212ms	remaining: 394ms
35:	learn: 0.5959628	total: 217ms	remaining: 386ms
36:	learn: 0.5959335	total: 223ms	remaining: 380ms
37:	learn: 0.5959132	total: 229ms	remaining: 374ms
38:	learn: 0.5958987	total: 234ms	remaining: 366ms
39:	learn: 0.5958766	total: 240ms	remaining: 360ms
40:	learn: 0.5958504	total: 246ms	remaining: 354ms
41:	learn: 0.5958414	total: 252ms	remaining: 349ms
42:	learn: 0.5958184	total: 259ms	remaining: 343ms
43:	learn: 0.5958133	total: 265ms	remaining: 337ms
44:	learn: 0.5958040	total: 269ms	remaining: 329ms
45:	learn: 0.5957951	total: 275ms	remaining: 323ms
46:	learn: 0.5957891	total: 281ms	remaining: 316ms
47:	learn: 0.5957778	total: 287ms	remaining: 311ms
48:	learn: 0.5957653	total: 293ms	remaining: 305ms
49:	learn: 0.5957638	total: 299ms	remaining: 299ms
50:	learn: 0.5957627	total: 305

0:	learn: 0.6766565	total: 8.51ms	remaining: 842ms
1:	learn: 0.6631590	total: 16.9ms	remaining: 826ms
2:	learn: 0.6522423	total: 24.5ms	remaining: 793ms
3:	learn: 0.6435159	total: 31.5ms	remaining: 756ms
4:	learn: 0.6361158	total: 39.8ms	remaining: 756ms
5:	learn: 0.6300859	total: 48.5ms	remaining: 760ms
6:	learn: 0.6251868	total: 55.9ms	remaining: 742ms
7:	learn: 0.6210870	total: 64.5ms	remaining: 742ms
8:	learn: 0.6177688	total: 73.2ms	remaining: 740ms
9:	learn: 0.6150431	total: 81.3ms	remaining: 732ms
10:	learn: 0.6128237	total: 89.8ms	remaining: 727ms
11:	learn: 0.6110231	total: 98ms	remaining: 719ms
12:	learn: 0.6094542	total: 107ms	remaining: 715ms
13:	learn: 0.6081960	total: 115ms	remaining: 706ms
14:	learn: 0.6071893	total: 124ms	remaining: 702ms
15:	learn: 0.6063015	total: 132ms	remaining: 696ms
16:	learn: 0.6056017	total: 140ms	remaining: 685ms
17:	learn: 0.6050391	total: 148ms	remaining: 676ms
18:	learn: 0.6045659	total: 157ms	remaining: 669ms
19:	learn: 0.6042078	total: 164

75:	learn: 0.6028835	total: 537ms	remaining: 170ms
76:	learn: 0.6028780	total: 541ms	remaining: 162ms
77:	learn: 0.6028811	total: 545ms	remaining: 154ms
78:	learn: 0.6028812	total: 549ms	remaining: 146ms
79:	learn: 0.6028746	total: 553ms	remaining: 138ms
80:	learn: 0.6028715	total: 556ms	remaining: 130ms
81:	learn: 0.6028750	total: 560ms	remaining: 123ms
82:	learn: 0.6028732	total: 565ms	remaining: 116ms
83:	learn: 0.6028788	total: 571ms	remaining: 109ms
84:	learn: 0.6028711	total: 576ms	remaining: 102ms
85:	learn: 0.6028724	total: 581ms	remaining: 94.7ms
86:	learn: 0.6028736	total: 587ms	remaining: 87.7ms
87:	learn: 0.6028738	total: 591ms	remaining: 80.7ms
88:	learn: 0.6028670	total: 597ms	remaining: 73.8ms
89:	learn: 0.6028749	total: 603ms	remaining: 67ms
90:	learn: 0.6028747	total: 609ms	remaining: 60.2ms
91:	learn: 0.6028745	total: 613ms	remaining: 53.3ms
92:	learn: 0.6028745	total: 618ms	remaining: 46.5ms
93:	learn: 0.6028677	total: 624ms	remaining: 39.8ms
94:	learn: 0.6028674	tot

56:	learn: 0.5990619	total: 361ms	remaining: 272ms
57:	learn: 0.5990598	total: 369ms	remaining: 267ms
58:	learn: 0.5990587	total: 377ms	remaining: 262ms
59:	learn: 0.5990534	total: 383ms	remaining: 256ms
60:	learn: 0.5990537	total: 390ms	remaining: 250ms
61:	learn: 0.5990518	total: 397ms	remaining: 244ms
62:	learn: 0.5990556	total: 406ms	remaining: 238ms
63:	learn: 0.5990531	total: 413ms	remaining: 232ms
64:	learn: 0.5990467	total: 420ms	remaining: 226ms
65:	learn: 0.5990462	total: 427ms	remaining: 220ms
66:	learn: 0.5990432	total: 433ms	remaining: 213ms
67:	learn: 0.5990469	total: 439ms	remaining: 207ms
68:	learn: 0.5990466	total: 448ms	remaining: 201ms
69:	learn: 0.5990396	total: 454ms	remaining: 195ms
70:	learn: 0.5990383	total: 462ms	remaining: 189ms
71:	learn: 0.5990419	total: 468ms	remaining: 182ms
72:	learn: 0.5990405	total: 473ms	remaining: 175ms
73:	learn: 0.5990397	total: 479ms	remaining: 168ms
74:	learn: 0.5990383	total: 483ms	remaining: 161ms
75:	learn: 0.5990326	total: 488

In [36]:
# Regular to Short

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['RtoS']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6440217	total: 7.95ms	remaining: 787ms
1:	learn: 0.6040765	total: 16.1ms	remaining: 789ms
2:	learn: 0.5710437	total: 23.6ms	remaining: 764ms
3:	learn: 0.5437291	total: 30.8ms	remaining: 739ms
4:	learn: 0.5206703	total: 38.2ms	remaining: 726ms
5:	learn: 0.5009527	total: 46.6ms	remaining: 730ms
6:	learn: 0.4846151	total: 53.6ms	remaining: 712ms
7:	learn: 0.4703589	total: 62.1ms	remaining: 714ms
8:	learn: 0.4581919	total: 69.4ms	remaining: 702ms
9:	learn: 0.4477497	total: 77.1ms	remaining: 694ms
10:	learn: 0.4390061	total: 84.2ms	remaining: 681ms
11:	learn: 0.4314498	total: 91.8ms	remaining: 673ms
12:	learn: 0.4251505	total: 98ms	remaining: 656ms
13:	learn: 0.4195702	total: 105ms	remaining: 645ms
14:	learn: 0.4148963	total: 112ms	remaining: 633ms
15:	learn: 0.4108126	total: 119ms	remaining: 623ms
16:	learn: 0.4072941	total: 126ms	remaining: 613ms
17:	learn: 0.4042542	total: 133ms	remaining: 606ms
18:	learn: 0.4017287	total: 140ms	remaining: 596ms
19:	learn: 0.3995560	total: 14

68:	learn: 0.3938073	total: 353ms	remaining: 159ms
69:	learn: 0.3938029	total: 357ms	remaining: 153ms
70:	learn: 0.3938012	total: 361ms	remaining: 148ms
71:	learn: 0.3938020	total: 366ms	remaining: 142ms
72:	learn: 0.3938014	total: 369ms	remaining: 137ms
73:	learn: 0.3937993	total: 374ms	remaining: 131ms
74:	learn: 0.3937947	total: 379ms	remaining: 126ms
75:	learn: 0.3937967	total: 383ms	remaining: 121ms
76:	learn: 0.3937915	total: 387ms	remaining: 116ms
77:	learn: 0.3937903	total: 391ms	remaining: 110ms
78:	learn: 0.3937909	total: 394ms	remaining: 105ms
79:	learn: 0.3937903	total: 398ms	remaining: 99.4ms
80:	learn: 0.3937885	total: 401ms	remaining: 94ms
81:	learn: 0.3937861	total: 405ms	remaining: 88.9ms
82:	learn: 0.3937867	total: 409ms	remaining: 83.7ms
83:	learn: 0.3937864	total: 413ms	remaining: 78.6ms
84:	learn: 0.3937843	total: 416ms	remaining: 73.4ms
85:	learn: 0.3937844	total: 420ms	remaining: 68.4ms
86:	learn: 0.3937838	total: 424ms	remaining: 63.4ms
87:	learn: 0.3937850	tota

60:	learn: 0.3722768	total: 372ms	remaining: 238ms
61:	learn: 0.3722800	total: 380ms	remaining: 233ms
62:	learn: 0.3722771	total: 384ms	remaining: 226ms
63:	learn: 0.3722710	total: 389ms	remaining: 219ms
64:	learn: 0.3722713	total: 396ms	remaining: 213ms
65:	learn: 0.3722636	total: 402ms	remaining: 207ms
66:	learn: 0.3722622	total: 408ms	remaining: 201ms
67:	learn: 0.3722607	total: 413ms	remaining: 194ms
68:	learn: 0.3722590	total: 417ms	remaining: 187ms
69:	learn: 0.3722545	total: 424ms	remaining: 182ms
70:	learn: 0.3722552	total: 431ms	remaining: 176ms
71:	learn: 0.3722507	total: 436ms	remaining: 170ms
72:	learn: 0.3722487	total: 443ms	remaining: 164ms
73:	learn: 0.3722484	total: 449ms	remaining: 158ms
74:	learn: 0.3722454	total: 456ms	remaining: 152ms
75:	learn: 0.3722462	total: 463ms	remaining: 146ms
76:	learn: 0.3722438	total: 469ms	remaining: 140ms
77:	learn: 0.3722419	total: 475ms	remaining: 134ms
78:	learn: 0.3722463	total: 480ms	remaining: 128ms
79:	learn: 0.3722414	total: 486

31:	learn: 0.3945576	total: 187ms	remaining: 397ms
32:	learn: 0.3944058	total: 194ms	remaining: 394ms
33:	learn: 0.3941946	total: 200ms	remaining: 388ms
34:	learn: 0.3940262	total: 203ms	remaining: 378ms
35:	learn: 0.3938686	total: 208ms	remaining: 371ms
36:	learn: 0.3937334	total: 213ms	remaining: 364ms
37:	learn: 0.3936099	total: 218ms	remaining: 356ms
38:	learn: 0.3935243	total: 223ms	remaining: 349ms
39:	learn: 0.3934385	total: 229ms	remaining: 343ms
40:	learn: 0.3933660	total: 234ms	remaining: 336ms
41:	learn: 0.3933320	total: 240ms	remaining: 331ms
42:	learn: 0.3932811	total: 247ms	remaining: 327ms
43:	learn: 0.3932629	total: 252ms	remaining: 320ms
44:	learn: 0.3932449	total: 258ms	remaining: 315ms
45:	learn: 0.3932053	total: 265ms	remaining: 311ms
46:	learn: 0.3931759	total: 271ms	remaining: 306ms
47:	learn: 0.3931547	total: 278ms	remaining: 302ms
48:	learn: 0.3931316	total: 285ms	remaining: 297ms
49:	learn: 0.3931260	total: 289ms	remaining: 289ms
50:	learn: 0.3931095	total: 294

0:	learn: 0.6449287	total: 6.58ms	remaining: 652ms
1:	learn: 0.6057169	total: 13.1ms	remaining: 643ms
2:	learn: 0.5732825	total: 19.1ms	remaining: 619ms
3:	learn: 0.5464875	total: 25.1ms	remaining: 603ms
4:	learn: 0.5238586	total: 32.9ms	remaining: 624ms
5:	learn: 0.5045190	total: 39.9ms	remaining: 625ms
6:	learn: 0.4884921	total: 45.8ms	remaining: 608ms
7:	learn: 0.4745093	total: 52.2ms	remaining: 600ms
8:	learn: 0.4625860	total: 59ms	remaining: 596ms
9:	learn: 0.4523377	total: 65.8ms	remaining: 592ms
10:	learn: 0.4437520	total: 72.3ms	remaining: 585ms
11:	learn: 0.4363460	total: 79.1ms	remaining: 580ms
12:	learn: 0.4301734	total: 84.5ms	remaining: 565ms
13:	learn: 0.4246924	total: 91ms	remaining: 559ms
14:	learn: 0.4201130	total: 97.4ms	remaining: 552ms
15:	learn: 0.4161062	total: 104ms	remaining: 548ms
16:	learn: 0.4126622	total: 111ms	remaining: 542ms
17:	learn: 0.4096791	total: 117ms	remaining: 535ms
18:	learn: 0.4071880	total: 124ms	remaining: 529ms
19:	learn: 0.4050668	total: 13

...

61:	learn: 0.3819899	total: 366ms	remaining: 224ms
62:	learn: 0.3819883	total: 370ms	remaining: 218ms
63:	learn: 0.3819901	total: 377ms	remaining: 212ms
64:	learn: 0.3819875	total: 383ms	remaining: 206ms
65:	learn: 0.3819841	total: 388ms	remaining: 200ms
66:	learn: 0.3819767	total: 394ms	remaining: 194ms
67:	learn: 0.3819790	total: 400ms	remaining: 188ms
68:	learn: 0.3819704	total: 405ms	remaining: 182ms
69:	learn: 0.3819758	total: 410ms	remaining: 176ms
70:	learn: 0.3819743	total: 417ms	remaining: 170ms
71:	learn: 0.3819693	total: 422ms	remaining: 164ms
72:	learn: 0.3819677	total: 427ms	remaining: 158ms
73:	learn: 0.3819640	total: 433ms	remaining: 152ms
74:	learn: 0.3819634	total: 437ms	remaining: 146ms
75:	learn: 0.3819615	total: 443ms	remaining: 140ms
76:	learn: 0.3819600	total: 448ms	remaining: 134ms
77:	learn: 0.3819596	total: 453ms	remaining: 128ms
78:	learn: 0.3819559	total: 459ms	remaining: 122ms
79:	learn: 0.3819565	total: 463ms	remaining: 116ms
80:	learn: 0.3819538	total: 468

38:	learn: 0.3885212	total: 384ms	remaining: 600ms
39:	learn: 0.3884295	total: 394ms	remaining: 591ms
40:	learn: 0.3883599	total: 403ms	remaining: 580ms
41:	learn: 0.3883248	total: 414ms	remaining: 571ms
42:	learn: 0.3883030	total: 423ms	remaining: 561ms
43:	learn: 0.3882761	total: 433ms	remaining: 551ms
44:	learn: 0.3882621	total: 439ms	remaining: 537ms
45:	learn: 0.3882132	total: 449ms	remaining: 527ms
46:	learn: 0.3881688	total: 458ms	remaining: 516ms
47:	learn: 0.3881597	total: 468ms	remaining: 507ms
48:	learn: 0.3881245	total: 478ms	remaining: 497ms
49:	learn: 0.3880968	total: 488ms	remaining: 488ms
50:	learn: 0.3880747	total: 496ms	remaining: 477ms
51:	learn: 0.3880611	total: 504ms	remaining: 465ms
52:	learn: 0.3880422	total: 514ms	remaining: 456ms
53:	learn: 0.3880268	total: 523ms	remaining: 446ms
54:	learn: 0.3880170	total: 533ms	remaining: 436ms
55:	learn: 0.3880145	total: 542ms	remaining: 426ms
56:	learn: 0.3880115	total: 551ms	remaining: 416ms
57:	learn: 0.3880023	total: 559

0:	learn: 0.6445101	total: 9.08ms	remaining: 899ms
1:	learn: 0.6049758	total: 16.4ms	remaining: 803ms
2:	learn: 0.5722889	total: 23.1ms	remaining: 748ms
3:	learn: 0.5452663	total: 29.8ms	remaining: 716ms
4:	learn: 0.5224283	total: 36.9ms	remaining: 701ms
5:	learn: 0.5029041	total: 44ms	remaining: 689ms
6:	learn: 0.4867201	total: 49.9ms	remaining: 663ms
7:	learn: 0.4726136	total: 56.8ms	remaining: 654ms
8:	learn: 0.4605543	total: 63.8ms	remaining: 645ms
9:	learn: 0.4501921	total: 70.9ms	remaining: 638ms
10:	learn: 0.4414944	total: 77.9ms	remaining: 631ms
11:	learn: 0.4339831	total: 85.6ms	remaining: 627ms
12:	learn: 0.4277195	total: 91.5ms	remaining: 613ms
13:	learn: 0.4221583	total: 98.6ms	remaining: 606ms
14:	learn: 0.4175152	total: 106ms	remaining: 598ms
15:	learn: 0.4134348	total: 113ms	remaining: 591ms
16:	learn: 0.4099236	total: 119ms	remaining: 582ms
17:	learn: 0.4068886	total: 126ms	remaining: 575ms
18:	learn: 0.4043488	total: 133ms	remaining: 567ms
19:	learn: 0.4021725	total: 1

67:	learn: 0.3902701	total: 370ms	remaining: 174ms
68:	learn: 0.3902675	total: 374ms	remaining: 168ms
69:	learn: 0.3902664	total: 378ms	remaining: 162ms
70:	learn: 0.3902665	total: 381ms	remaining: 156ms
71:	learn: 0.3902651	total: 387ms	remaining: 150ms
72:	learn: 0.3902600	total: 393ms	remaining: 145ms
73:	learn: 0.3902615	total: 397ms	remaining: 139ms
74:	learn: 0.3902625	total: 403ms	remaining: 134ms
75:	learn: 0.3902593	total: 408ms	remaining: 129ms
76:	learn: 0.3902598	total: 412ms	remaining: 123ms
77:	learn: 0.3902578	total: 415ms	remaining: 117ms
78:	learn: 0.3902569	total: 419ms	remaining: 111ms
79:	learn: 0.3902582	total: 424ms	remaining: 106ms
80:	learn: 0.3902589	total: 428ms	remaining: 100ms
81:	learn: 0.3902568	total: 431ms	remaining: 94.7ms
82:	learn: 0.3902527	total: 435ms	remaining: 89.1ms
83:	learn: 0.3902537	total: 439ms	remaining: 83.6ms
84:	learn: 0.3902533	total: 443ms	remaining: 78.2ms
85:	learn: 0.3902503	total: 446ms	remaining: 72.7ms
86:	learn: 0.3902516	total

32:	learn: 0.3910897	total: 219ms	remaining: 446ms
33:	learn: 0.3908065	total: 227ms	remaining: 440ms
34:	learn: 0.3905652	total: 234ms	remaining: 434ms
35:	learn: 0.3903872	total: 240ms	remaining: 427ms
36:	learn: 0.3902401	total: 245ms	remaining: 418ms
37:	learn: 0.3901033	total: 252ms	remaining: 412ms
38:	learn: 0.3899877	total: 259ms	remaining: 405ms
39:	learn: 0.3898864	total: 265ms	remaining: 398ms
40:	learn: 0.3898123	total: 271ms	remaining: 390ms
41:	learn: 0.3897306	total: 278ms	remaining: 384ms
42:	learn: 0.3896631	total: 285ms	remaining: 378ms
43:	learn: 0.3896324	total: 291ms	remaining: 371ms
44:	learn: 0.3895860	total: 299ms	remaining: 365ms
45:	learn: 0.3895747	total: 304ms	remaining: 356ms
46:	learn: 0.3895642	total: 310ms	remaining: 349ms
47:	learn: 0.3895302	total: 317ms	remaining: 343ms
48:	learn: 0.3895063	total: 323ms	remaining: 336ms
49:	learn: 0.3894956	total: 330ms	remaining: 330ms
50:	learn: 0.3894753	total: 336ms	remaining: 323ms
51:	learn: 0.3894648	total: 341

In [37]:
# Regular to Regular

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['RtoR']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6285786	total: 8.55ms	remaining: 846ms
1:	learn: 0.5762462	total: 16.4ms	remaining: 803ms
2:	learn: 0.5324625	total: 25.3ms	remaining: 817ms
3:	learn: 0.4958510	total: 32.7ms	remaining: 785ms
4:	learn: 0.4642428	total: 40.6ms	remaining: 772ms
5:	learn: 0.4376212	total: 47.9ms	remaining: 750ms
6:	learn: 0.4150022	total: 55.3ms	remaining: 734ms
7:	learn: 0.3955792	total: 62.6ms	remaining: 720ms
8:	learn: 0.3787091	total: 70.6ms	remaining: 714ms
9:	learn: 0.3640530	total: 78ms	remaining: 702ms
10:	learn: 0.3509889	total: 85.9ms	remaining: 695ms
11:	learn: 0.3399779	total: 92.6ms	remaining: 679ms
12:	learn: 0.3302313	total: 101ms	remaining: 679ms
13:	learn: 0.3214791	total: 111ms	remaining: 680ms
14:	learn: 0.3141095	total: 119ms	remaining: 674ms
15:	learn: 0.3078808	total: 127ms	remaining: 665ms
16:	learn: 0.3018291	total: 135ms	remaining: 658ms
17:	learn: 0.2974584	total: 143ms	remaining: 651ms
18:	learn: 0.2927417	total: 151ms	remaining: 646ms
19:	learn: 0.2887694	total: 159

83:	learn: 0.2603792	total: 541ms	remaining: 103ms
84:	learn: 0.2603778	total: 548ms	remaining: 96.6ms
85:	learn: 0.2603708	total: 555ms	remaining: 90.3ms
86:	learn: 0.2603533	total: 563ms	remaining: 84.1ms
87:	learn: 0.2603507	total: 570ms	remaining: 77.7ms
88:	learn: 0.2603348	total: 577ms	remaining: 71.3ms
89:	learn: 0.2603231	total: 583ms	remaining: 64.8ms
90:	learn: 0.2603148	total: 589ms	remaining: 58.3ms
91:	learn: 0.2603068	total: 596ms	remaining: 51.8ms
92:	learn: 0.2603002	total: 603ms	remaining: 45.4ms
93:	learn: 0.2602920	total: 610ms	remaining: 38.9ms
94:	learn: 0.2602908	total: 616ms	remaining: 32.4ms
95:	learn: 0.2602854	total: 623ms	remaining: 25.9ms
96:	learn: 0.2602850	total: 628ms	remaining: 19.4ms
97:	learn: 0.2602849	total: 633ms	remaining: 12.9ms
98:	learn: 0.2602821	total: 640ms	remaining: 6.46ms
99:	learn: 0.2602771	total: 646ms	remaining: 0us
0:	learn: 0.6300022	total: 11.2ms	remaining: 1.11s
1:	learn: 0.5784158	total: 22.2ms	remaining: 1.09s
2:	learn: 0.535299

58:	learn: 0.2532973	total: 546ms	remaining: 380ms
59:	learn: 0.2532273	total: 556ms	remaining: 370ms
60:	learn: 0.2531605	total: 565ms	remaining: 361ms
61:	learn: 0.2530984	total: 575ms	remaining: 352ms
62:	learn: 0.2530449	total: 583ms	remaining: 342ms
63:	learn: 0.2529954	total: 590ms	remaining: 332ms
64:	learn: 0.2529885	total: 598ms	remaining: 322ms
65:	learn: 0.2529441	total: 608ms	remaining: 313ms
66:	learn: 0.2529428	total: 615ms	remaining: 303ms
67:	learn: 0.2529003	total: 624ms	remaining: 294ms
68:	learn: 0.2528793	total: 633ms	remaining: 284ms
69:	learn: 0.2528502	total: 642ms	remaining: 275ms
70:	learn: 0.2528482	total: 649ms	remaining: 265ms
71:	learn: 0.2528161	total: 658ms	remaining: 256ms
72:	learn: 0.2527895	total: 666ms	remaining: 246ms
73:	learn: 0.2527844	total: 676ms	remaining: 238ms
74:	learn: 0.2527831	total: 686ms	remaining: 229ms
75:	learn: 0.2527797	total: 695ms	remaining: 220ms
76:	learn: 0.2527664	total: 704ms	remaining: 210ms
77:	learn: 0.2527469	total: 714

25:	learn: 0.2730305	total: 192ms	remaining: 545ms
26:	learn: 0.2712168	total: 199ms	remaining: 538ms
27:	learn: 0.2700000	total: 207ms	remaining: 531ms
28:	learn: 0.2685407	total: 215ms	remaining: 526ms
29:	learn: 0.2672875	total: 222ms	remaining: 519ms
30:	learn: 0.2662221	total: 229ms	remaining: 510ms
31:	learn: 0.2652391	total: 236ms	remaining: 502ms
32:	learn: 0.2643814	total: 243ms	remaining: 494ms
33:	learn: 0.2635936	total: 251ms	remaining: 486ms
34:	learn: 0.2629370	total: 257ms	remaining: 478ms
35:	learn: 0.2623106	total: 264ms	remaining: 469ms
36:	learn: 0.2616969	total: 271ms	remaining: 461ms
37:	learn: 0.2612382	total: 279ms	remaining: 455ms
38:	learn: 0.2607826	total: 286ms	remaining: 448ms
39:	learn: 0.2603377	total: 293ms	remaining: 440ms
40:	learn: 0.2601310	total: 300ms	remaining: 432ms
41:	learn: 0.2597437	total: 307ms	remaining: 424ms
42:	learn: 0.2594448	total: 314ms	remaining: 416ms
43:	learn: 0.2591465	total: 321ms	remaining: 408ms
44:	learn: 0.2588758	total: 328

86:	learn: 0.2544487	total: 547ms	remaining: 81.7ms
87:	learn: 0.2544475	total: 554ms	remaining: 75.5ms
88:	learn: 0.2544340	total: 561ms	remaining: 69.3ms
89:	learn: 0.2544297	total: 567ms	remaining: 63.1ms
90:	learn: 0.2544202	total: 575ms	remaining: 56.8ms
91:	learn: 0.2544105	total: 580ms	remaining: 50.4ms
92:	learn: 0.2544038	total: 586ms	remaining: 44.1ms
93:	learn: 0.2543949	total: 593ms	remaining: 37.8ms
94:	learn: 0.2543925	total: 599ms	remaining: 31.5ms
95:	learn: 0.2543841	total: 605ms	remaining: 25.2ms
96:	learn: 0.2543844	total: 611ms	remaining: 18.9ms
97:	learn: 0.2543798	total: 617ms	remaining: 12.6ms
98:	learn: 0.2543800	total: 622ms	remaining: 6.28ms
99:	learn: 0.2543786	total: 627ms	remaining: 0us
0:	learn: 0.6296130	total: 10.4ms	remaining: 1.03s
1:	learn: 0.5777171	total: 20.5ms	remaining: 1s
2:	learn: 0.5343220	total: 31.1ms	remaining: 1s
3:	learn: 0.4980398	total: 41.4ms	remaining: 993ms
4:	learn: 0.4668566	total: 51.7ms	remaining: 983ms
5:	learn: 0.4405062	total:

...

62:	learn: 0.2623816	total: 370ms	remaining: 217ms
63:	learn: 0.2623238	total: 375ms	remaining: 211ms
64:	learn: 0.2623153	total: 380ms	remaining: 205ms
65:	learn: 0.2622674	total: 385ms	remaining: 198ms
66:	learn: 0.2622640	total: 390ms	remaining: 192ms
67:	learn: 0.2622328	total: 394ms	remaining: 185ms
68:	learn: 0.2621938	total: 399ms	remaining: 179ms
69:	learn: 0.2621611	total: 403ms	remaining: 173ms
70:	learn: 0.2621509	total: 408ms	remaining: 166ms
71:	learn: 0.2621219	total: 412ms	remaining: 160ms
72:	learn: 0.2621016	total: 417ms	remaining: 154ms
73:	learn: 0.2620748	total: 421ms	remaining: 148ms
74:	learn: 0.2620462	total: 426ms	remaining: 142ms
75:	learn: 0.2620303	total: 430ms	remaining: 136ms
76:	learn: 0.2620279	total: 435ms	remaining: 130ms
77:	learn: 0.2620045	total: 439ms	remaining: 124ms
78:	learn: 0.2619931	total: 444ms	remaining: 118ms
79:	learn: 0.2619765	total: 448ms	remaining: 112ms
80:	learn: 0.2619606	total: 453ms	remaining: 106ms
81:	learn: 0.2619589	total: 457

27:	learn: 0.2696214	total: 191ms	remaining: 492ms
28:	learn: 0.2683287	total: 198ms	remaining: 485ms
29:	learn: 0.2670722	total: 205ms	remaining: 479ms
30:	learn: 0.2658585	total: 212ms	remaining: 472ms
31:	learn: 0.2648202	total: 219ms	remaining: 466ms
32:	learn: 0.2639071	total: 226ms	remaining: 458ms
33:	learn: 0.2630724	total: 232ms	remaining: 450ms
34:	learn: 0.2623580	total: 239ms	remaining: 444ms
35:	learn: 0.2616941	total: 246ms	remaining: 437ms
36:	learn: 0.2610931	total: 252ms	remaining: 429ms
37:	learn: 0.2606311	total: 258ms	remaining: 421ms
38:	learn: 0.2601508	total: 265ms	remaining: 415ms
39:	learn: 0.2597684	total: 271ms	remaining: 407ms
40:	learn: 0.2593962	total: 278ms	remaining: 401ms
41:	learn: 0.2590501	total: 285ms	remaining: 394ms
42:	learn: 0.2587823	total: 292ms	remaining: 387ms
43:	learn: 0.2584913	total: 298ms	remaining: 379ms
44:	learn: 0.2582943	total: 304ms	remaining: 372ms
45:	learn: 0.2580354	total: 311ms	remaining: 365ms
46:	learn: 0.2578297	total: 317

0:	learn: 0.6293552	total: 12.7ms	remaining: 1.26s
1:	learn: 0.5772710	total: 23.5ms	remaining: 1.15s
2:	learn: 0.5337020	total: 33.9ms	remaining: 1.09s
3:	learn: 0.4970209	total: 44.2ms	remaining: 1.06s
4:	learn: 0.4655590	total: 56ms	remaining: 1.06s
5:	learn: 0.4391168	total: 67ms	remaining: 1.05s
6:	learn: 0.4165856	total: 78ms	remaining: 1.03s
7:	learn: 0.3970317	total: 88.5ms	remaining: 1.02s
8:	learn: 0.3802925	total: 100ms	remaining: 1.01s
9:	learn: 0.3658555	total: 112ms	remaining: 1s
10:	learn: 0.3528701	total: 122ms	remaining: 991ms
11:	learn: 0.3418649	total: 140ms	remaining: 1.03s
12:	learn: 0.3322420	total: 154ms	remaining: 1.03s
13:	learn: 0.3236215	total: 165ms	remaining: 1.02s
14:	learn: 0.3162789	total: 177ms	remaining: 1s
15:	learn: 0.3097749	total: 187ms	remaining: 982ms
16:	learn: 0.3039712	total: 199ms	remaining: 974ms
17:	learn: 0.2987635	total: 212ms	remaining: 967ms
18:	learn: 0.2942278	total: 224ms	remaining: 953ms
19:	learn: 0.2906341	total: 233ms	remaining: 

71:	learn: 0.2642613	total: 549ms	remaining: 213ms
72:	learn: 0.2642296	total: 556ms	remaining: 206ms
73:	learn: 0.2641992	total: 564ms	remaining: 198ms
74:	learn: 0.2641985	total: 570ms	remaining: 190ms
75:	learn: 0.2641720	total: 578ms	remaining: 183ms
76:	learn: 0.2641501	total: 586ms	remaining: 175ms
77:	learn: 0.2641437	total: 594ms	remaining: 168ms
78:	learn: 0.2641273	total: 602ms	remaining: 160ms
79:	learn: 0.2641085	total: 610ms	remaining: 153ms
80:	learn: 0.2640966	total: 617ms	remaining: 145ms
81:	learn: 0.2640933	total: 626ms	remaining: 137ms
82:	learn: 0.2640917	total: 633ms	remaining: 130ms
83:	learn: 0.2640798	total: 641ms	remaining: 122ms
84:	learn: 0.2640654	total: 648ms	remaining: 114ms
85:	learn: 0.2640662	total: 654ms	remaining: 106ms
86:	learn: 0.2640554	total: 659ms	remaining: 98.5ms
87:	learn: 0.2640533	total: 667ms	remaining: 90.9ms
88:	learn: 0.2640500	total: 675ms	remaining: 83.4ms
89:	learn: 0.2640415	total: 681ms	remaining: 75.6ms
90:	learn: 0.2640323	total:

55:	learn: 0.2654712	total: 371ms	remaining: 292ms
56:	learn: 0.2654177	total: 379ms	remaining: 286ms
57:	learn: 0.2654131	total: 383ms	remaining: 277ms
58:	learn: 0.2653317	total: 390ms	remaining: 271ms
59:	learn: 0.2652464	total: 397ms	remaining: 265ms
60:	learn: 0.2652368	total: 404ms	remaining: 258ms
61:	learn: 0.2651649	total: 411ms	remaining: 252ms
62:	learn: 0.2651063	total: 417ms	remaining: 245ms
63:	learn: 0.2650460	total: 423ms	remaining: 238ms
64:	learn: 0.2650374	total: 430ms	remaining: 231ms
65:	learn: 0.2649872	total: 436ms	remaining: 224ms
66:	learn: 0.2649828	total: 442ms	remaining: 218ms
67:	learn: 0.2649544	total: 449ms	remaining: 211ms
68:	learn: 0.2649126	total: 455ms	remaining: 204ms
69:	learn: 0.2648811	total: 461ms	remaining: 198ms
70:	learn: 0.2648675	total: 467ms	remaining: 191ms
71:	learn: 0.2648340	total: 474ms	remaining: 184ms
72:	learn: 0.2648138	total: 480ms	remaining: 178ms
73:	learn: 0.2647957	total: 487ms	remaining: 171ms
74:	learn: 0.2647648	total: 493

In [38]:
# Regular to Long

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['RtoL']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6315778	total: 5.98ms	remaining: 592ms
1:	learn: 0.5811871	total: 11ms	remaining: 537ms
2:	learn: 0.5392146	total: 16.5ms	remaining: 532ms
3:	learn: 0.5046022	total: 21.8ms	remaining: 523ms
4:	learn: 0.4756565	total: 26.3ms	remaining: 500ms
5:	learn: 0.4507038	total: 31.8ms	remaining: 498ms
6:	learn: 0.4295342	total: 37.1ms	remaining: 493ms
7:	learn: 0.4115691	total: 43.3ms	remaining: 498ms
8:	learn: 0.3962381	total: 49ms	remaining: 495ms
9:	learn: 0.3831699	total: 57.1ms	remaining: 514ms
10:	learn: 0.3720092	total: 62.9ms	remaining: 509ms
11:	learn: 0.3624211	total: 69.4ms	remaining: 509ms
12:	learn: 0.3542008	total: 76.2ms	remaining: 510ms
13:	learn: 0.3471622	total: 82.7ms	remaining: 508ms
14:	learn: 0.3411598	total: 88.6ms	remaining: 502ms
15:	learn: 0.3360234	total: 93.9ms	remaining: 493ms
16:	learn: 0.3315921	total: 99.5ms	remaining: 486ms
17:	learn: 0.3278303	total: 105ms	remaining: 477ms
18:	learn: 0.3246263	total: 110ms	remaining: 469ms
19:	learn: 0.3218745	total: 

69:	learn: 0.3142312	total: 542ms	remaining: 232ms
70:	learn: 0.3142339	total: 548ms	remaining: 224ms
71:	learn: 0.3142311	total: 555ms	remaining: 216ms
72:	learn: 0.3142312	total: 563ms	remaining: 208ms
73:	learn: 0.3142299	total: 569ms	remaining: 200ms
74:	learn: 0.3142288	total: 576ms	remaining: 192ms
75:	learn: 0.3142313	total: 584ms	remaining: 184ms
76:	learn: 0.3142318	total: 592ms	remaining: 177ms
77:	learn: 0.3142304	total: 599ms	remaining: 169ms
78:	learn: 0.3142293	total: 607ms	remaining: 161ms
79:	learn: 0.3142267	total: 615ms	remaining: 154ms
80:	learn: 0.3142291	total: 623ms	remaining: 146ms
81:	learn: 0.3142288	total: 631ms	remaining: 139ms
82:	learn: 0.3142266	total: 639ms	remaining: 131ms
83:	learn: 0.3142269	total: 646ms	remaining: 123ms
84:	learn: 0.3142264	total: 652ms	remaining: 115ms
85:	learn: 0.3142267	total: 658ms	remaining: 107ms
86:	learn: 0.3142290	total: 666ms	remaining: 99.5ms
87:	learn: 0.3142285	total: 672ms	remaining: 91.7ms
88:	learn: 0.3142233	total: 6

32:	learn: 0.3116535	total: 190ms	remaining: 386ms
33:	learn: 0.3114136	total: 197ms	remaining: 382ms
34:	learn: 0.3111787	total: 203ms	remaining: 376ms
35:	learn: 0.3110389	total: 208ms	remaining: 369ms
36:	learn: 0.3108551	total: 213ms	remaining: 362ms
37:	learn: 0.3106831	total: 218ms	remaining: 356ms
38:	learn: 0.3105362	total: 224ms	remaining: 351ms
39:	learn: 0.3104330	total: 229ms	remaining: 343ms
40:	learn: 0.3103704	total: 233ms	remaining: 336ms
41:	learn: 0.3103188	total: 238ms	remaining: 328ms
42:	learn: 0.3102576	total: 244ms	remaining: 323ms
43:	learn: 0.3102221	total: 248ms	remaining: 316ms
44:	learn: 0.3101555	total: 254ms	remaining: 311ms
45:	learn: 0.3100981	total: 260ms	remaining: 305ms
46:	learn: 0.3100535	total: 265ms	remaining: 299ms
47:	learn: 0.3100157	total: 271ms	remaining: 294ms
48:	learn: 0.3099809	total: 277ms	remaining: 288ms
49:	learn: 0.3099723	total: 282ms	remaining: 282ms
50:	learn: 0.3099462	total: 288ms	remaining: 277ms
51:	learn: 0.3099242	total: 293

0:	learn: 0.6326685	total: 9.38ms	remaining: 929ms
1:	learn: 0.5831929	total: 17.3ms	remaining: 848ms
2:	learn: 0.5420128	total: 26.7ms	remaining: 863ms
3:	learn: 0.5080737	total: 34.9ms	remaining: 837ms
4:	learn: 0.4796825	total: 42.8ms	remaining: 813ms
5:	learn: 0.4552741	total: 51.9ms	remaining: 813ms
6:	learn: 0.4345857	total: 60.2ms	remaining: 799ms
7:	learn: 0.4170681	total: 69.6ms	remaining: 801ms
8:	learn: 0.4021209	total: 78.4ms	remaining: 792ms
9:	learn: 0.3894193	total: 88.1ms	remaining: 793ms
10:	learn: 0.3785807	total: 96.5ms	remaining: 781ms
11:	learn: 0.3693125	total: 104ms	remaining: 766ms
12:	learn: 0.3613875	total: 113ms	remaining: 759ms
13:	learn: 0.3546012	total: 122ms	remaining: 750ms
14:	learn: 0.3488240	total: 131ms	remaining: 742ms
15:	learn: 0.3439207	total: 139ms	remaining: 730ms
16:	learn: 0.3397157	total: 148ms	remaining: 721ms
17:	learn: 0.3361138	total: 156ms	remaining: 709ms
18:	learn: 0.3330898	total: 164ms	remaining: 700ms
19:	learn: 0.3305042	total: 17

67:	learn: 0.2994027	total: 558ms	remaining: 263ms
68:	learn: 0.2994025	total: 563ms	remaining: 253ms
69:	learn: 0.2993959	total: 566ms	remaining: 243ms
70:	learn: 0.2993981	total: 569ms	remaining: 232ms
71:	learn: 0.2993928	total: 573ms	remaining: 223ms
72:	learn: 0.2993930	total: 577ms	remaining: 213ms
73:	learn: 0.2993917	total: 581ms	remaining: 204ms
74:	learn: 0.2993948	total: 585ms	remaining: 195ms
75:	learn: 0.2993894	total: 590ms	remaining: 186ms
76:	learn: 0.2993863	total: 596ms	remaining: 178ms
77:	learn: 0.2993925	total: 601ms	remaining: 170ms
78:	learn: 0.2993922	total: 605ms	remaining: 161ms
79:	learn: 0.2993899	total: 611ms	remaining: 153ms
80:	learn: 0.2993905	total: 617ms	remaining: 145ms
81:	learn: 0.2993881	total: 623ms	remaining: 137ms
82:	learn: 0.2993917	total: 629ms	remaining: 129ms
83:	learn: 0.2993893	total: 633ms	remaining: 121ms
84:	learn: 0.2993856	total: 638ms	remaining: 113ms
85:	learn: 0.2993874	total: 643ms	remaining: 105ms
86:	learn: 0.2993888	total: 649

...

89:	learn: 0.2713888	total: 534ms	remaining: 59.3ms
90:	learn: 0.2713850	total: 540ms	remaining: 53.4ms
91:	learn: 0.2713893	total: 546ms	remaining: 47.5ms
92:	learn: 0.2713871	total: 551ms	remaining: 41.5ms
93:	learn: 0.2713857	total: 557ms	remaining: 35.5ms
94:	learn: 0.2713857	total: 563ms	remaining: 29.6ms
95:	learn: 0.2713871	total: 569ms	remaining: 23.7ms
96:	learn: 0.2713873	total: 574ms	remaining: 17.8ms
97:	learn: 0.2713870	total: 579ms	remaining: 11.8ms
98:	learn: 0.2713898	total: 584ms	remaining: 5.9ms
99:	learn: 0.2713852	total: 590ms	remaining: 0us
0:	learn: 0.6317773	total: 5.77ms	remaining: 571ms
1:	learn: 0.5815700	total: 10.6ms	remaining: 521ms
2:	learn: 0.5397198	total: 16.1ms	remaining: 521ms
3:	learn: 0.5048551	total: 22.3ms	remaining: 536ms
4:	learn: 0.4755444	total: 28.1ms	remaining: 533ms
5:	learn: 0.4512588	total: 33ms	remaining: 516ms
6:	learn: 0.4307329	total: 37.3ms	remaining: 496ms
7:	learn: 0.4128427	total: 42.5ms	remaining: 489ms
8:	learn: 0.3975987	total:

78:	learn: 0.3142781	total: 567ms	remaining: 151ms
79:	learn: 0.3142767	total: 573ms	remaining: 143ms
80:	learn: 0.3142803	total: 577ms	remaining: 135ms
81:	learn: 0.3142837	total: 581ms	remaining: 128ms
82:	learn: 0.3142789	total: 585ms	remaining: 120ms
83:	learn: 0.3142743	total: 590ms	remaining: 112ms
84:	learn: 0.3142794	total: 594ms	remaining: 105ms
85:	learn: 0.3142761	total: 599ms	remaining: 97.4ms
86:	learn: 0.3142764	total: 605ms	remaining: 90.3ms
87:	learn: 0.3142786	total: 610ms	remaining: 83.2ms
88:	learn: 0.3142725	total: 616ms	remaining: 76.2ms
89:	learn: 0.3142718	total: 622ms	remaining: 69.1ms
90:	learn: 0.3142724	total: 626ms	remaining: 62ms
91:	learn: 0.3142745	total: 631ms	remaining: 54.9ms
92:	learn: 0.3142785	total: 637ms	remaining: 47.9ms
93:	learn: 0.3142777	total: 642ms	remaining: 41ms
94:	learn: 0.3142739	total: 647ms	remaining: 34ms
95:	learn: 0.3142760	total: 653ms	remaining: 27.2ms
96:	learn: 0.3142749	total: 658ms	remaining: 20.4ms
97:	learn: 0.3142765	tota

48:	learn: 0.3037465	total: 372ms	remaining: 387ms
49:	learn: 0.3037363	total: 380ms	remaining: 380ms
50:	learn: 0.3037161	total: 389ms	remaining: 373ms
51:	learn: 0.3036951	total: 396ms	remaining: 365ms
52:	learn: 0.3036707	total: 404ms	remaining: 358ms
53:	learn: 0.3036547	total: 411ms	remaining: 350ms
54:	learn: 0.3036511	total: 418ms	remaining: 342ms
55:	learn: 0.3036397	total: 426ms	remaining: 335ms
56:	learn: 0.3036384	total: 432ms	remaining: 326ms
57:	learn: 0.3036357	total: 439ms	remaining: 318ms
58:	learn: 0.3036318	total: 446ms	remaining: 310ms
59:	learn: 0.3036291	total: 452ms	remaining: 301ms
60:	learn: 0.3036246	total: 458ms	remaining: 293ms
61:	learn: 0.3036229	total: 464ms	remaining: 284ms
62:	learn: 0.3036175	total: 472ms	remaining: 277ms
63:	learn: 0.3036174	total: 479ms	remaining: 270ms
64:	learn: 0.3036141	total: 485ms	remaining: 261ms
65:	learn: 0.3036165	total: 491ms	remaining: 253ms
66:	learn: 0.3036149	total: 498ms	remaining: 245ms
67:	learn: 0.3036091	total: 506

23:	learn: 0.3228085	total: 194ms	remaining: 614ms
24:	learn: 0.3216465	total: 202ms	remaining: 605ms
25:	learn: 0.3206688	total: 211ms	remaining: 600ms
26:	learn: 0.3200188	total: 218ms	remaining: 590ms
27:	learn: 0.3193027	total: 227ms	remaining: 584ms
28:	learn: 0.3188624	total: 234ms	remaining: 574ms
29:	learn: 0.3183419	total: 243ms	remaining: 566ms
30:	learn: 0.3178982	total: 250ms	remaining: 557ms
31:	learn: 0.3175293	total: 259ms	remaining: 549ms
32:	learn: 0.3172580	total: 264ms	remaining: 537ms
33:	learn: 0.3170296	total: 272ms	remaining: 529ms
34:	learn: 0.3168282	total: 279ms	remaining: 519ms
35:	learn: 0.3166933	total: 285ms	remaining: 507ms
36:	learn: 0.3165445	total: 292ms	remaining: 496ms
37:	learn: 0.3163958	total: 298ms	remaining: 487ms
38:	learn: 0.3162709	total: 306ms	remaining: 479ms
39:	learn: 0.3161867	total: 312ms	remaining: 469ms
40:	learn: 0.3161222	total: 318ms	remaining: 458ms
41:	learn: 0.3160746	total: 324ms	remaining: 448ms
42:	learn: 0.3160142	total: 332

0:	learn: 0.6325324	total: 6.18ms	remaining: 612ms
1:	learn: 0.5829396	total: 12ms	remaining: 587ms
2:	learn: 0.5416053	total: 18.4ms	remaining: 595ms
3:	learn: 0.5071904	total: 25ms	remaining: 601ms
4:	learn: 0.4782663	total: 31.3ms	remaining: 594ms
5:	learn: 0.4543147	total: 36.5ms	remaining: 572ms
6:	learn: 0.4341060	total: 41.2ms	remaining: 547ms
7:	learn: 0.4164778	total: 46.5ms	remaining: 535ms
8:	learn: 0.4014798	total: 52.8ms	remaining: 534ms
9:	learn: 0.3887225	total: 59.1ms	remaining: 532ms
10:	learn: 0.3778435	total: 65.5ms	remaining: 530ms
11:	learn: 0.3685897	total: 71.6ms	remaining: 525ms
12:	learn: 0.3606401	total: 77.8ms	remaining: 520ms
13:	learn: 0.3538442	total: 84.1ms	remaining: 517ms
14:	learn: 0.3480689	total: 90.6ms	remaining: 514ms
15:	learn: 0.3431688	total: 96.9ms	remaining: 509ms
16:	learn: 0.3389987	total: 103ms	remaining: 504ms
17:	learn: 0.3354160	total: 109ms	remaining: 496ms
18:	learn: 0.3323907	total: 115ms	remaining: 489ms
19:	learn: 0.3298390	total: 1

In [39]:
# Long to Short

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['LtoS']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6631618	total: 5.87ms	remaining: 581ms
1:	learn: 0.6387252	total: 13.3ms	remaining: 653ms
2:	learn: 0.6186559	total: 18.8ms	remaining: 609ms
3:	learn: 0.6022958	total: 23.8ms	remaining: 571ms
4:	learn: 0.5886098	total: 28.7ms	remaining: 546ms
5:	learn: 0.5770265	total: 35.3ms	remaining: 553ms
6:	learn: 0.5673306	total: 42.5ms	remaining: 565ms
7:	learn: 0.5591532	total: 49.8ms	remaining: 573ms
8:	learn: 0.5523416	total: 57.2ms	remaining: 578ms
9:	learn: 0.5465537	total: 63.3ms	remaining: 570ms
10:	learn: 0.5417057	total: 70.3ms	remaining: 569ms
11:	learn: 0.5375870	total: 76.4ms	remaining: 560ms
12:	learn: 0.5341255	total: 82.9ms	remaining: 555ms
13:	learn: 0.5311870	total: 89.4ms	remaining: 549ms
14:	learn: 0.5287207	total: 95.4ms	remaining: 541ms
15:	learn: 0.5266134	total: 102ms	remaining: 535ms
16:	learn: 0.5248844	total: 107ms	remaining: 523ms
17:	learn: 0.5233480	total: 113ms	remaining: 514ms
18:	learn: 0.5221149	total: 119ms	remaining: 505ms
19:	learn: 0.5210827	total

62:	learn: 0.5178321	total: 554ms	remaining: 326ms
63:	learn: 0.5178318	total: 564ms	remaining: 317ms
64:	learn: 0.5178265	total: 573ms	remaining: 308ms
65:	learn: 0.5178189	total: 580ms	remaining: 299ms
66:	learn: 0.5178185	total: 590ms	remaining: 291ms
67:	learn: 0.5178153	total: 597ms	remaining: 281ms
68:	learn: 0.5178111	total: 603ms	remaining: 271ms
69:	learn: 0.5178100	total: 608ms	remaining: 260ms
70:	learn: 0.5178062	total: 613ms	remaining: 250ms
71:	learn: 0.5178034	total: 618ms	remaining: 241ms
72:	learn: 0.5177986	total: 623ms	remaining: 231ms
73:	learn: 0.5177954	total: 629ms	remaining: 221ms
74:	learn: 0.5177945	total: 635ms	remaining: 212ms
75:	learn: 0.5177971	total: 640ms	remaining: 202ms
76:	learn: 0.5177936	total: 646ms	remaining: 193ms
77:	learn: 0.5177861	total: 651ms	remaining: 184ms
78:	learn: 0.5177926	total: 656ms	remaining: 174ms
79:	learn: 0.5177842	total: 661ms	remaining: 165ms
80:	learn: 0.5177817	total: 665ms	remaining: 156ms
81:	learn: 0.5177781	total: 670

54:	learn: 0.5146759	total: 407ms	remaining: 333ms
55:	learn: 0.5146710	total: 413ms	remaining: 324ms
56:	learn: 0.5146619	total: 419ms	remaining: 316ms
57:	learn: 0.5146620	total: 423ms	remaining: 307ms
58:	learn: 0.5146583	total: 427ms	remaining: 297ms
59:	learn: 0.5146455	total: 431ms	remaining: 287ms
60:	learn: 0.5146422	total: 436ms	remaining: 279ms
61:	learn: 0.5146386	total: 439ms	remaining: 269ms
62:	learn: 0.5146364	total: 444ms	remaining: 261ms
63:	learn: 0.5146309	total: 448ms	remaining: 252ms
64:	learn: 0.5146275	total: 453ms	remaining: 244ms
65:	learn: 0.5146220	total: 459ms	remaining: 236ms
66:	learn: 0.5146208	total: 465ms	remaining: 229ms
67:	learn: 0.5146150	total: 471ms	remaining: 221ms
68:	learn: 0.5146151	total: 475ms	remaining: 213ms
69:	learn: 0.5146104	total: 481ms	remaining: 206ms
70:	learn: 0.5146089	total: 487ms	remaining: 199ms
71:	learn: 0.5146096	total: 493ms	remaining: 192ms
72:	learn: 0.5146075	total: 498ms	remaining: 184ms
73:	learn: 0.5145982	total: 504

30:	learn: 0.5227872	total: 186ms	remaining: 415ms
31:	learn: 0.5226811	total: 193ms	remaining: 409ms
32:	learn: 0.5225435	total: 200ms	remaining: 405ms
33:	learn: 0.5224557	total: 204ms	remaining: 396ms
34:	learn: 0.5223623	total: 209ms	remaining: 389ms
35:	learn: 0.5223100	total: 214ms	remaining: 380ms
36:	learn: 0.5222485	total: 219ms	remaining: 372ms
37:	learn: 0.5222035	total: 223ms	remaining: 365ms
38:	learn: 0.5221612	total: 227ms	remaining: 356ms
39:	learn: 0.5221112	total: 232ms	remaining: 348ms
40:	learn: 0.5220881	total: 235ms	remaining: 339ms
41:	learn: 0.5220614	total: 239ms	remaining: 331ms
42:	learn: 0.5220193	total: 244ms	remaining: 323ms
43:	learn: 0.5219768	total: 248ms	remaining: 316ms
44:	learn: 0.5219547	total: 252ms	remaining: 309ms
45:	learn: 0.5219357	total: 257ms	remaining: 301ms
46:	learn: 0.5219157	total: 261ms	remaining: 295ms
47:	learn: 0.5218870	total: 266ms	remaining: 288ms
48:	learn: 0.5218714	total: 271ms	remaining: 282ms
49:	learn: 0.5218573	total: 275

0:	learn: 0.6637754	total: 8.94ms	remaining: 885ms
1:	learn: 0.6398558	total: 15.6ms	remaining: 766ms
2:	learn: 0.6202538	total: 21.3ms	remaining: 689ms
3:	learn: 0.6041736	total: 26.6ms	remaining: 638ms
4:	learn: 0.5908068	total: 31.5ms	remaining: 599ms
5:	learn: 0.5795543	total: 37.8ms	remaining: 592ms
6:	learn: 0.5701312	total: 44.1ms	remaining: 586ms
7:	learn: 0.5622271	total: 50.2ms	remaining: 578ms
8:	learn: 0.5556455	total: 56.4ms	remaining: 570ms
9:	learn: 0.5500764	total: 62.4ms	remaining: 562ms
10:	learn: 0.5454211	total: 68.3ms	remaining: 553ms
11:	learn: 0.5414981	total: 74.4ms	remaining: 546ms
12:	learn: 0.5381719	total: 80.6ms	remaining: 540ms
13:	learn: 0.5353886	total: 87ms	remaining: 535ms
14:	learn: 0.5330625	total: 93.1ms	remaining: 527ms
15:	learn: 0.5310571	total: 99.3ms	remaining: 521ms
16:	learn: 0.5294204	total: 105ms	remaining: 512ms
17:	learn: 0.5279743	total: 111ms	remaining: 505ms
18:	learn: 0.5267980	total: 117ms	remaining: 499ms
19:	learn: 0.5258296	total:

...

90:	learn: 0.5103056	total: 769ms	remaining: 76ms
91:	learn: 0.5103137	total: 775ms	remaining: 67.4ms
92:	learn: 0.5103036	total: 781ms	remaining: 58.8ms
93:	learn: 0.5103121	total: 787ms	remaining: 50.2ms
94:	learn: 0.5103046	total: 792ms	remaining: 41.7ms
95:	learn: 0.5103085	total: 799ms	remaining: 33.3ms
96:	learn: 0.5103054	total: 805ms	remaining: 24.9ms
97:	learn: 0.5103071	total: 809ms	remaining: 16.5ms
98:	learn: 0.5103053	total: 815ms	remaining: 8.23ms
99:	learn: 0.5103056	total: 821ms	remaining: 0us
0:	learn: 0.6634970	total: 6.32ms	remaining: 626ms
1:	learn: 0.6393328	total: 12.5ms	remaining: 613ms
2:	learn: 0.6195399	total: 17.6ms	remaining: 568ms
3:	learn: 0.6033928	total: 22.6ms	remaining: 541ms
4:	learn: 0.5899257	total: 27.6ms	remaining: 524ms
5:	learn: 0.5785617	total: 33.6ms	remaining: 527ms
6:	learn: 0.5690517	total: 39.7ms	remaining: 527ms
7:	learn: 0.5610646	total: 45.6ms	remaining: 524ms
8:	learn: 0.5543724	total: 51.5ms	remaining: 521ms
9:	learn: 0.5487629	total:

71:	learn: 0.5128071	total: 552ms	remaining: 215ms
72:	learn: 0.5128039	total: 560ms	remaining: 207ms
73:	learn: 0.5127998	total: 568ms	remaining: 200ms
74:	learn: 0.5128001	total: 576ms	remaining: 192ms
75:	learn: 0.5127986	total: 585ms	remaining: 185ms
76:	learn: 0.5127975	total: 593ms	remaining: 177ms
77:	learn: 0.5128019	total: 600ms	remaining: 169ms
78:	learn: 0.5127933	total: 608ms	remaining: 162ms
79:	learn: 0.5127911	total: 613ms	remaining: 153ms
80:	learn: 0.5127994	total: 621ms	remaining: 146ms
81:	learn: 0.5127980	total: 627ms	remaining: 138ms
82:	learn: 0.5127929	total: 633ms	remaining: 130ms
83:	learn: 0.5127973	total: 639ms	remaining: 122ms
84:	learn: 0.5127963	total: 646ms	remaining: 114ms
85:	learn: 0.5127893	total: 654ms	remaining: 106ms
86:	learn: 0.5127850	total: 662ms	remaining: 98.9ms
87:	learn: 0.5127889	total: 669ms	remaining: 91.3ms
88:	learn: 0.5127851	total: 676ms	remaining: 83.5ms
89:	learn: 0.5127843	total: 683ms	remaining: 75.9ms
90:	learn: 0.5127809	total:

34:	learn: 0.5172061	total: 198ms	remaining: 368ms
35:	learn: 0.5171618	total: 204ms	remaining: 362ms
36:	learn: 0.5171143	total: 209ms	remaining: 356ms
37:	learn: 0.5170693	total: 215ms	remaining: 351ms
38:	learn: 0.5170165	total: 220ms	remaining: 345ms
39:	learn: 0.5169615	total: 227ms	remaining: 340ms
40:	learn: 0.5169391	total: 232ms	remaining: 334ms
41:	learn: 0.5169138	total: 238ms	remaining: 328ms
42:	learn: 0.5168736	total: 244ms	remaining: 323ms
43:	learn: 0.5168304	total: 249ms	remaining: 317ms
44:	learn: 0.5168151	total: 255ms	remaining: 311ms
45:	learn: 0.5167918	total: 261ms	remaining: 306ms
46:	learn: 0.5167914	total: 266ms	remaining: 300ms
47:	learn: 0.5167839	total: 271ms	remaining: 293ms
48:	learn: 0.5167569	total: 277ms	remaining: 288ms
49:	learn: 0.5167455	total: 282ms	remaining: 282ms
50:	learn: 0.5167431	total: 288ms	remaining: 276ms
51:	learn: 0.5167252	total: 293ms	remaining: 271ms
52:	learn: 0.5167199	total: 299ms	remaining: 265ms
53:	learn: 0.5167041	total: 304

99:	learn: 0.5210968	total: 553ms	remaining: 0us
0:	learn: 0.6634815	total: 8.37ms	remaining: 828ms
1:	learn: 0.6393185	total: 18.9ms	remaining: 928ms
2:	learn: 0.6195036	total: 28ms	remaining: 904ms
3:	learn: 0.6033302	total: 36.3ms	remaining: 871ms
4:	learn: 0.5898232	total: 43.8ms	remaining: 833ms
5:	learn: 0.5784262	total: 52.8ms	remaining: 827ms
6:	learn: 0.5688875	total: 62.3ms	remaining: 828ms
7:	learn: 0.5608655	total: 71.3ms	remaining: 820ms
8:	learn: 0.5541316	total: 80.2ms	remaining: 811ms
9:	learn: 0.5484794	total: 91.1ms	remaining: 820ms
10:	learn: 0.5437575	total: 101ms	remaining: 818ms
11:	learn: 0.5397518	total: 111ms	remaining: 811ms
12:	learn: 0.5363722	total: 120ms	remaining: 805ms
13:	learn: 0.5335404	total: 129ms	remaining: 792ms
14:	learn: 0.5311689	total: 138ms	remaining: 781ms
15:	learn: 0.5290861	total: 147ms	remaining: 770ms
16:	learn: 0.5274180	total: 155ms	remaining: 756ms
17:	learn: 0.5259437	total: 164ms	remaining: 746ms
18:	learn: 0.5247516	total: 172ms	r

64:	learn: 0.5195479	total: 357ms	remaining: 192ms
65:	learn: 0.5195499	total: 363ms	remaining: 187ms
66:	learn: 0.5195474	total: 369ms	remaining: 182ms
67:	learn: 0.5195449	total: 375ms	remaining: 176ms
68:	learn: 0.5195403	total: 382ms	remaining: 171ms
69:	learn: 0.5195428	total: 389ms	remaining: 167ms
70:	learn: 0.5195416	total: 395ms	remaining: 161ms
71:	learn: 0.5195361	total: 401ms	remaining: 156ms
72:	learn: 0.5195394	total: 407ms	remaining: 150ms
73:	learn: 0.5195393	total: 413ms	remaining: 145ms
74:	learn: 0.5195395	total: 418ms	remaining: 139ms
75:	learn: 0.5195333	total: 424ms	remaining: 134ms
76:	learn: 0.5195337	total: 429ms	remaining: 128ms
77:	learn: 0.5195289	total: 433ms	remaining: 122ms
78:	learn: 0.5195273	total: 438ms	remaining: 117ms
79:	learn: 0.5195234	total: 444ms	remaining: 111ms
80:	learn: 0.5195290	total: 450ms	remaining: 105ms
81:	learn: 0.5195253	total: 454ms	remaining: 99.8ms
82:	learn: 0.5195220	total: 459ms	remaining: 94.1ms
83:	learn: 0.5195278	total: 4

In [40]:
# Long to Regular

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['LtoR']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6532166	total: 6.47ms	remaining: 641ms
1:	learn: 0.6195994	total: 14.2ms	remaining: 695ms
2:	learn: 0.5923105	total: 20.2ms	remaining: 652ms
3:	learn: 0.5699060	total: 25.4ms	remaining: 609ms
4:	learn: 0.5508965	total: 31.6ms	remaining: 601ms
5:	learn: 0.5356271	total: 37.1ms	remaining: 581ms
6:	learn: 0.5222124	total: 44.3ms	remaining: 588ms
7:	learn: 0.5110739	total: 51.2ms	remaining: 588ms
8:	learn: 0.5016558	total: 58.6ms	remaining: 593ms
9:	learn: 0.4937373	total: 65.9ms	remaining: 593ms
10:	learn: 0.4871137	total: 73.2ms	remaining: 592ms
11:	learn: 0.4815968	total: 81.2ms	remaining: 596ms
12:	learn: 0.4769309	total: 88.4ms	remaining: 592ms
13:	learn: 0.4730310	total: 95.3ms	remaining: 585ms
14:	learn: 0.4697742	total: 102ms	remaining: 577ms
15:	learn: 0.4670886	total: 109ms	remaining: 570ms
16:	learn: 0.4649323	total: 117ms	remaining: 569ms
17:	learn: 0.4632013	total: 122ms	remaining: 558ms
18:	learn: 0.4617788	total: 128ms	remaining: 545ms
19:	learn: 0.4605938	total:

70:	learn: 0.4575591	total: 383ms	remaining: 157ms
71:	learn: 0.4575579	total: 388ms	remaining: 151ms
72:	learn: 0.4575579	total: 393ms	remaining: 145ms
73:	learn: 0.4575583	total: 397ms	remaining: 140ms
74:	learn: 0.4575591	total: 402ms	remaining: 134ms
75:	learn: 0.4575540	total: 407ms	remaining: 128ms
76:	learn: 0.4575552	total: 410ms	remaining: 123ms
77:	learn: 0.4575544	total: 414ms	remaining: 117ms
78:	learn: 0.4575540	total: 419ms	remaining: 111ms
79:	learn: 0.4575482	total: 423ms	remaining: 106ms
80:	learn: 0.4575533	total: 427ms	remaining: 100ms
81:	learn: 0.4575490	total: 431ms	remaining: 94.6ms
82:	learn: 0.4575533	total: 436ms	remaining: 89.2ms
83:	learn: 0.4575464	total: 440ms	remaining: 83.7ms
84:	learn: 0.4575506	total: 443ms	remaining: 78.3ms
85:	learn: 0.4575462	total: 447ms	remaining: 72.8ms
86:	learn: 0.4575494	total: 451ms	remaining: 67.3ms
87:	learn: 0.4575470	total: 453ms	remaining: 61.8ms
88:	learn: 0.4575454	total: 457ms	remaining: 56.5ms
89:	learn: 0.4575463	to

32:	learn: 0.4438758	total: 189ms	remaining: 384ms
33:	learn: 0.4437764	total: 197ms	remaining: 383ms
34:	learn: 0.4436577	total: 205ms	remaining: 380ms
35:	learn: 0.4436011	total: 210ms	remaining: 374ms
36:	learn: 0.4435251	total: 216ms	remaining: 367ms
37:	learn: 0.4434598	total: 221ms	remaining: 361ms
38:	learn: 0.4434101	total: 225ms	remaining: 352ms
39:	learn: 0.4433783	total: 229ms	remaining: 344ms
40:	learn: 0.4433373	total: 236ms	remaining: 339ms
41:	learn: 0.4432925	total: 242ms	remaining: 335ms
42:	learn: 0.4432699	total: 248ms	remaining: 329ms
43:	learn: 0.4432412	total: 255ms	remaining: 324ms
44:	learn: 0.4432203	total: 261ms	remaining: 319ms
45:	learn: 0.4432009	total: 268ms	remaining: 314ms
46:	learn: 0.4431859	total: 274ms	remaining: 309ms
47:	learn: 0.4431704	total: 280ms	remaining: 304ms
48:	learn: 0.4431588	total: 287ms	remaining: 298ms
49:	learn: 0.4431461	total: 293ms	remaining: 293ms
50:	learn: 0.4431411	total: 300ms	remaining: 288ms
51:	learn: 0.4431309	total: 306

0:	learn: 0.6537477	total: 11.2ms	remaining: 1.1s
1:	learn: 0.6204563	total: 22.4ms	remaining: 1.1s
2:	learn: 0.5934436	total: 33.2ms	remaining: 1.07s
3:	learn: 0.5712661	total: 43.2ms	remaining: 1.04s
4:	learn: 0.5524328	total: 57.5ms	remaining: 1.09s
5:	learn: 0.5366751	total: 69.3ms	remaining: 1.08s
6:	learn: 0.5235523	total: 82.4ms	remaining: 1.09s
7:	learn: 0.5125728	total: 94.8ms	remaining: 1.09s
8:	learn: 0.5033577	total: 105ms	remaining: 1.06s
9:	learn: 0.4959885	total: 111ms	remaining: 1s
10:	learn: 0.4894404	total: 118ms	remaining: 955ms
11:	learn: 0.4840815	total: 125ms	remaining: 919ms
12:	learn: 0.4794319	total: 132ms	remaining: 885ms
13:	learn: 0.4755157	total: 139ms	remaining: 853ms
14:	learn: 0.4725320	total: 145ms	remaining: 820ms
15:	learn: 0.4697792	total: 151ms	remaining: 791ms
16:	learn: 0.4676297	total: 156ms	remaining: 761ms
17:	learn: 0.4657230	total: 161ms	remaining: 732ms
18:	learn: 0.4642669	total: 166ms	remaining: 708ms
19:	learn: 0.4629200	total: 172ms	rema

73:	learn: 0.4440158	total: 361ms	remaining: 127ms
74:	learn: 0.4440128	total: 366ms	remaining: 122ms
75:	learn: 0.4440120	total: 370ms	remaining: 117ms
76:	learn: 0.4440108	total: 373ms	remaining: 111ms
77:	learn: 0.4440140	total: 376ms	remaining: 106ms
78:	learn: 0.4440111	total: 381ms	remaining: 101ms
79:	learn: 0.4440091	total: 385ms	remaining: 96.2ms
80:	learn: 0.4440065	total: 389ms	remaining: 91.2ms
81:	learn: 0.4440089	total: 393ms	remaining: 86.3ms
82:	learn: 0.4440084	total: 398ms	remaining: 81.4ms
83:	learn: 0.4440097	total: 401ms	remaining: 76.4ms
84:	learn: 0.4440069	total: 405ms	remaining: 71.5ms
85:	learn: 0.4440031	total: 409ms	remaining: 66.5ms
86:	learn: 0.4440059	total: 412ms	remaining: 61.6ms
87:	learn: 0.4440052	total: 415ms	remaining: 56.6ms
88:	learn: 0.4440048	total: 419ms	remaining: 51.7ms
89:	learn: 0.4440053	total: 423ms	remaining: 47ms
90:	learn: 0.4440032	total: 426ms	remaining: 42.1ms
91:	learn: 0.4440017	total: 430ms	remaining: 37.4ms
92:	learn: 0.4440046

...

52:	learn: 0.4523161	total: 373ms	remaining: 330ms
53:	learn: 0.4523097	total: 377ms	remaining: 321ms
54:	learn: 0.4523051	total: 381ms	remaining: 311ms
55:	learn: 0.4522979	total: 385ms	remaining: 302ms
56:	learn: 0.4522959	total: 388ms	remaining: 293ms
57:	learn: 0.4522946	total: 391ms	remaining: 283ms
58:	learn: 0.4522927	total: 396ms	remaining: 275ms
59:	learn: 0.4522914	total: 402ms	remaining: 268ms
60:	learn: 0.4522885	total: 408ms	remaining: 261ms
61:	learn: 0.4522821	total: 412ms	remaining: 253ms
62:	learn: 0.4522831	total: 418ms	remaining: 246ms
63:	learn: 0.4522799	total: 424ms	remaining: 239ms
64:	learn: 0.4522818	total: 428ms	remaining: 231ms
65:	learn: 0.4522792	total: 433ms	remaining: 223ms
66:	learn: 0.4522754	total: 439ms	remaining: 216ms
67:	learn: 0.4522738	total: 444ms	remaining: 209ms
68:	learn: 0.4522730	total: 449ms	remaining: 202ms
69:	learn: 0.4522759	total: 454ms	remaining: 195ms
70:	learn: 0.4522743	total: 460ms	remaining: 188ms
71:	learn: 0.4522680	total: 466

31:	learn: 0.4523721	total: 181ms	remaining: 385ms
32:	learn: 0.4522458	total: 188ms	remaining: 381ms
33:	learn: 0.4521478	total: 194ms	remaining: 376ms
34:	learn: 0.4520318	total: 200ms	remaining: 372ms
35:	learn: 0.4519707	total: 206ms	remaining: 365ms
36:	learn: 0.4519023	total: 212ms	remaining: 360ms
37:	learn: 0.4518440	total: 217ms	remaining: 355ms
38:	learn: 0.4517959	total: 223ms	remaining: 348ms
39:	learn: 0.4517557	total: 228ms	remaining: 341ms
40:	learn: 0.4517151	total: 233ms	remaining: 335ms
41:	learn: 0.4516876	total: 239ms	remaining: 330ms
42:	learn: 0.4516639	total: 244ms	remaining: 323ms
43:	learn: 0.4516344	total: 250ms	remaining: 318ms
44:	learn: 0.4515993	total: 256ms	remaining: 313ms
45:	learn: 0.4515777	total: 262ms	remaining: 308ms
46:	learn: 0.4515660	total: 268ms	remaining: 303ms
47:	learn: 0.4515458	total: 274ms	remaining: 297ms
48:	learn: 0.4515345	total: 280ms	remaining: 291ms
49:	learn: 0.4515255	total: 286ms	remaining: 286ms
50:	learn: 0.4515207	total: 292

0:	learn: 0.6521615	total: 9.66ms	remaining: 957ms
1:	learn: 0.6187398	total: 18.1ms	remaining: 888ms
2:	learn: 0.5914574	total: 26.3ms	remaining: 852ms
3:	learn: 0.5688030	total: 34.6ms	remaining: 832ms
4:	learn: 0.5499464	total: 43.3ms	remaining: 823ms
5:	learn: 0.5345842	total: 51.6ms	remaining: 808ms
6:	learn: 0.5213690	total: 59.6ms	remaining: 792ms
7:	learn: 0.5102966	total: 69ms	remaining: 794ms
8:	learn: 0.5009763	total: 77.4ms	remaining: 783ms
9:	learn: 0.4935995	total: 84.5ms	remaining: 760ms
10:	learn: 0.4869530	total: 93.9ms	remaining: 760ms
11:	learn: 0.4814059	total: 103ms	remaining: 757ms
12:	learn: 0.4768966	total: 111ms	remaining: 745ms
13:	learn: 0.4729603	total: 120ms	remaining: 736ms
14:	learn: 0.4699081	total: 127ms	remaining: 718ms
15:	learn: 0.4671380	total: 135ms	remaining: 710ms
16:	learn: 0.4648288	total: 144ms	remaining: 702ms
17:	learn: 0.4629300	total: 152ms	remaining: 694ms
18:	learn: 0.4614238	total: 161ms	remaining: 687ms
19:	learn: 0.4601309	total: 169m

0:	learn: 0.6531760	total: 6.16ms	remaining: 610ms
1:	learn: 0.6193681	total: 12.4ms	remaining: 608ms
2:	learn: 0.5918140	total: 18.2ms	remaining: 588ms
3:	learn: 0.5690967	total: 23.6ms	remaining: 566ms
4:	learn: 0.5502371	total: 28.9ms	remaining: 548ms
5:	learn: 0.5348888	total: 33.8ms	remaining: 530ms
6:	learn: 0.5214992	total: 40.1ms	remaining: 533ms
7:	learn: 0.5102928	total: 47ms	remaining: 540ms
8:	learn: 0.5008008	total: 53.9ms	remaining: 545ms
9:	learn: 0.4930079	total: 59.5ms	remaining: 536ms
10:	learn: 0.4863153	total: 66ms	remaining: 534ms
11:	learn: 0.4807820	total: 72.3ms	remaining: 530ms
12:	learn: 0.4764200	total: 77.7ms	remaining: 520ms
13:	learn: 0.4724231	total: 83.7ms	remaining: 514ms
14:	learn: 0.4691953	total: 89.5ms	remaining: 507ms
15:	learn: 0.4663826	total: 95.8ms	remaining: 503ms
16:	learn: 0.4642395	total: 103ms	remaining: 501ms
17:	learn: 0.4622577	total: 109ms	remaining: 497ms
18:	learn: 0.4606066	total: 116ms	remaining: 493ms
19:	learn: 0.4592233	total: 1

61:	learn: 0.4504345	total: 372ms	remaining: 228ms
62:	learn: 0.4504294	total: 379ms	remaining: 223ms
63:	learn: 0.4504333	total: 384ms	remaining: 216ms
64:	learn: 0.4504314	total: 391ms	remaining: 211ms
65:	learn: 0.4504232	total: 397ms	remaining: 204ms
66:	learn: 0.4504259	total: 404ms	remaining: 199ms
67:	learn: 0.4504232	total: 410ms	remaining: 193ms
68:	learn: 0.4504198	total: 416ms	remaining: 187ms
69:	learn: 0.4504132	total: 422ms	remaining: 181ms
70:	learn: 0.4504135	total: 426ms	remaining: 174ms
71:	learn: 0.4504151	total: 433ms	remaining: 168ms
72:	learn: 0.4504118	total: 439ms	remaining: 162ms
73:	learn: 0.4504105	total: 446ms	remaining: 157ms
74:	learn: 0.4504133	total: 452ms	remaining: 151ms
75:	learn: 0.4504083	total: 459ms	remaining: 145ms
76:	learn: 0.4504079	total: 463ms	remaining: 138ms
77:	learn: 0.4504055	total: 468ms	remaining: 132ms
78:	learn: 0.4504026	total: 474ms	remaining: 126ms
79:	learn: 0.4504040	total: 479ms	remaining: 120ms
80:	learn: 0.4504027	total: 484

In [41]:
# Long to Long

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['LtoL']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6791695	total: 6.61ms	remaining: 655ms
1:	learn: 0.6677574	total: 12.8ms	remaining: 627ms
2:	learn: 0.6584184	total: 18.4ms	remaining: 595ms
3:	learn: 0.6507037	total: 23.7ms	remaining: 569ms
4:	learn: 0.6442824	total: 28.8ms	remaining: 547ms
5:	learn: 0.6389237	total: 34ms	remaining: 533ms
6:	learn: 0.6344537	total: 38.8ms	remaining: 516ms
7:	learn: 0.6306947	total: 44.8ms	remaining: 515ms
8:	learn: 0.6275586	total: 50.3ms	remaining: 508ms
9:	learn: 0.6249062	total: 56.5ms	remaining: 508ms
10:	learn: 0.6226956	total: 61.5ms	remaining: 497ms
11:	learn: 0.6208246	total: 67.1ms	remaining: 492ms
12:	learn: 0.6192391	total: 73.5ms	remaining: 492ms
13:	learn: 0.6179342	total: 79.8ms	remaining: 490ms
14:	learn: 0.6168276	total: 86.2ms	remaining: 489ms
15:	learn: 0.6159103	total: 91.6ms	remaining: 481ms
16:	learn: 0.6151196	total: 97.2ms	remaining: 475ms
17:	learn: 0.6144744	total: 103ms	remaining: 468ms
18:	learn: 0.6139213	total: 108ms	remaining: 462ms
19:	learn: 0.6134497	total

89:	learn: 0.6127601	total: 561ms	remaining: 62.3ms
90:	learn: 0.6127599	total: 565ms	remaining: 55.9ms
91:	learn: 0.6127591	total: 570ms	remaining: 49.5ms
92:	learn: 0.6127637	total: 574ms	remaining: 43.2ms
93:	learn: 0.6127634	total: 578ms	remaining: 36.9ms
94:	learn: 0.6127606	total: 582ms	remaining: 30.7ms
95:	learn: 0.6127701	total: 587ms	remaining: 24.4ms
96:	learn: 0.6127701	total: 590ms	remaining: 18.2ms
97:	learn: 0.6127615	total: 593ms	remaining: 12.1ms
98:	learn: 0.6127623	total: 596ms	remaining: 6.02ms
99:	learn: 0.6127654	total: 600ms	remaining: 0us
0:	learn: 0.6796663	total: 3.84ms	remaining: 380ms
1:	learn: 0.6686856	total: 7.66ms	remaining: 375ms
2:	learn: 0.6596973	total: 11.2ms	remaining: 362ms
3:	learn: 0.6522829	total: 14.8ms	remaining: 355ms
4:	learn: 0.6461428	total: 19ms	remaining: 361ms
5:	learn: 0.6410397	total: 23.3ms	remaining: 365ms
6:	learn: 0.6367772	total: 27.7ms	remaining: 368ms
7:	learn: 0.6331872	total: 32.7ms	remaining: 376ms
8:	learn: 0.6302162	total

50:	learn: 0.6051923	total: 367ms	remaining: 352ms
51:	learn: 0.6051987	total: 372ms	remaining: 343ms
52:	learn: 0.6051917	total: 377ms	remaining: 334ms
53:	learn: 0.6051944	total: 381ms	remaining: 325ms
54:	learn: 0.6051892	total: 386ms	remaining: 316ms
55:	learn: 0.6051894	total: 392ms	remaining: 308ms
56:	learn: 0.6051929	total: 397ms	remaining: 300ms
57:	learn: 0.6051843	total: 403ms	remaining: 291ms
58:	learn: 0.6051873	total: 409ms	remaining: 284ms
59:	learn: 0.6051837	total: 414ms	remaining: 276ms
60:	learn: 0.6051855	total: 419ms	remaining: 268ms
61:	learn: 0.6051720	total: 426ms	remaining: 261ms
62:	learn: 0.6051849	total: 431ms	remaining: 253ms
63:	learn: 0.6051766	total: 437ms	remaining: 246ms
64:	learn: 0.6051761	total: 441ms	remaining: 238ms
65:	learn: 0.6051821	total: 447ms	remaining: 230ms
66:	learn: 0.6051796	total: 452ms	remaining: 223ms
67:	learn: 0.6051788	total: 456ms	remaining: 215ms
68:	learn: 0.6051769	total: 461ms	remaining: 207ms
69:	learn: 0.6051821	total: 466

31:	learn: 0.6160170	total: 181ms	remaining: 384ms
32:	learn: 0.6159817	total: 187ms	remaining: 379ms
33:	learn: 0.6159461	total: 193ms	remaining: 374ms
34:	learn: 0.6159150	total: 200ms	remaining: 371ms
35:	learn: 0.6158980	total: 207ms	remaining: 368ms
36:	learn: 0.6158723	total: 213ms	remaining: 363ms
37:	learn: 0.6158706	total: 218ms	remaining: 356ms
38:	learn: 0.6158548	total: 223ms	remaining: 349ms
39:	learn: 0.6158406	total: 228ms	remaining: 342ms
40:	learn: 0.6158387	total: 233ms	remaining: 335ms
41:	learn: 0.6158391	total: 238ms	remaining: 329ms
42:	learn: 0.6158227	total: 243ms	remaining: 322ms
43:	learn: 0.6158274	total: 248ms	remaining: 315ms
44:	learn: 0.6158179	total: 254ms	remaining: 310ms
45:	learn: 0.6158141	total: 259ms	remaining: 305ms
46:	learn: 0.6158137	total: 264ms	remaining: 297ms
47:	learn: 0.6158129	total: 269ms	remaining: 291ms
48:	learn: 0.6158139	total: 275ms	remaining: 286ms
49:	learn: 0.6158065	total: 279ms	remaining: 279ms
50:	learn: 0.6158103	total: 285

95:	learn: 0.6078057	total: 756ms	remaining: 31.5ms
96:	learn: 0.6078036	total: 765ms	remaining: 23.7ms
97:	learn: 0.6078035	total: 772ms	remaining: 15.8ms
98:	learn: 0.6078033	total: 780ms	remaining: 7.87ms
99:	learn: 0.6078131	total: 787ms	remaining: 0us
0:	learn: 0.6794993	total: 5.72ms	remaining: 566ms
1:	learn: 0.6683566	total: 11.1ms	remaining: 544ms
2:	learn: 0.6592606	total: 16.3ms	remaining: 526ms
3:	learn: 0.6517500	total: 21ms	remaining: 503ms
4:	learn: 0.6455045	total: 25.4ms	remaining: 482ms
5:	learn: 0.6403294	total: 30ms	remaining: 470ms
6:	learn: 0.6359955	total: 34.3ms	remaining: 455ms
7:	learn: 0.6323542	total: 40.1ms	remaining: 461ms
8:	learn: 0.6293392	total: 45ms	remaining: 455ms
9:	learn: 0.6267868	total: 50.7ms	remaining: 456ms
10:	learn: 0.6246648	total: 55.1ms	remaining: 446ms
11:	learn: 0.6228790	total: 60.6ms	remaining: 444ms
12:	learn: 0.6213738	total: 66.3ms	remaining: 444ms
13:	learn: 0.6201016	total: 71.9ms	remaining: 441ms
14:	learn: 0.6190600	total: 77m

...

54:	learn: 0.6055416	total: 361ms	remaining: 295ms
55:	learn: 0.6055463	total: 369ms	remaining: 290ms
56:	learn: 0.6055433	total: 376ms	remaining: 284ms
57:	learn: 0.6055379	total: 382ms	remaining: 277ms
58:	learn: 0.6055342	total: 390ms	remaining: 271ms
59:	learn: 0.6055300	total: 397ms	remaining: 265ms
60:	learn: 0.6055268	total: 405ms	remaining: 259ms
61:	learn: 0.6055404	total: 410ms	remaining: 251ms
62:	learn: 0.6055374	total: 415ms	remaining: 244ms
63:	learn: 0.6055393	total: 422ms	remaining: 237ms
64:	learn: 0.6055313	total: 428ms	remaining: 230ms
65:	learn: 0.6055367	total: 434ms	remaining: 224ms
66:	learn: 0.6055303	total: 441ms	remaining: 217ms
67:	learn: 0.6055333	total: 446ms	remaining: 210ms
68:	learn: 0.6055213	total: 453ms	remaining: 203ms
69:	learn: 0.6055342	total: 460ms	remaining: 197ms
70:	learn: 0.6055342	total: 467ms	remaining: 191ms
71:	learn: 0.6055361	total: 472ms	remaining: 184ms
72:	learn: 0.6055302	total: 478ms	remaining: 177ms
73:	learn: 0.6055289	total: 485

33:	learn: 0.6096923	total: 187ms	remaining: 364ms
34:	learn: 0.6096675	total: 194ms	remaining: 361ms
35:	learn: 0.6096428	total: 201ms	remaining: 357ms
36:	learn: 0.6096329	total: 205ms	remaining: 349ms
37:	learn: 0.6096348	total: 210ms	remaining: 342ms
38:	learn: 0.6096217	total: 216ms	remaining: 337ms
39:	learn: 0.6096107	total: 221ms	remaining: 332ms
40:	learn: 0.6095982	total: 226ms	remaining: 325ms
41:	learn: 0.6095846	total: 231ms	remaining: 320ms
42:	learn: 0.6095751	total: 236ms	remaining: 313ms
43:	learn: 0.6095777	total: 241ms	remaining: 307ms
44:	learn: 0.6095641	total: 247ms	remaining: 301ms
45:	learn: 0.6095586	total: 253ms	remaining: 297ms
46:	learn: 0.6095608	total: 257ms	remaining: 290ms
47:	learn: 0.6095602	total: 262ms	remaining: 284ms
48:	learn: 0.6095531	total: 268ms	remaining: 279ms
49:	learn: 0.6095448	total: 273ms	remaining: 273ms
50:	learn: 0.6095502	total: 278ms	remaining: 267ms
51:	learn: 0.6095347	total: 285ms	remaining: 263ms
52:	learn: 0.6095384	total: 290

0:	learn: 0.6791701	total: 5.07ms	remaining: 502ms
1:	learn: 0.6678019	total: 9.52ms	remaining: 467ms
2:	learn: 0.6585076	total: 13.5ms	remaining: 438ms
3:	learn: 0.6508268	total: 17.4ms	remaining: 418ms
4:	learn: 0.6444745	total: 21.2ms	remaining: 403ms
5:	learn: 0.6391971	total: 25.1ms	remaining: 393ms
6:	learn: 0.6348100	total: 29.2ms	remaining: 388ms
7:	learn: 0.6311205	total: 33.7ms	remaining: 388ms
8:	learn: 0.6280648	total: 37.9ms	remaining: 383ms
9:	learn: 0.6254979	total: 43.3ms	remaining: 390ms
10:	learn: 0.6233662	total: 48.2ms	remaining: 390ms
11:	learn: 0.6215583	total: 54.4ms	remaining: 399ms
12:	learn: 0.6200614	total: 60.6ms	remaining: 405ms
13:	learn: 0.6187849	total: 66.7ms	remaining: 410ms
14:	learn: 0.6177435	total: 72ms	remaining: 408ms
15:	learn: 0.6168885	total: 77ms	remaining: 404ms
16:	learn: 0.6161466	total: 82.3ms	remaining: 402ms
17:	learn: 0.6155406	total: 87.4ms	remaining: 398ms
18:	learn: 0.6150243	total: 93.4ms	remaining: 398ms
19:	learn: 0.6145989	total

63:	learn: 0.6131332	total: 355ms	remaining: 199ms
64:	learn: 0.6131314	total: 360ms	remaining: 194ms
65:	learn: 0.6131311	total: 366ms	remaining: 189ms
66:	learn: 0.6131301	total: 372ms	remaining: 183ms
67:	learn: 0.6131301	total: 377ms	remaining: 178ms
68:	learn: 0.6131301	total: 383ms	remaining: 172ms
69:	learn: 0.6131210	total: 389ms	remaining: 167ms
70:	learn: 0.6131289	total: 395ms	remaining: 161ms
71:	learn: 0.6131266	total: 401ms	remaining: 156ms
72:	learn: 0.6131257	total: 407ms	remaining: 151ms
73:	learn: 0.6131247	total: 412ms	remaining: 145ms
74:	learn: 0.6131186	total: 418ms	remaining: 139ms
75:	learn: 0.6131242	total: 423ms	remaining: 134ms
76:	learn: 0.6131248	total: 429ms	remaining: 128ms
77:	learn: 0.6131177	total: 435ms	remaining: 123ms
78:	learn: 0.6131238	total: 441ms	remaining: 117ms
79:	learn: 0.6131193	total: 446ms	remaining: 112ms
80:	learn: 0.6131269	total: 451ms	remaining: 106ms
81:	learn: 0.6131242	total: 456ms	remaining: 100ms
82:	learn: 0.6131265	total: 462

47:	learn: 0.6108445	total: 196ms	remaining: 212ms
48:	learn: 0.6108372	total: 202ms	remaining: 210ms
49:	learn: 0.6108414	total: 207ms	remaining: 207ms
50:	learn: 0.6108355	total: 212ms	remaining: 203ms
51:	learn: 0.6108328	total: 215ms	remaining: 198ms
52:	learn: 0.6108276	total: 218ms	remaining: 194ms
53:	learn: 0.6108376	total: 222ms	remaining: 189ms
54:	learn: 0.6108231	total: 228ms	remaining: 186ms
55:	learn: 0.6108288	total: 234ms	remaining: 184ms
56:	learn: 0.6108318	total: 240ms	remaining: 181ms
57:	learn: 0.6108292	total: 245ms	remaining: 177ms
58:	learn: 0.6108260	total: 251ms	remaining: 174ms
59:	learn: 0.6108174	total: 256ms	remaining: 171ms
60:	learn: 0.6108258	total: 263ms	remaining: 168ms
61:	learn: 0.6108264	total: 268ms	remaining: 164ms
62:	learn: 0.6108195	total: 274ms	remaining: 161ms
63:	learn: 0.6108216	total: 279ms	remaining: 157ms
64:	learn: 0.6108113	total: 285ms	remaining: 154ms
65:	learn: 0.6108185	total: 290ms	remaining: 149ms
66:	learn: 0.6108164	total: 296

In [42]:
# Standard Deviation

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['STD']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6438176	total: 8.18ms	remaining: 810ms
1:	learn: 0.6031914	total: 15.7ms	remaining: 768ms
2:	learn: 0.5697684	total: 23.1ms	remaining: 748ms
3:	learn: 0.5413925	total: 31.5ms	remaining: 755ms
4:	learn: 0.5177507	total: 39.6ms	remaining: 753ms
5:	learn: 0.4973403	total: 47.5ms	remaining: 745ms
6:	learn: 0.4794144	total: 55.5ms	remaining: 737ms
7:	learn: 0.4643749	total: 62.9ms	remaining: 723ms
8:	learn: 0.4511758	total: 70.7ms	remaining: 715ms
9:	learn: 0.4396864	total: 77.8ms	remaining: 700ms
10:	learn: 0.4300255	total: 85.5ms	remaining: 692ms
11:	learn: 0.4215801	total: 92.5ms	remaining: 678ms
12:	learn: 0.4139535	total: 99.5ms	remaining: 666ms
13:	learn: 0.4073867	total: 107ms	remaining: 659ms
14:	learn: 0.4017670	total: 115ms	remaining: 651ms
15:	learn: 0.3967131	total: 122ms	remaining: 641ms
16:	learn: 0.3927393	total: 130ms	remaining: 635ms
17:	learn: 0.3887836	total: 137ms	remaining: 626ms
18:	learn: 0.3851814	total: 144ms	remaining: 615ms
19:	learn: 0.3824361	total: 

73:	learn: 0.3657956	total: 549ms	remaining: 193ms
74:	learn: 0.3657840	total: 556ms	remaining: 185ms
75:	learn: 0.3657662	total: 564ms	remaining: 178ms
76:	learn: 0.3657540	total: 571ms	remaining: 171ms
77:	learn: 0.3657500	total: 578ms	remaining: 163ms
78:	learn: 0.3657461	total: 586ms	remaining: 156ms
79:	learn: 0.3657407	total: 593ms	remaining: 148ms
80:	learn: 0.3657346	total: 600ms	remaining: 141ms
81:	learn: 0.3657325	total: 607ms	remaining: 133ms
82:	learn: 0.3657176	total: 614ms	remaining: 126ms
83:	learn: 0.3657120	total: 621ms	remaining: 118ms
84:	learn: 0.3657009	total: 628ms	remaining: 111ms
85:	learn: 0.3656899	total: 635ms	remaining: 103ms
86:	learn: 0.3656809	total: 642ms	remaining: 95.9ms
87:	learn: 0.3656732	total: 649ms	remaining: 88.5ms
88:	learn: 0.3656662	total: 656ms	remaining: 81ms
89:	learn: 0.3656613	total: 662ms	remaining: 73.6ms
90:	learn: 0.3656527	total: 669ms	remaining: 66.2ms
91:	learn: 0.3656465	total: 676ms	remaining: 58.8ms
92:	learn: 0.3656351	total:

37:	learn: 0.3405655	total: 392ms	remaining: 639ms
38:	learn: 0.3402524	total: 403ms	remaining: 630ms
39:	learn: 0.3399319	total: 413ms	remaining: 620ms
40:	learn: 0.3396441	total: 424ms	remaining: 611ms
41:	learn: 0.3394837	total: 434ms	remaining: 600ms
42:	learn: 0.3392865	total: 444ms	remaining: 588ms
43:	learn: 0.3391245	total: 454ms	remaining: 578ms
44:	learn: 0.3389564	total: 465ms	remaining: 568ms
45:	learn: 0.3388040	total: 475ms	remaining: 557ms
46:	learn: 0.3386826	total: 484ms	remaining: 546ms
47:	learn: 0.3385691	total: 492ms	remaining: 533ms
48:	learn: 0.3384634	total: 498ms	remaining: 519ms
49:	learn: 0.3383731	total: 505ms	remaining: 505ms
50:	learn: 0.3383026	total: 512ms	remaining: 492ms
51:	learn: 0.3382413	total: 518ms	remaining: 479ms
52:	learn: 0.3382146	total: 525ms	remaining: 466ms
53:	learn: 0.3381986	total: 532ms	remaining: 454ms
54:	learn: 0.3381439	total: 539ms	remaining: 441ms
55:	learn: 0.3381172	total: 545ms	remaining: 429ms
56:	learn: 0.3380773	total: 551

0:	learn: 0.6445311	total: 10.3ms	remaining: 1.02s
1:	learn: 0.6047844	total: 19.5ms	remaining: 953ms
2:	learn: 0.5722965	total: 28.2ms	remaining: 912ms
3:	learn: 0.5449638	total: 37.4ms	remaining: 899ms
4:	learn: 0.5216590	total: 46.2ms	remaining: 878ms
5:	learn: 0.5017951	total: 54.9ms	remaining: 860ms
6:	learn: 0.4844730	total: 63.4ms	remaining: 842ms
7:	learn: 0.4695660	total: 71.5ms	remaining: 823ms
8:	learn: 0.4569140	total: 80.1ms	remaining: 810ms
9:	learn: 0.4459225	total: 88.4ms	remaining: 795ms
10:	learn: 0.4367333	total: 97.1ms	remaining: 786ms
11:	learn: 0.4282778	total: 106ms	remaining: 776ms
12:	learn: 0.4208148	total: 114ms	remaining: 765ms
13:	learn: 0.4142678	total: 123ms	remaining: 754ms
14:	learn: 0.4086832	total: 131ms	remaining: 742ms
15:	learn: 0.4036581	total: 139ms	remaining: 731ms
16:	learn: 0.3997765	total: 147ms	remaining: 720ms
17:	learn: 0.3959497	total: 156ms	remaining: 711ms
18:	learn: 0.3924415	total: 164ms	remaining: 700ms
19:	learn: 0.3893369	total: 17

71:	learn: 0.3534153	total: 572ms	remaining: 223ms
72:	learn: 0.3533940	total: 580ms	remaining: 214ms
73:	learn: 0.3533749	total: 588ms	remaining: 206ms
74:	learn: 0.3533512	total: 596ms	remaining: 199ms
75:	learn: 0.3533326	total: 604ms	remaining: 191ms
76:	learn: 0.3533195	total: 611ms	remaining: 183ms
77:	learn: 0.3533159	total: 619ms	remaining: 174ms
78:	learn: 0.3533133	total: 626ms	remaining: 167ms
79:	learn: 0.3533096	total: 633ms	remaining: 158ms
80:	learn: 0.3533022	total: 640ms	remaining: 150ms
81:	learn: 0.3532918	total: 647ms	remaining: 142ms
82:	learn: 0.3532808	total: 655ms	remaining: 134ms
83:	learn: 0.3532739	total: 662ms	remaining: 126ms
84:	learn: 0.3532641	total: 668ms	remaining: 118ms
85:	learn: 0.3532517	total: 675ms	remaining: 110ms
86:	learn: 0.3532394	total: 683ms	remaining: 102ms
87:	learn: 0.3532335	total: 690ms	remaining: 94.1ms
88:	learn: 0.3532302	total: 697ms	remaining: 86.2ms
89:	learn: 0.3532141	total: 704ms	remaining: 78.2ms
90:	learn: 0.3532030	total: 

...

47:	learn: 0.3562569	total: 372ms	remaining: 403ms
48:	learn: 0.3561404	total: 380ms	remaining: 395ms
49:	learn: 0.3560653	total: 387ms	remaining: 387ms
50:	learn: 0.3560443	total: 394ms	remaining: 379ms
51:	learn: 0.3559710	total: 401ms	remaining: 370ms
52:	learn: 0.3559176	total: 408ms	remaining: 362ms
53:	learn: 0.3558715	total: 415ms	remaining: 354ms
54:	learn: 0.3558362	total: 422ms	remaining: 345ms
55:	learn: 0.3558274	total: 431ms	remaining: 339ms
56:	learn: 0.3558033	total: 438ms	remaining: 330ms
57:	learn: 0.3557468	total: 445ms	remaining: 322ms
58:	learn: 0.3557227	total: 451ms	remaining: 314ms
59:	learn: 0.3556789	total: 458ms	remaining: 306ms
60:	learn: 0.3556686	total: 465ms	remaining: 297ms
61:	learn: 0.3556652	total: 472ms	remaining: 289ms
62:	learn: 0.3556387	total: 479ms	remaining: 281ms
63:	learn: 0.3556301	total: 487ms	remaining: 274ms
64:	learn: 0.3556132	total: 494ms	remaining: 266ms
65:	learn: 0.3555875	total: 501ms	remaining: 258ms
66:	learn: 0.3555811	total: 508

25:	learn: 0.3740445	total: 201ms	remaining: 573ms
26:	learn: 0.3728474	total: 209ms	remaining: 566ms
27:	learn: 0.3717073	total: 217ms	remaining: 558ms
28:	learn: 0.3709147	total: 225ms	remaining: 550ms
29:	learn: 0.3698865	total: 232ms	remaining: 542ms
30:	learn: 0.3690528	total: 239ms	remaining: 532ms
31:	learn: 0.3685525	total: 246ms	remaining: 523ms
32:	learn: 0.3678455	total: 253ms	remaining: 514ms
33:	learn: 0.3672227	total: 260ms	remaining: 505ms
34:	learn: 0.3666661	total: 267ms	remaining: 496ms
35:	learn: 0.3661646	total: 274ms	remaining: 487ms
36:	learn: 0.3657296	total: 281ms	remaining: 478ms
37:	learn: 0.3653056	total: 288ms	remaining: 469ms
38:	learn: 0.3649724	total: 295ms	remaining: 461ms
39:	learn: 0.3646645	total: 302ms	remaining: 453ms
40:	learn: 0.3643914	total: 309ms	remaining: 444ms
41:	learn: 0.3641638	total: 316ms	remaining: 436ms
42:	learn: 0.3640361	total: 323ms	remaining: 428ms
43:	learn: 0.3638679	total: 330ms	remaining: 420ms
44:	learn: 0.3637042	total: 337

95:	learn: 0.3318558	total: 940ms	remaining: 39.2ms
96:	learn: 0.3318465	total: 948ms	remaining: 29.3ms
97:	learn: 0.3318425	total: 955ms	remaining: 19.5ms
98:	learn: 0.3318326	total: 963ms	remaining: 9.72ms
99:	learn: 0.3318194	total: 970ms	remaining: 0us
0:	learn: 0.6442548	total: 8.71ms	remaining: 862ms
1:	learn: 0.6038127	total: 16.1ms	remaining: 788ms
2:	learn: 0.5706329	total: 23.7ms	remaining: 767ms
3:	learn: 0.5430330	total: 30.7ms	remaining: 736ms
4:	learn: 0.5194521	total: 38.2ms	remaining: 725ms
5:	learn: 0.4986414	total: 45.2ms	remaining: 708ms
6:	learn: 0.4809087	total: 52.1ms	remaining: 692ms
7:	learn: 0.4657868	total: 59.4ms	remaining: 683ms
8:	learn: 0.4526396	total: 66.6ms	remaining: 674ms
9:	learn: 0.4414362	total: 73.7ms	remaining: 664ms
10:	learn: 0.4317976	total: 81.4ms	remaining: 659ms
11:	learn: 0.4231608	total: 88.8ms	remaining: 651ms
12:	learn: 0.4156443	total: 96ms	remaining: 642ms
13:	learn: 0.4094118	total: 103ms	remaining: 634ms
14:	learn: 0.4037696	total: 

68:	learn: 0.3634648	total: 583ms	remaining: 262ms
69:	learn: 0.3634536	total: 593ms	remaining: 254ms
70:	learn: 0.3634421	total: 602ms	remaining: 246ms
71:	learn: 0.3634321	total: 612ms	remaining: 238ms
72:	learn: 0.3634235	total: 620ms	remaining: 229ms
73:	learn: 0.3634112	total: 628ms	remaining: 221ms
74:	learn: 0.3634037	total: 637ms	remaining: 212ms
75:	learn: 0.3633950	total: 645ms	remaining: 204ms
76:	learn: 0.3633881	total: 653ms	remaining: 195ms
77:	learn: 0.3633670	total: 661ms	remaining: 186ms
78:	learn: 0.3633629	total: 670ms	remaining: 178ms
79:	learn: 0.3633591	total: 678ms	remaining: 170ms
80:	learn: 0.3633490	total: 687ms	remaining: 161ms
81:	learn: 0.3633352	total: 696ms	remaining: 153ms
82:	learn: 0.3633303	total: 705ms	remaining: 144ms
83:	learn: 0.3633262	total: 713ms	remaining: 136ms
84:	learn: 0.3633218	total: 722ms	remaining: 127ms
85:	learn: 0.3633147	total: 730ms	remaining: 119ms
86:	learn: 0.3633049	total: 738ms	remaining: 110ms
87:	learn: 0.3632942	total: 745

35:	learn: 0.3656913	total: 374ms	remaining: 664ms
36:	learn: 0.3652609	total: 384ms	remaining: 654ms
37:	learn: 0.3648657	total: 395ms	remaining: 644ms
38:	learn: 0.3645477	total: 406ms	remaining: 635ms
39:	learn: 0.3642401	total: 417ms	remaining: 626ms
40:	learn: 0.3639863	total: 429ms	remaining: 617ms
41:	learn: 0.3637642	total: 439ms	remaining: 606ms
42:	learn: 0.3636314	total: 448ms	remaining: 594ms
43:	learn: 0.3634602	total: 458ms	remaining: 582ms
44:	learn: 0.3633040	total: 468ms	remaining: 572ms
45:	learn: 0.3631650	total: 477ms	remaining: 560ms
46:	learn: 0.3630427	total: 488ms	remaining: 550ms
47:	learn: 0.3629175	total: 497ms	remaining: 539ms
48:	learn: 0.3628210	total: 507ms	remaining: 528ms
49:	learn: 0.3627981	total: 517ms	remaining: 517ms
50:	learn: 0.3627168	total: 527ms	remaining: 507ms
51:	learn: 0.3626667	total: 537ms	remaining: 496ms
52:	learn: 0.3626448	total: 545ms	remaining: 483ms
53:	learn: 0.3625854	total: 553ms	remaining: 471ms
54:	learn: 0.3625228	total: 560

In [43]:
# Coefficient of Variance

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['CoefVar']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6348617	total: 8.31ms	remaining: 822ms
1:	learn: 0.5874000	total: 15.6ms	remaining: 766ms
2:	learn: 0.5482075	total: 23.2ms	remaining: 749ms
3:	learn: 0.5152097	total: 30.7ms	remaining: 736ms
4:	learn: 0.4873962	total: 38.1ms	remaining: 723ms
5:	learn: 0.4638325	total: 46.7ms	remaining: 732ms
6:	learn: 0.4434839	total: 54.5ms	remaining: 725ms
7:	learn: 0.4256842	total: 62.3ms	remaining: 716ms
8:	learn: 0.4104716	total: 70ms	remaining: 707ms
9:	learn: 0.3972009	total: 78ms	remaining: 702ms
10:	learn: 0.3858810	total: 85.8ms	remaining: 694ms
11:	learn: 0.3761019	total: 93.1ms	remaining: 683ms
12:	learn: 0.3673282	total: 100ms	remaining: 672ms
13:	learn: 0.3598581	total: 108ms	remaining: 665ms
14:	learn: 0.3531652	total: 116ms	remaining: 656ms
15:	learn: 0.3473668	total: 124ms	remaining: 651ms
16:	learn: 0.3424496	total: 132ms	remaining: 644ms
17:	learn: 0.3380058	total: 140ms	remaining: 639ms
18:	learn: 0.3338525	total: 148ms	remaining: 630ms
19:	learn: 0.3301421	total: 155ms

68:	learn: 0.3081928	total: 569ms	remaining: 256ms
69:	learn: 0.3081764	total: 577ms	remaining: 247ms
70:	learn: 0.3081501	total: 584ms	remaining: 239ms
71:	learn: 0.3081331	total: 592ms	remaining: 230ms
72:	learn: 0.3081240	total: 599ms	remaining: 222ms
73:	learn: 0.3081051	total: 606ms	remaining: 213ms
74:	learn: 0.3080846	total: 613ms	remaining: 204ms
75:	learn: 0.3080794	total: 620ms	remaining: 196ms
76:	learn: 0.3080702	total: 627ms	remaining: 187ms
77:	learn: 0.3080554	total: 634ms	remaining: 179ms
78:	learn: 0.3080416	total: 641ms	remaining: 170ms
79:	learn: 0.3080287	total: 648ms	remaining: 162ms
80:	learn: 0.3080210	total: 654ms	remaining: 153ms
81:	learn: 0.3080136	total: 662ms	remaining: 145ms
82:	learn: 0.3080003	total: 668ms	remaining: 137ms
83:	learn: 0.3079913	total: 675ms	remaining: 129ms
84:	learn: 0.3079834	total: 681ms	remaining: 120ms
85:	learn: 0.3079748	total: 688ms	remaining: 112ms
86:	learn: 0.3079591	total: 695ms	remaining: 104ms
87:	learn: 0.3079567	total: 702

49:	learn: 0.2903246	total: 390ms	remaining: 390ms
50:	learn: 0.2902887	total: 398ms	remaining: 383ms
51:	learn: 0.2901786	total: 405ms	remaining: 373ms
52:	learn: 0.2900956	total: 412ms	remaining: 365ms
53:	learn: 0.2900134	total: 418ms	remaining: 356ms
54:	learn: 0.2899415	total: 424ms	remaining: 347ms
55:	learn: 0.2898719	total: 430ms	remaining: 338ms
56:	learn: 0.2898139	total: 437ms	remaining: 329ms
57:	learn: 0.2897596	total: 443ms	remaining: 321ms
58:	learn: 0.2897108	total: 451ms	remaining: 313ms
59:	learn: 0.2896941	total: 459ms	remaining: 306ms
60:	learn: 0.2896648	total: 466ms	remaining: 298ms
61:	learn: 0.2896523	total: 474ms	remaining: 290ms
62:	learn: 0.2896138	total: 482ms	remaining: 283ms
63:	learn: 0.2895951	total: 489ms	remaining: 275ms
64:	learn: 0.2895601	total: 497ms	remaining: 268ms
65:	learn: 0.2895255	total: 505ms	remaining: 260ms
66:	learn: 0.2895086	total: 513ms	remaining: 253ms
67:	learn: 0.2894830	total: 520ms	remaining: 245ms
68:	learn: 0.2894554	total: 528

23:	learn: 0.3235680	total: 190ms	remaining: 602ms
24:	learn: 0.3216595	total: 198ms	remaining: 595ms
25:	learn: 0.3199865	total: 208ms	remaining: 591ms
26:	learn: 0.3186184	total: 215ms	remaining: 582ms
27:	learn: 0.3173293	total: 223ms	remaining: 573ms
28:	learn: 0.3161700	total: 231ms	remaining: 565ms
29:	learn: 0.3151896	total: 238ms	remaining: 556ms
30:	learn: 0.3142946	total: 246ms	remaining: 548ms
31:	learn: 0.3136179	total: 254ms	remaining: 539ms
32:	learn: 0.3127933	total: 262ms	remaining: 531ms
33:	learn: 0.3120761	total: 269ms	remaining: 523ms
34:	learn: 0.3114659	total: 278ms	remaining: 516ms
35:	learn: 0.3109610	total: 285ms	remaining: 507ms
36:	learn: 0.3104972	total: 293ms	remaining: 498ms
37:	learn: 0.3101072	total: 300ms	remaining: 489ms
38:	learn: 0.3097443	total: 307ms	remaining: 479ms
39:	learn: 0.3094286	total: 314ms	remaining: 471ms
40:	learn: 0.3091466	total: 322ms	remaining: 463ms
41:	learn: 0.3088696	total: 329ms	remaining: 455ms
42:	learn: 0.3086588	total: 337

86:	learn: 0.2972580	total: 580ms	remaining: 86.6ms
87:	learn: 0.2972377	total: 585ms	remaining: 79.8ms
88:	learn: 0.2972273	total: 590ms	remaining: 73ms
89:	learn: 0.2972001	total: 596ms	remaining: 66.2ms
90:	learn: 0.2971846	total: 601ms	remaining: 59.5ms
91:	learn: 0.2971773	total: 607ms	remaining: 52.8ms
92:	learn: 0.2971603	total: 614ms	remaining: 46.2ms
93:	learn: 0.2971476	total: 622ms	remaining: 39.7ms
94:	learn: 0.2971227	total: 628ms	remaining: 33ms
95:	learn: 0.2971223	total: 636ms	remaining: 26.5ms
96:	learn: 0.2971016	total: 644ms	remaining: 19.9ms
97:	learn: 0.2970915	total: 652ms	remaining: 13.3ms
98:	learn: 0.2970793	total: 660ms	remaining: 6.66ms
99:	learn: 0.2970696	total: 667ms	remaining: 0us
0:	learn: 0.6353567	total: 11.6ms	remaining: 1.14s
1:	learn: 0.5882383	total: 21.8ms	remaining: 1.07s
2:	learn: 0.5490109	total: 32ms	remaining: 1.03s
3:	learn: 0.5163435	total: 42.4ms	remaining: 1.02s
4:	learn: 0.4887714	total: 52.7ms	remaining: 1s
5:	learn: 0.4651738	total: 63

...

36:	learn: 0.3078224	total: 371ms	remaining: 631ms
37:	learn: 0.3073871	total: 377ms	remaining: 615ms
38:	learn: 0.3069803	total: 383ms	remaining: 599ms
39:	learn: 0.3066631	total: 389ms	remaining: 583ms
40:	learn: 0.3063384	total: 396ms	remaining: 570ms
41:	learn: 0.3060943	total: 403ms	remaining: 557ms
42:	learn: 0.3058601	total: 411ms	remaining: 545ms
43:	learn: 0.3056547	total: 418ms	remaining: 532ms
44:	learn: 0.3055654	total: 425ms	remaining: 519ms
45:	learn: 0.3053781	total: 432ms	remaining: 508ms
46:	learn: 0.3052196	total: 440ms	remaining: 496ms
47:	learn: 0.3050710	total: 447ms	remaining: 484ms
48:	learn: 0.3049200	total: 455ms	remaining: 473ms
49:	learn: 0.3048080	total: 462ms	remaining: 462ms
50:	learn: 0.3046918	total: 469ms	remaining: 451ms
51:	learn: 0.3045883	total: 476ms	remaining: 439ms
52:	learn: 0.3045137	total: 483ms	remaining: 428ms
53:	learn: 0.3045011	total: 490ms	remaining: 417ms
54:	learn: 0.3044226	total: 497ms	remaining: 407ms
55:	learn: 0.3043586	total: 504

97:	learn: 0.3039804	total: 724ms	remaining: 14.8ms
98:	learn: 0.3039694	total: 732ms	remaining: 7.39ms
99:	learn: 0.3039634	total: 739ms	remaining: 0us
0:	learn: 0.6357690	total: 8.3ms	remaining: 822ms
1:	learn: 0.5888363	total: 17.4ms	remaining: 851ms
2:	learn: 0.5501100	total: 26.1ms	remaining: 844ms
3:	learn: 0.5174825	total: 34ms	remaining: 815ms
4:	learn: 0.4898270	total: 42ms	remaining: 798ms
5:	learn: 0.4661868	total: 49.6ms	remaining: 777ms
6:	learn: 0.4457344	total: 57.4ms	remaining: 762ms
7:	learn: 0.4278952	total: 64.8ms	remaining: 745ms
8:	learn: 0.4129286	total: 72.9ms	remaining: 737ms
9:	learn: 0.3999294	total: 80ms	remaining: 720ms
10:	learn: 0.3886411	total: 87.4ms	remaining: 707ms
11:	learn: 0.3786113	total: 95.2ms	remaining: 698ms
12:	learn: 0.3699077	total: 103ms	remaining: 691ms
13:	learn: 0.3623390	total: 111ms	remaining: 682ms
14:	learn: 0.3554280	total: 118ms	remaining: 671ms
15:	learn: 0.3499040	total: 126ms	remaining: 661ms
16:	learn: 0.3447199	total: 133ms	re

63:	learn: 0.2715664	total: 590ms	remaining: 332ms
64:	learn: 0.2715199	total: 599ms	remaining: 323ms
65:	learn: 0.2714829	total: 609ms	remaining: 314ms
66:	learn: 0.2714677	total: 618ms	remaining: 304ms
67:	learn: 0.2714424	total: 627ms	remaining: 295ms
68:	learn: 0.2714168	total: 636ms	remaining: 286ms
69:	learn: 0.2714003	total: 645ms	remaining: 276ms
70:	learn: 0.2713713	total: 653ms	remaining: 267ms
71:	learn: 0.2713499	total: 662ms	remaining: 257ms
72:	learn: 0.2713273	total: 670ms	remaining: 248ms
73:	learn: 0.2713161	total: 679ms	remaining: 239ms
74:	learn: 0.2713007	total: 688ms	remaining: 229ms
75:	learn: 0.2712853	total: 697ms	remaining: 220ms
76:	learn: 0.2712710	total: 706ms	remaining: 211ms
77:	learn: 0.2712597	total: 715ms	remaining: 202ms
78:	learn: 0.2712490	total: 724ms	remaining: 193ms
79:	learn: 0.2712412	total: 733ms	remaining: 183ms
80:	learn: 0.2712365	total: 741ms	remaining: 174ms
81:	learn: 0.2712206	total: 750ms	remaining: 165ms
82:	learn: 0.2712115	total: 759

25:	learn: 0.3214423	total: 202ms	remaining: 576ms
26:	learn: 0.3198921	total: 211ms	remaining: 569ms
27:	learn: 0.3185110	total: 219ms	remaining: 563ms
28:	learn: 0.3174040	total: 227ms	remaining: 556ms
29:	learn: 0.3162903	total: 235ms	remaining: 549ms
30:	learn: 0.3154080	total: 243ms	remaining: 540ms
31:	learn: 0.3145718	total: 250ms	remaining: 532ms
32:	learn: 0.3137930	total: 258ms	remaining: 524ms
33:	learn: 0.3131263	total: 266ms	remaining: 516ms
34:	learn: 0.3128849	total: 273ms	remaining: 507ms
35:	learn: 0.3125832	total: 281ms	remaining: 499ms
36:	learn: 0.3120679	total: 288ms	remaining: 491ms
37:	learn: 0.3115932	total: 296ms	remaining: 482ms
38:	learn: 0.3112200	total: 303ms	remaining: 474ms
39:	learn: 0.3108562	total: 311ms	remaining: 466ms
40:	learn: 0.3104866	total: 318ms	remaining: 458ms
41:	learn: 0.3101891	total: 327ms	remaining: 451ms
42:	learn: 0.3099137	total: 335ms	remaining: 444ms
43:	learn: 0.3096668	total: 342ms	remaining: 436ms
44:	learn: 0.3095855	total: 350

0:	learn: 0.6364529	total: 8.28ms	remaining: 820ms
1:	learn: 0.5902419	total: 16.2ms	remaining: 793ms
2:	learn: 0.5518369	total: 24.2ms	remaining: 781ms
3:	learn: 0.5198542	total: 32.1ms	remaining: 770ms
4:	learn: 0.4924534	total: 39.7ms	remaining: 754ms
5:	learn: 0.4690190	total: 47.7ms	remaining: 747ms
6:	learn: 0.4488047	total: 55.5ms	remaining: 738ms
7:	learn: 0.4311492	total: 63.1ms	remaining: 726ms
8:	learn: 0.4163107	total: 71.2ms	remaining: 720ms
9:	learn: 0.4034860	total: 79.9ms	remaining: 719ms
10:	learn: 0.3923145	total: 88.2ms	remaining: 714ms
11:	learn: 0.3823494	total: 96.7ms	remaining: 709ms
12:	learn: 0.3736816	total: 105ms	remaining: 701ms
13:	learn: 0.3659526	total: 113ms	remaining: 695ms
14:	learn: 0.3593143	total: 121ms	remaining: 685ms
15:	learn: 0.3538488	total: 129ms	remaining: 675ms
16:	learn: 0.3491818	total: 136ms	remaining: 666ms
17:	learn: 0.3446014	total: 144ms	remaining: 656ms
18:	learn: 0.3403853	total: 151ms	remaining: 645ms
19:	learn: 0.3366921	total: 1

In [44]:
# Range

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['Range']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6485296	total: 9.72ms	remaining: 962ms
1:	learn: 0.6120289	total: 19ms	remaining: 930ms
2:	learn: 0.5818391	total: 27.5ms	remaining: 890ms
3:	learn: 0.5568366	total: 35.7ms	remaining: 856ms
4:	learn: 0.5354348	total: 43.8ms	remaining: 831ms
5:	learn: 0.5172526	total: 51.6ms	remaining: 808ms
6:	learn: 0.5016971	total: 59.5ms	remaining: 791ms
7:	learn: 0.4885685	total: 67.7ms	remaining: 779ms
8:	learn: 0.4769217	total: 76.1ms	remaining: 769ms
9:	learn: 0.4670726	total: 83.9ms	remaining: 755ms
10:	learn: 0.4584000	total: 92.3ms	remaining: 747ms
11:	learn: 0.4510759	total: 100ms	remaining: 737ms
12:	learn: 0.4443872	total: 108ms	remaining: 726ms
13:	learn: 0.4385507	total: 117ms	remaining: 717ms
14:	learn: 0.4334041	total: 124ms	remaining: 705ms
15:	learn: 0.4289339	total: 133ms	remaining: 696ms
16:	learn: 0.4250221	total: 142ms	remaining: 693ms
17:	learn: 0.4215239	total: 151ms	remaining: 687ms
18:	learn: 0.4184288	total: 159ms	remaining: 678ms
19:	learn: 0.4157516	total: 167m

79:	learn: 0.4027530	total: 560ms	remaining: 140ms
80:	learn: 0.4027335	total: 568ms	remaining: 133ms
81:	learn: 0.4027264	total: 576ms	remaining: 126ms
82:	learn: 0.4027098	total: 584ms	remaining: 120ms
83:	learn: 0.4026935	total: 593ms	remaining: 113ms
84:	learn: 0.4026820	total: 601ms	remaining: 106ms
85:	learn: 0.4026670	total: 608ms	remaining: 99ms
86:	learn: 0.4026474	total: 615ms	remaining: 92ms
87:	learn: 0.4026381	total: 622ms	remaining: 84.9ms
88:	learn: 0.4026256	total: 629ms	remaining: 77.8ms
89:	learn: 0.4026174	total: 637ms	remaining: 70.7ms
90:	learn: 0.4026068	total: 643ms	remaining: 63.6ms
91:	learn: 0.4025941	total: 650ms	remaining: 56.5ms
92:	learn: 0.4025794	total: 657ms	remaining: 49.5ms
93:	learn: 0.4025707	total: 664ms	remaining: 42.4ms
94:	learn: 0.4025573	total: 672ms	remaining: 35.3ms
95:	learn: 0.4025555	total: 679ms	remaining: 28.3ms
96:	learn: 0.4025402	total: 686ms	remaining: 21.2ms
97:	learn: 0.4025325	total: 694ms	remaining: 14.2ms
98:	learn: 0.4025235	t

40:	learn: 0.3750435	total: 372ms	remaining: 535ms
41:	learn: 0.3749644	total: 381ms	remaining: 527ms
42:	learn: 0.3747197	total: 389ms	remaining: 516ms
43:	learn: 0.3745042	total: 396ms	remaining: 504ms
44:	learn: 0.3743142	total: 402ms	remaining: 492ms
45:	learn: 0.3741552	total: 409ms	remaining: 481ms
46:	learn: 0.3741116	total: 416ms	remaining: 469ms
47:	learn: 0.3739569	total: 423ms	remaining: 458ms
48:	learn: 0.3738744	total: 430ms	remaining: 448ms
49:	learn: 0.3737755	total: 437ms	remaining: 437ms
50:	learn: 0.3736804	total: 444ms	remaining: 427ms
51:	learn: 0.3736482	total: 451ms	remaining: 416ms
52:	learn: 0.3735588	total: 458ms	remaining: 406ms
53:	learn: 0.3734877	total: 464ms	remaining: 396ms
54:	learn: 0.3734251	total: 471ms	remaining: 385ms
55:	learn: 0.3733639	total: 478ms	remaining: 376ms
56:	learn: 0.3733511	total: 484ms	remaining: 365ms
57:	learn: 0.3733043	total: 491ms	remaining: 355ms
58:	learn: 0.3732630	total: 497ms	remaining: 346ms
59:	learn: 0.3732564	total: 504

18:	learn: 0.4258971	total: 194ms	remaining: 829ms
19:	learn: 0.4232912	total: 205ms	remaining: 820ms
20:	learn: 0.4209847	total: 215ms	remaining: 811ms
21:	learn: 0.4189889	total: 226ms	remaining: 801ms
22:	learn: 0.4171419	total: 237ms	remaining: 792ms
23:	learn: 0.4156145	total: 247ms	remaining: 781ms
24:	learn: 0.4141654	total: 257ms	remaining: 770ms
25:	learn: 0.4128771	total: 267ms	remaining: 759ms
26:	learn: 0.4120518	total: 276ms	remaining: 747ms
27:	learn: 0.4110381	total: 286ms	remaining: 735ms
28:	learn: 0.4101527	total: 295ms	remaining: 723ms
29:	learn: 0.4093730	total: 306ms	remaining: 713ms
30:	learn: 0.4086476	total: 316ms	remaining: 704ms
31:	learn: 0.4081566	total: 326ms	remaining: 693ms
32:	learn: 0.4076103	total: 339ms	remaining: 688ms
33:	learn: 0.4071045	total: 349ms	remaining: 678ms
34:	learn: 0.4069832	total: 359ms	remaining: 668ms
35:	learn: 0.4065121	total: 369ms	remaining: 657ms
36:	learn: 0.4061257	total: 380ms	remaining: 646ms
37:	learn: 0.4058055	total: 391

0:	learn: 0.6489571	total: 12.4ms	remaining: 1.23s
1:	learn: 0.6127964	total: 24.3ms	remaining: 1.19s
2:	learn: 0.5829316	total: 35.7ms	remaining: 1.15s
3:	learn: 0.5581580	total: 46.9ms	remaining: 1.12s
4:	learn: 0.5369518	total: 58.3ms	remaining: 1.11s
5:	learn: 0.5189425	total: 69.6ms	remaining: 1.09s
6:	learn: 0.5035071	total: 79.7ms	remaining: 1.06s
7:	learn: 0.4905007	total: 89.4ms	remaining: 1.03s
8:	learn: 0.4790024	total: 99.6ms	remaining: 1.01s
9:	learn: 0.4692133	total: 110ms	remaining: 986ms
10:	learn: 0.4605885	total: 120ms	remaining: 967ms
11:	learn: 0.4533136	total: 129ms	remaining: 948ms
12:	learn: 0.4466634	total: 139ms	remaining: 932ms
13:	learn: 0.4408465	total: 149ms	remaining: 915ms
14:	learn: 0.4357289	total: 159ms	remaining: 899ms
15:	learn: 0.4312874	total: 168ms	remaining: 884ms
16:	learn: 0.4274065	total: 178ms	remaining: 870ms
17:	learn: 0.4239278	total: 188ms	remaining: 856ms
18:	learn: 0.4208547	total: 199ms	remaining: 850ms
19:	learn: 0.4181904	total: 210m

...

69:	learn: 0.3913369	total: 559ms	remaining: 239ms
70:	learn: 0.3913286	total: 567ms	remaining: 232ms
71:	learn: 0.3913165	total: 575ms	remaining: 224ms
72:	learn: 0.3912903	total: 583ms	remaining: 216ms
73:	learn: 0.3912655	total: 593ms	remaining: 208ms
74:	learn: 0.3912477	total: 601ms	remaining: 200ms
75:	learn: 0.3912404	total: 609ms	remaining: 192ms
76:	learn: 0.3912191	total: 617ms	remaining: 184ms
77:	learn: 0.3912145	total: 625ms	remaining: 176ms
78:	learn: 0.3912061	total: 632ms	remaining: 168ms
79:	learn: 0.3911932	total: 640ms	remaining: 160ms
80:	learn: 0.3911858	total: 647ms	remaining: 152ms
81:	learn: 0.3911774	total: 654ms	remaining: 143ms
82:	learn: 0.3911653	total: 660ms	remaining: 135ms
83:	learn: 0.3911463	total: 667ms	remaining: 127ms
84:	learn: 0.3911250	total: 673ms	remaining: 119ms
85:	learn: 0.3911121	total: 681ms	remaining: 111ms
86:	learn: 0.3910994	total: 687ms	remaining: 103ms
87:	learn: 0.3910931	total: 693ms	remaining: 94.6ms
88:	learn: 0.3910807	total: 70

48:	learn: 0.3946972	total: 370ms	remaining: 385ms
49:	learn: 0.3946774	total: 378ms	remaining: 378ms
50:	learn: 0.3945795	total: 387ms	remaining: 371ms
51:	learn: 0.3944903	total: 395ms	remaining: 364ms
52:	learn: 0.3944034	total: 402ms	remaining: 357ms
53:	learn: 0.3943715	total: 410ms	remaining: 349ms
54:	learn: 0.3943190	total: 417ms	remaining: 341ms
55:	learn: 0.3942520	total: 424ms	remaining: 333ms
56:	learn: 0.3942099	total: 431ms	remaining: 325ms
57:	learn: 0.3941733	total: 439ms	remaining: 318ms
58:	learn: 0.3941250	total: 448ms	remaining: 311ms
59:	learn: 0.3941166	total: 456ms	remaining: 304ms
60:	learn: 0.3940938	total: 464ms	remaining: 297ms
61:	learn: 0.3940401	total: 473ms	remaining: 290ms
62:	learn: 0.3940291	total: 481ms	remaining: 283ms
63:	learn: 0.3939961	total: 489ms	remaining: 275ms
64:	learn: 0.3939566	total: 497ms	remaining: 268ms
65:	learn: 0.3939452	total: 505ms	remaining: 260ms
66:	learn: 0.3939176	total: 513ms	remaining: 253ms
67:	learn: 0.3939069	total: 520

23:	learn: 0.4104443	total: 199ms	remaining: 630ms
24:	learn: 0.4089983	total: 207ms	remaining: 622ms
25:	learn: 0.4076692	total: 215ms	remaining: 612ms
26:	learn: 0.4067885	total: 223ms	remaining: 604ms
27:	learn: 0.4057351	total: 231ms	remaining: 594ms
28:	learn: 0.4048193	total: 239ms	remaining: 585ms
29:	learn: 0.4040084	total: 247ms	remaining: 577ms
30:	learn: 0.4032575	total: 255ms	remaining: 567ms
31:	learn: 0.4027465	total: 263ms	remaining: 559ms
32:	learn: 0.4021713	total: 271ms	remaining: 550ms
33:	learn: 0.4016394	total: 279ms	remaining: 542ms
34:	learn: 0.4012170	total: 287ms	remaining: 533ms
35:	learn: 0.4007701	total: 295ms	remaining: 524ms
36:	learn: 0.4003835	total: 303ms	remaining: 516ms
37:	learn: 0.4000748	total: 311ms	remaining: 507ms
38:	learn: 0.3998846	total: 319ms	remaining: 499ms
39:	learn: 0.3996189	total: 326ms	remaining: 489ms
40:	learn: 0.3995645	total: 334ms	remaining: 481ms
41:	learn: 0.3994963	total: 342ms	remaining: 473ms
42:	learn: 0.3992705	total: 350

0:	learn: 0.6486011	total: 10.2ms	remaining: 1.01s
1:	learn: 0.6121513	total: 20.3ms	remaining: 993ms
2:	learn: 0.5820690	total: 30.2ms	remaining: 975ms
3:	learn: 0.5570352	total: 40.2ms	remaining: 965ms
4:	learn: 0.5356889	total: 50.3ms	remaining: 956ms
5:	learn: 0.5175440	total: 60.5ms	remaining: 948ms
6:	learn: 0.5020148	total: 70.6ms	remaining: 938ms
7:	learn: 0.4889296	total: 80.9ms	remaining: 931ms
8:	learn: 0.4773405	total: 91.6ms	remaining: 927ms
9:	learn: 0.4674942	total: 102ms	remaining: 914ms
10:	learn: 0.4587825	total: 112ms	remaining: 903ms
11:	learn: 0.4514404	total: 122ms	remaining: 893ms
12:	learn: 0.4447438	total: 132ms	remaining: 884ms
13:	learn: 0.4389286	total: 142ms	remaining: 873ms
14:	learn: 0.4337538	total: 153ms	remaining: 867ms
15:	learn: 0.4293055	total: 163ms	remaining: 857ms
16:	learn: 0.4253936	total: 173ms	remaining: 846ms
17:	learn: 0.4218866	total: 184ms	remaining: 838ms
18:	learn: 0.4187975	total: 196ms	remaining: 835ms
19:	learn: 0.4161055	total: 208m

64:	learn: 0.3959555	total: 561ms	remaining: 302ms
65:	learn: 0.3959442	total: 567ms	remaining: 292ms
66:	learn: 0.3959154	total: 571ms	remaining: 281ms
67:	learn: 0.3958902	total: 576ms	remaining: 271ms
68:	learn: 0.3958705	total: 581ms	remaining: 261ms
69:	learn: 0.3958457	total: 587ms	remaining: 251ms
70:	learn: 0.3958322	total: 592ms	remaining: 242ms
71:	learn: 0.3958180	total: 597ms	remaining: 232ms
72:	learn: 0.3958100	total: 602ms	remaining: 223ms
73:	learn: 0.3957866	total: 606ms	remaining: 213ms
74:	learn: 0.3957813	total: 611ms	remaining: 204ms
75:	learn: 0.3957788	total: 617ms	remaining: 195ms
76:	learn: 0.3957593	total: 622ms	remaining: 186ms
77:	learn: 0.3957474	total: 627ms	remaining: 177ms
78:	learn: 0.3957325	total: 632ms	remaining: 168ms
79:	learn: 0.3957190	total: 636ms	remaining: 159ms
80:	learn: 0.3957028	total: 641ms	remaining: 150ms
81:	learn: 0.3956974	total: 646ms	remaining: 142ms
82:	learn: 0.3956881	total: 651ms	remaining: 133ms
83:	learn: 0.3956751	total: 656

In [45]:
# Volatility

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['Volatility']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6493356	total: 8.29ms	remaining: 821ms
1:	learn: 0.6137619	total: 16ms	remaining: 782ms
2:	learn: 0.5842512	total: 23ms	remaining: 744ms
3:	learn: 0.5593711	total: 30.4ms	remaining: 729ms
4:	learn: 0.5384716	total: 39.4ms	remaining: 749ms
5:	learn: 0.5206765	total: 47.2ms	remaining: 739ms
6:	learn: 0.5053282	total: 54.6ms	remaining: 725ms
7:	learn: 0.4923439	total: 61.7ms	remaining: 710ms
8:	learn: 0.4811591	total: 69.3ms	remaining: 701ms
9:	learn: 0.4715284	total: 76.4ms	remaining: 688ms
10:	learn: 0.4632673	total: 84.2ms	remaining: 681ms
11:	learn: 0.4557326	total: 91.7ms	remaining: 672ms
12:	learn: 0.4492247	total: 98.9ms	remaining: 662ms
13:	learn: 0.4434811	total: 107ms	remaining: 655ms
14:	learn: 0.4384709	total: 115ms	remaining: 651ms
15:	learn: 0.4340526	total: 123ms	remaining: 644ms
16:	learn: 0.4301236	total: 130ms	remaining: 635ms
17:	learn: 0.4266753	total: 138ms	remaining: 628ms
18:	learn: 0.4236568	total: 145ms	remaining: 617ms
19:	learn: 0.4212898	total: 151m

73:	learn: 0.4077144	total: 545ms	remaining: 191ms
74:	learn: 0.4077070	total: 552ms	remaining: 184ms
75:	learn: 0.4076968	total: 560ms	remaining: 177ms
76:	learn: 0.4076943	total: 568ms	remaining: 170ms
77:	learn: 0.4076893	total: 576ms	remaining: 162ms
78:	learn: 0.4076858	total: 583ms	remaining: 155ms
79:	learn: 0.4076832	total: 589ms	remaining: 147ms
80:	learn: 0.4076716	total: 597ms	remaining: 140ms
81:	learn: 0.4076692	total: 604ms	remaining: 132ms
82:	learn: 0.4076446	total: 611ms	remaining: 125ms
83:	learn: 0.4076380	total: 618ms	remaining: 118ms
84:	learn: 0.4076300	total: 624ms	remaining: 110ms
85:	learn: 0.4076219	total: 631ms	remaining: 103ms
86:	learn: 0.4076124	total: 638ms	remaining: 95.3ms
87:	learn: 0.4076101	total: 645ms	remaining: 87.9ms
88:	learn: 0.4076076	total: 652ms	remaining: 80.6ms
89:	learn: 0.4075904	total: 659ms	remaining: 73.2ms
90:	learn: 0.4075785	total: 666ms	remaining: 65.9ms
91:	learn: 0.4075684	total: 673ms	remaining: 58.5ms
92:	learn: 0.4075535	tota

35:	learn: 0.3799799	total: 379ms	remaining: 674ms
36:	learn: 0.3796007	total: 390ms	remaining: 664ms
37:	learn: 0.3792417	total: 400ms	remaining: 652ms
38:	learn: 0.3789411	total: 410ms	remaining: 641ms
39:	learn: 0.3786532	total: 421ms	remaining: 631ms
40:	learn: 0.3784173	total: 430ms	remaining: 619ms
41:	learn: 0.3782399	total: 441ms	remaining: 609ms
42:	learn: 0.3780593	total: 451ms	remaining: 598ms
43:	learn: 0.3779053	total: 461ms	remaining: 586ms
44:	learn: 0.3777669	total: 471ms	remaining: 576ms
45:	learn: 0.3777337	total: 482ms	remaining: 566ms
46:	learn: 0.3776389	total: 492ms	remaining: 555ms
47:	learn: 0.3775227	total: 502ms	remaining: 544ms
48:	learn: 0.3774105	total: 512ms	remaining: 533ms
49:	learn: 0.3773900	total: 522ms	remaining: 522ms
50:	learn: 0.3773118	total: 531ms	remaining: 511ms
51:	learn: 0.3772183	total: 541ms	remaining: 500ms
52:	learn: 0.3771860	total: 549ms	remaining: 487ms
53:	learn: 0.3771292	total: 557ms	remaining: 474ms
54:	learn: 0.3770700	total: 564

0:	learn: 0.6504893	total: 6.71ms	remaining: 664ms
1:	learn: 0.6158870	total: 12.5ms	remaining: 612ms
2:	learn: 0.5871682	total: 19ms	remaining: 613ms
3:	learn: 0.5633763	total: 25.7ms	remaining: 617ms
4:	learn: 0.5431048	total: 33.4ms	remaining: 635ms
5:	learn: 0.5256568	total: 41.6ms	remaining: 651ms
6:	learn: 0.5106760	total: 49.3ms	remaining: 655ms
7:	learn: 0.4983532	total: 57.1ms	remaining: 656ms
8:	learn: 0.4871799	total: 64.8ms	remaining: 655ms
9:	learn: 0.4775505	total: 72.5ms	remaining: 653ms
10:	learn: 0.4692540	total: 80.3ms	remaining: 649ms
11:	learn: 0.4619308	total: 88.3ms	remaining: 648ms
12:	learn: 0.4555472	total: 95.7ms	remaining: 641ms
13:	learn: 0.4499700	total: 103ms	remaining: 635ms
14:	learn: 0.4450253	total: 111ms	remaining: 628ms
15:	learn: 0.4409535	total: 118ms	remaining: 621ms
16:	learn: 0.4371375	total: 126ms	remaining: 614ms
17:	learn: 0.4338714	total: 133ms	remaining: 607ms
18:	learn: 0.4309399	total: 141ms	remaining: 599ms
19:	learn: 0.4282470	total: 14

71:	learn: 0.3920607	total: 558ms	remaining: 217ms
72:	learn: 0.3920467	total: 564ms	remaining: 208ms
73:	learn: 0.3920265	total: 569ms	remaining: 200ms
74:	learn: 0.3920083	total: 574ms	remaining: 191ms
75:	learn: 0.3919978	total: 579ms	remaining: 183ms
76:	learn: 0.3919883	total: 584ms	remaining: 175ms
77:	learn: 0.3919579	total: 590ms	remaining: 166ms
78:	learn: 0.3919541	total: 595ms	remaining: 158ms
79:	learn: 0.3919505	total: 600ms	remaining: 150ms
80:	learn: 0.3919446	total: 605ms	remaining: 142ms
81:	learn: 0.3919388	total: 611ms	remaining: 134ms
82:	learn: 0.3919276	total: 616ms	remaining: 126ms
83:	learn: 0.3919163	total: 623ms	remaining: 119ms
84:	learn: 0.3919069	total: 629ms	remaining: 111ms
85:	learn: 0.3918979	total: 634ms	remaining: 103ms
86:	learn: 0.3918858	total: 642ms	remaining: 95.9ms
87:	learn: 0.3918803	total: 647ms	remaining: 88.3ms
88:	learn: 0.3918789	total: 653ms	remaining: 80.7ms
89:	learn: 0.3918468	total: 659ms	remaining: 73.2ms
90:	learn: 0.3918386	total:

...

26:	learn: 0.4081753	total: 198ms	remaining: 535ms
27:	learn: 0.4071781	total: 205ms	remaining: 528ms
28:	learn: 0.4062064	total: 213ms	remaining: 522ms
29:	learn: 0.4052928	total: 221ms	remaining: 516ms
30:	learn: 0.4046150	total: 229ms	remaining: 509ms
31:	learn: 0.4041713	total: 236ms	remaining: 502ms
32:	learn: 0.4034899	total: 243ms	remaining: 494ms
33:	learn: 0.4028976	total: 251ms	remaining: 487ms
34:	learn: 0.4023799	total: 258ms	remaining: 479ms
35:	learn: 0.4019146	total: 265ms	remaining: 471ms
36:	learn: 0.4015051	total: 272ms	remaining: 463ms
37:	learn: 0.4011201	total: 279ms	remaining: 455ms
38:	learn: 0.4008046	total: 287ms	remaining: 448ms
39:	learn: 0.4005462	total: 294ms	remaining: 441ms
40:	learn: 0.4003004	total: 301ms	remaining: 433ms
41:	learn: 0.4000714	total: 308ms	remaining: 426ms
42:	learn: 0.3998841	total: 316ms	remaining: 418ms
43:	learn: 0.3997233	total: 323ms	remaining: 411ms
44:	learn: 0.3995632	total: 330ms	remaining: 403ms
45:	learn: 0.3994880	total: 337

0:	learn: 0.6497212	total: 7.96ms	remaining: 788ms
1:	learn: 0.6144522	total: 15.4ms	remaining: 756ms
2:	learn: 0.5851247	total: 22.8ms	remaining: 736ms
3:	learn: 0.5604904	total: 30.1ms	remaining: 722ms
4:	learn: 0.5394959	total: 37.6ms	remaining: 715ms
5:	learn: 0.5216409	total: 45.1ms	remaining: 707ms
6:	learn: 0.5064317	total: 52.7ms	remaining: 700ms
7:	learn: 0.4932478	total: 60ms	remaining: 690ms
8:	learn: 0.4818341	total: 67.4ms	remaining: 681ms
9:	learn: 0.4718660	total: 74.7ms	remaining: 672ms
10:	learn: 0.4632777	total: 82.2ms	remaining: 665ms
11:	learn: 0.4557189	total: 89.8ms	remaining: 658ms
12:	learn: 0.4491575	total: 96.8ms	remaining: 648ms
13:	learn: 0.4435551	total: 104ms	remaining: 640ms
14:	learn: 0.4384454	total: 112ms	remaining: 632ms
15:	learn: 0.4339150	total: 119ms	remaining: 623ms
16:	learn: 0.4302101	total: 126ms	remaining: 616ms
17:	learn: 0.4266962	total: 133ms	remaining: 607ms
18:	learn: 0.4239774	total: 141ms	remaining: 600ms
19:	learn: 0.4213602	total: 14

79:	learn: 0.3678400	total: 731ms	remaining: 183ms
80:	learn: 0.3678378	total: 738ms	remaining: 173ms
81:	learn: 0.3678273	total: 746ms	remaining: 164ms
82:	learn: 0.3678223	total: 753ms	remaining: 154ms
83:	learn: 0.3678171	total: 761ms	remaining: 145ms
84:	learn: 0.3678112	total: 768ms	remaining: 136ms
85:	learn: 0.3678044	total: 774ms	remaining: 126ms
86:	learn: 0.3677981	total: 782ms	remaining: 117ms
87:	learn: 0.3677969	total: 790ms	remaining: 108ms
88:	learn: 0.3677945	total: 797ms	remaining: 98.5ms
89:	learn: 0.3677806	total: 804ms	remaining: 89.3ms
90:	learn: 0.3677783	total: 812ms	remaining: 80.3ms
91:	learn: 0.3677700	total: 819ms	remaining: 71.2ms
92:	learn: 0.3677530	total: 827ms	remaining: 62.3ms
93:	learn: 0.3677501	total: 834ms	remaining: 53.2ms
94:	learn: 0.3677393	total: 841ms	remaining: 44.3ms
95:	learn: 0.3677310	total: 848ms	remaining: 35.3ms
96:	learn: 0.3677262	total: 855ms	remaining: 26.4ms
97:	learn: 0.3677144	total: 862ms	remaining: 17.6ms
98:	learn: 0.3676990	

47:	learn: 0.4017782	total: 365ms	remaining: 396ms
48:	learn: 0.4016741	total: 371ms	remaining: 386ms
49:	learn: 0.4016520	total: 376ms	remaining: 376ms
50:	learn: 0.4015869	total: 381ms	remaining: 366ms
51:	learn: 0.4015330	total: 387ms	remaining: 357ms
52:	learn: 0.4014950	total: 392ms	remaining: 348ms
53:	learn: 0.4014441	total: 398ms	remaining: 339ms
54:	learn: 0.4013906	total: 403ms	remaining: 330ms
55:	learn: 0.4013785	total: 408ms	remaining: 321ms
56:	learn: 0.4013335	total: 413ms	remaining: 312ms
57:	learn: 0.4013157	total: 418ms	remaining: 302ms
58:	learn: 0.4012983	total: 423ms	remaining: 294ms
59:	learn: 0.4012649	total: 428ms	remaining: 285ms
60:	learn: 0.4012354	total: 433ms	remaining: 277ms
61:	learn: 0.4012253	total: 438ms	remaining: 268ms
62:	learn: 0.4011879	total: 443ms	remaining: 260ms
63:	learn: 0.4011644	total: 448ms	remaining: 252ms
64:	learn: 0.4011330	total: 453ms	remaining: 244ms
65:	learn: 0.4011120	total: 458ms	remaining: 236ms
66:	learn: 0.4011018	total: 463

19:	learn: 0.4232020	total: 198ms	remaining: 792ms
20:	learn: 0.4207237	total: 208ms	remaining: 784ms
21:	learn: 0.4184809	total: 219ms	remaining: 775ms
22:	learn: 0.4165153	total: 229ms	remaining: 765ms
23:	learn: 0.4147011	total: 240ms	remaining: 759ms
24:	learn: 0.4132442	total: 248ms	remaining: 743ms
25:	learn: 0.4119666	total: 256ms	remaining: 729ms
26:	learn: 0.4106722	total: 262ms	remaining: 710ms
27:	learn: 0.4096508	total: 268ms	remaining: 689ms
28:	learn: 0.4087301	total: 273ms	remaining: 669ms
29:	learn: 0.4077850	total: 279ms	remaining: 650ms
30:	learn: 0.4070064	total: 284ms	remaining: 632ms
31:	learn: 0.4065514	total: 289ms	remaining: 614ms
32:	learn: 0.4058686	total: 294ms	remaining: 597ms
33:	learn: 0.4052642	total: 300ms	remaining: 582ms
34:	learn: 0.4047464	total: 306ms	remaining: 568ms
35:	learn: 0.4042436	total: 314ms	remaining: 557ms
36:	learn: 0.4038236	total: 321ms	remaining: 547ms
37:	learn: 0.4034410	total: 329ms	remaining: 536ms
38:	learn: 0.4031631	total: 337

In [46]:
# Shannon Entropy

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['ShEn']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6316394	total: 8.31ms	remaining: 823ms
1:	learn: 0.5814550	total: 16.2ms	remaining: 793ms
2:	learn: 0.5397875	total: 23.4ms	remaining: 756ms
3:	learn: 0.5046861	total: 31.1ms	remaining: 747ms
4:	learn: 0.4751167	total: 39ms	remaining: 741ms
5:	learn: 0.4500673	total: 47ms	remaining: 736ms
6:	learn: 0.4284712	total: 55.2ms	remaining: 733ms
7:	learn: 0.4098754	total: 63.6ms	remaining: 731ms
8:	learn: 0.3937820	total: 71.5ms	remaining: 723ms
9:	learn: 0.3802202	total: 79.5ms	remaining: 715ms
10:	learn: 0.3681981	total: 87ms	remaining: 704ms
11:	learn: 0.3576289	total: 95.2ms	remaining: 698ms
12:	learn: 0.3486155	total: 103ms	remaining: 688ms
13:	learn: 0.3405793	total: 111ms	remaining: 679ms
14:	learn: 0.3336891	total: 118ms	remaining: 669ms
15:	learn: 0.3276443	total: 125ms	remaining: 659ms
16:	learn: 0.3226772	total: 133ms	remaining: 648ms
17:	learn: 0.3180458	total: 140ms	remaining: 639ms
18:	learn: 0.3139962	total: 148ms	remaining: 630ms
19:	learn: 0.3103665	total: 155ms	r

72:	learn: 0.2804978	total: 576ms	remaining: 213ms
73:	learn: 0.2804633	total: 580ms	remaining: 204ms
74:	learn: 0.2803994	total: 586ms	remaining: 195ms
75:	learn: 0.2803155	total: 591ms	remaining: 187ms
76:	learn: 0.2802436	total: 597ms	remaining: 178ms
77:	learn: 0.2801554	total: 601ms	remaining: 170ms
78:	learn: 0.2800789	total: 606ms	remaining: 161ms
79:	learn: 0.2800278	total: 610ms	remaining: 153ms
80:	learn: 0.2799886	total: 615ms	remaining: 144ms
81:	learn: 0.2799136	total: 619ms	remaining: 136ms
82:	learn: 0.2798396	total: 623ms	remaining: 128ms
83:	learn: 0.2798114	total: 627ms	remaining: 120ms
84:	learn: 0.2797729	total: 632ms	remaining: 112ms
85:	learn: 0.2796944	total: 636ms	remaining: 104ms
86:	learn: 0.2796760	total: 641ms	remaining: 95.8ms
87:	learn: 0.2796271	total: 645ms	remaining: 88ms
88:	learn: 0.2795436	total: 649ms	remaining: 80.3ms
89:	learn: 0.2794863	total: 654ms	remaining: 72.6ms
90:	learn: 0.2794201	total: 658ms	remaining: 65.1ms
91:	learn: 0.2793647	total: 

43:	learn: 0.2788414	total: 382ms	remaining: 486ms
44:	learn: 0.2785552	total: 390ms	remaining: 477ms
45:	learn: 0.2782336	total: 399ms	remaining: 468ms
46:	learn: 0.2781347	total: 408ms	remaining: 460ms
47:	learn: 0.2778891	total: 417ms	remaining: 452ms
48:	learn: 0.2776722	total: 426ms	remaining: 443ms
49:	learn: 0.2775609	total: 435ms	remaining: 435ms
50:	learn: 0.2775152	total: 444ms	remaining: 426ms
51:	learn: 0.2773206	total: 452ms	remaining: 417ms
52:	learn: 0.2771347	total: 461ms	remaining: 409ms
53:	learn: 0.2769826	total: 469ms	remaining: 400ms
54:	learn: 0.2768127	total: 478ms	remaining: 391ms
55:	learn: 0.2766666	total: 485ms	remaining: 381ms
56:	learn: 0.2766190	total: 493ms	remaining: 372ms
57:	learn: 0.2764378	total: 501ms	remaining: 363ms
58:	learn: 0.2763675	total: 509ms	remaining: 354ms
59:	learn: 0.2762630	total: 517ms	remaining: 345ms
60:	learn: 0.2761495	total: 526ms	remaining: 336ms
61:	learn: 0.2760673	total: 535ms	remaining: 328ms
62:	learn: 0.2759482	total: 543

19:	learn: 0.3079639	total: 196ms	remaining: 786ms
20:	learn: 0.3046611	total: 207ms	remaining: 779ms
21:	learn: 0.3019212	total: 217ms	remaining: 771ms
22:	learn: 0.2998249	total: 227ms	remaining: 761ms
23:	learn: 0.2973979	total: 238ms	remaining: 753ms
24:	learn: 0.2955185	total: 248ms	remaining: 744ms
25:	learn: 0.2936687	total: 258ms	remaining: 734ms
26:	learn: 0.2918702	total: 267ms	remaining: 723ms
27:	learn: 0.2904648	total: 276ms	remaining: 711ms
28:	learn: 0.2890181	total: 286ms	remaining: 700ms
29:	learn: 0.2880586	total: 295ms	remaining: 689ms
30:	learn: 0.2869621	total: 306ms	remaining: 681ms
31:	learn: 0.2859080	total: 315ms	remaining: 670ms
32:	learn: 0.2848738	total: 324ms	remaining: 659ms
33:	learn: 0.2840930	total: 334ms	remaining: 648ms
34:	learn: 0.2833531	total: 343ms	remaining: 636ms
35:	learn: 0.2830713	total: 353ms	remaining: 628ms
36:	learn: 0.2827395	total: 363ms	remaining: 619ms
37:	learn: 0.2821756	total: 372ms	remaining: 607ms
38:	learn: 0.2816602	total: 381

0:	learn: 0.6324077	total: 6.4ms	remaining: 634ms
1:	learn: 0.5823486	total: 13.5ms	remaining: 660ms
2:	learn: 0.5413921	total: 19.9ms	remaining: 644ms
3:	learn: 0.5070631	total: 26.8ms	remaining: 642ms
4:	learn: 0.4778381	total: 33.9ms	remaining: 644ms
5:	learn: 0.4530204	total: 40.9ms	remaining: 641ms
6:	learn: 0.4315976	total: 48.2ms	remaining: 640ms
7:	learn: 0.4133060	total: 55.1ms	remaining: 633ms
8:	learn: 0.3976354	total: 62.7ms	remaining: 634ms
9:	learn: 0.3839011	total: 69.9ms	remaining: 629ms
10:	learn: 0.3719700	total: 77.1ms	remaining: 624ms
11:	learn: 0.3616370	total: 83.2ms	remaining: 610ms
12:	learn: 0.3529857	total: 89.1ms	remaining: 596ms
13:	learn: 0.3453207	total: 96.2ms	remaining: 591ms
14:	learn: 0.3384586	total: 104ms	remaining: 588ms
15:	learn: 0.3325627	total: 111ms	remaining: 581ms
16:	learn: 0.3277207	total: 117ms	remaining: 574ms
17:	learn: 0.3233197	total: 124ms	remaining: 566ms
18:	learn: 0.3191482	total: 131ms	remaining: 559ms
19:	learn: 0.3155459	total: 

...

77:	learn: 0.2847403	total: 556ms	remaining: 157ms
78:	learn: 0.2846925	total: 564ms	remaining: 150ms
79:	learn: 0.2846570	total: 572ms	remaining: 143ms
80:	learn: 0.2846077	total: 580ms	remaining: 136ms
81:	learn: 0.2845751	total: 587ms	remaining: 129ms
82:	learn: 0.2844753	total: 594ms	remaining: 122ms
83:	learn: 0.2843655	total: 599ms	remaining: 114ms
84:	learn: 0.2843327	total: 604ms	remaining: 107ms
85:	learn: 0.2842808	total: 608ms	remaining: 99ms
86:	learn: 0.2842433	total: 613ms	remaining: 91.6ms
87:	learn: 0.2842114	total: 617ms	remaining: 84.1ms
88:	learn: 0.2841552	total: 622ms	remaining: 76.8ms
89:	learn: 0.2840984	total: 626ms	remaining: 69.6ms
90:	learn: 0.2840684	total: 631ms	remaining: 62.4ms
91:	learn: 0.2840399	total: 635ms	remaining: 55.3ms
92:	learn: 0.2840059	total: 640ms	remaining: 48.2ms
93:	learn: 0.2839536	total: 645ms	remaining: 41.2ms
94:	learn: 0.2839187	total: 650ms	remaining: 34.2ms
95:	learn: 0.2838623	total: 654ms	remaining: 27.3ms
96:	learn: 0.2838513	t

55:	learn: 0.2760307	total: 411ms	remaining: 323ms
56:	learn: 0.2758843	total: 419ms	remaining: 316ms
57:	learn: 0.2756709	total: 427ms	remaining: 309ms
58:	learn: 0.2755857	total: 434ms	remaining: 302ms
59:	learn: 0.2754014	total: 441ms	remaining: 294ms
60:	learn: 0.2752603	total: 449ms	remaining: 287ms
61:	learn: 0.2750810	total: 456ms	remaining: 279ms
62:	learn: 0.2749306	total: 463ms	remaining: 272ms
63:	learn: 0.2748829	total: 470ms	remaining: 264ms
64:	learn: 0.2747135	total: 478ms	remaining: 257ms
65:	learn: 0.2746685	total: 484ms	remaining: 249ms
66:	learn: 0.2745834	total: 491ms	remaining: 242ms
67:	learn: 0.2745014	total: 498ms	remaining: 234ms
68:	learn: 0.2743868	total: 505ms	remaining: 227ms
69:	learn: 0.2742506	total: 512ms	remaining: 220ms
70:	learn: 0.2740978	total: 519ms	remaining: 212ms
71:	learn: 0.2740649	total: 527ms	remaining: 205ms
72:	learn: 0.2739880	total: 534ms	remaining: 198ms
73:	learn: 0.2738735	total: 542ms	remaining: 191ms
74:	learn: 0.2738003	total: 550

25:	learn: 0.2978984	total: 195ms	remaining: 554ms
26:	learn: 0.2961018	total: 201ms	remaining: 544ms
27:	learn: 0.2948049	total: 206ms	remaining: 531ms
28:	learn: 0.2936520	total: 212ms	remaining: 519ms
29:	learn: 0.2929711	total: 217ms	remaining: 506ms
30:	learn: 0.2917744	total: 222ms	remaining: 493ms
31:	learn: 0.2908379	total: 227ms	remaining: 483ms
32:	learn: 0.2898776	total: 232ms	remaining: 472ms
33:	learn: 0.2892215	total: 238ms	remaining: 461ms
34:	learn: 0.2886389	total: 244ms	remaining: 453ms
35:	learn: 0.2879370	total: 250ms	remaining: 445ms
36:	learn: 0.2877079	total: 257ms	remaining: 437ms
37:	learn: 0.2870533	total: 263ms	remaining: 429ms
38:	learn: 0.2864795	total: 269ms	remaining: 421ms
39:	learn: 0.2859935	total: 276ms	remaining: 413ms
40:	learn: 0.2855702	total: 282ms	remaining: 406ms
41:	learn: 0.2851254	total: 288ms	remaining: 398ms
42:	learn: 0.2847005	total: 294ms	remaining: 390ms
43:	learn: 0.2843408	total: 301ms	remaining: 383ms
44:	learn: 0.2840815	total: 307

0:	learn: 0.6317389	total: 8.46ms	remaining: 838ms
1:	learn: 0.5816259	total: 16.9ms	remaining: 828ms
2:	learn: 0.5398516	total: 24.9ms	remaining: 806ms
3:	learn: 0.5048180	total: 33.3ms	remaining: 799ms
4:	learn: 0.4752996	total: 41.3ms	remaining: 785ms
5:	learn: 0.4502817	total: 49.3ms	remaining: 773ms
6:	learn: 0.4287508	total: 56.8ms	remaining: 755ms
7:	learn: 0.4101603	total: 65.1ms	remaining: 748ms
8:	learn: 0.3941015	total: 73.9ms	remaining: 747ms
9:	learn: 0.3805765	total: 81.7ms	remaining: 735ms
10:	learn: 0.3685801	total: 89.5ms	remaining: 724ms
11:	learn: 0.3580943	total: 97.6ms	remaining: 716ms
12:	learn: 0.3490987	total: 106ms	remaining: 708ms
13:	learn: 0.3411564	total: 114ms	remaining: 698ms
14:	learn: 0.3343304	total: 122ms	remaining: 690ms
15:	learn: 0.3283234	total: 130ms	remaining: 680ms
16:	learn: 0.3233652	total: 137ms	remaining: 671ms
17:	learn: 0.3187485	total: 146ms	remaining: 663ms
18:	learn: 0.3147155	total: 154ms	remaining: 654ms
19:	learn: 0.3111525	total: 1

63:	learn: 0.2875363	total: 566ms	remaining: 318ms
64:	learn: 0.2874661	total: 572ms	remaining: 308ms
65:	learn: 0.2874375	total: 576ms	remaining: 297ms
66:	learn: 0.2873842	total: 581ms	remaining: 286ms
67:	learn: 0.2873206	total: 586ms	remaining: 276ms
68:	learn: 0.2872206	total: 594ms	remaining: 267ms
69:	learn: 0.2871126	total: 602ms	remaining: 258ms
70:	learn: 0.2869963	total: 609ms	remaining: 249ms
71:	learn: 0.2869302	total: 616ms	remaining: 240ms
72:	learn: 0.2868772	total: 623ms	remaining: 230ms
73:	learn: 0.2868460	total: 631ms	remaining: 222ms
74:	learn: 0.2867883	total: 638ms	remaining: 213ms
75:	learn: 0.2867191	total: 645ms	remaining: 204ms
76:	learn: 0.2866652	total: 652ms	remaining: 195ms
77:	learn: 0.2866225	total: 658ms	remaining: 186ms
78:	learn: 0.2865676	total: 664ms	remaining: 177ms
79:	learn: 0.2865237	total: 670ms	remaining: 168ms
80:	learn: 0.2864859	total: 676ms	remaining: 159ms
81:	learn: 0.2864553	total: 683ms	remaining: 150ms
82:	learn: 0.2863814	total: 690

In [47]:
# Approximate Entropy

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['AppEn']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6261789	total: 8ms	remaining: 792ms
1:	learn: 0.5712487	total: 16.6ms	remaining: 812ms
2:	learn: 0.5257331	total: 25ms	remaining: 808ms
3:	learn: 0.4872373	total: 32.9ms	remaining: 789ms
4:	learn: 0.4544441	total: 41.9ms	remaining: 795ms
5:	learn: 0.4263090	total: 50.7ms	remaining: 795ms
6:	learn: 0.4020811	total: 58.2ms	remaining: 774ms
7:	learn: 0.3812626	total: 66.3ms	remaining: 762ms
8:	learn: 0.3633119	total: 73.8ms	remaining: 746ms
9:	learn: 0.3475391	total: 81.3ms	remaining: 732ms
10:	learn: 0.3338803	total: 88.9ms	remaining: 719ms
11:	learn: 0.3216934	total: 96.3ms	remaining: 706ms
12:	learn: 0.3109789	total: 104ms	remaining: 694ms
13:	learn: 0.3019155	total: 111ms	remaining: 682ms
14:	learn: 0.2936138	total: 118ms	remaining: 670ms
15:	learn: 0.2862267	total: 125ms	remaining: 658ms
16:	learn: 0.2797139	total: 133ms	remaining: 648ms
17:	learn: 0.2740219	total: 140ms	remaining: 639ms
18:	learn: 0.2689846	total: 148ms	remaining: 630ms
19:	learn: 0.2646629	total: 155ms	

72:	learn: 0.2332651	total: 547ms	remaining: 202ms
73:	learn: 0.2332297	total: 555ms	remaining: 195ms
74:	learn: 0.2331948	total: 563ms	remaining: 188ms
75:	learn: 0.2331388	total: 571ms	remaining: 180ms
76:	learn: 0.2331263	total: 578ms	remaining: 173ms
77:	learn: 0.2330639	total: 585ms	remaining: 165ms
78:	learn: 0.2330475	total: 592ms	remaining: 157ms
79:	learn: 0.2330048	total: 599ms	remaining: 150ms
80:	learn: 0.2329752	total: 606ms	remaining: 142ms
81:	learn: 0.2329294	total: 614ms	remaining: 135ms
82:	learn: 0.2328830	total: 621ms	remaining: 127ms
83:	learn: 0.2328673	total: 628ms	remaining: 120ms
84:	learn: 0.2327695	total: 635ms	remaining: 112ms
85:	learn: 0.2326878	total: 641ms	remaining: 104ms
86:	learn: 0.2326759	total: 648ms	remaining: 96.9ms
87:	learn: 0.2325986	total: 655ms	remaining: 89.3ms
88:	learn: 0.2325475	total: 662ms	remaining: 81.8ms
89:	learn: 0.2325335	total: 669ms	remaining: 74.3ms
90:	learn: 0.2324967	total: 676ms	remaining: 66.9ms
91:	learn: 0.2324769	total

47:	learn: 0.2176538	total: 371ms	remaining: 402ms
48:	learn: 0.2174939	total: 379ms	remaining: 395ms
49:	learn: 0.2172532	total: 388ms	remaining: 388ms
50:	learn: 0.2169490	total: 396ms	remaining: 380ms
51:	learn: 0.2168705	total: 403ms	remaining: 372ms
52:	learn: 0.2167180	total: 411ms	remaining: 364ms
53:	learn: 0.2164880	total: 418ms	remaining: 356ms
54:	learn: 0.2162997	total: 426ms	remaining: 348ms
55:	learn: 0.2161043	total: 433ms	remaining: 340ms
56:	learn: 0.2159461	total: 440ms	remaining: 332ms
57:	learn: 0.2158140	total: 447ms	remaining: 323ms
58:	learn: 0.2157298	total: 454ms	remaining: 315ms
59:	learn: 0.2156045	total: 461ms	remaining: 307ms
60:	learn: 0.2155674	total: 468ms	remaining: 300ms
61:	learn: 0.2154776	total: 476ms	remaining: 292ms
62:	learn: 0.2153823	total: 483ms	remaining: 284ms
63:	learn: 0.2153246	total: 491ms	remaining: 276ms
64:	learn: 0.2153002	total: 498ms	remaining: 268ms
65:	learn: 0.2152268	total: 507ms	remaining: 261ms
66:	learn: 0.2151628	total: 514

21:	learn: 0.2629191	total: 198ms	remaining: 701ms
22:	learn: 0.2598907	total: 207ms	remaining: 694ms
23:	learn: 0.2571529	total: 217ms	remaining: 686ms
24:	learn: 0.2546570	total: 226ms	remaining: 678ms
25:	learn: 0.2524597	total: 234ms	remaining: 666ms
26:	learn: 0.2509057	total: 243ms	remaining: 657ms
27:	learn: 0.2492680	total: 251ms	remaining: 646ms
28:	learn: 0.2479576	total: 259ms	remaining: 635ms
29:	learn: 0.2464703	total: 268ms	remaining: 625ms
30:	learn: 0.2452528	total: 276ms	remaining: 615ms
31:	learn: 0.2438046	total: 285ms	remaining: 606ms
32:	learn: 0.2429451	total: 294ms	remaining: 596ms
33:	learn: 0.2420454	total: 302ms	remaining: 586ms
34:	learn: 0.2410866	total: 310ms	remaining: 576ms
35:	learn: 0.2402521	total: 319ms	remaining: 567ms
36:	learn: 0.2398081	total: 327ms	remaining: 557ms
37:	learn: 0.2391035	total: 335ms	remaining: 547ms
38:	learn: 0.2386934	total: 344ms	remaining: 538ms
39:	learn: 0.2380946	total: 352ms	remaining: 529ms
40:	learn: 0.2376619	total: 361

85:	learn: 0.2188430	total: 764ms	remaining: 124ms
86:	learn: 0.2188185	total: 771ms	remaining: 115ms
87:	learn: 0.2187732	total: 779ms	remaining: 106ms
88:	learn: 0.2187234	total: 787ms	remaining: 97.3ms
89:	learn: 0.2186420	total: 794ms	remaining: 88.2ms
90:	learn: 0.2186089	total: 801ms	remaining: 79.2ms
91:	learn: 0.2185983	total: 808ms	remaining: 70.2ms
92:	learn: 0.2185562	total: 815ms	remaining: 61.3ms
93:	learn: 0.2185512	total: 821ms	remaining: 52.4ms
94:	learn: 0.2184846	total: 828ms	remaining: 43.6ms
95:	learn: 0.2184793	total: 835ms	remaining: 34.8ms
96:	learn: 0.2184744	total: 842ms	remaining: 26ms
97:	learn: 0.2184234	total: 850ms	remaining: 17.3ms
98:	learn: 0.2184081	total: 856ms	remaining: 8.65ms
99:	learn: 0.2183720	total: 863ms	remaining: 0us
0:	learn: 0.6271393	total: 9.94ms	remaining: 985ms
1:	learn: 0.5734652	total: 19.7ms	remaining: 963ms
2:	learn: 0.5287181	total: 29.1ms	remaining: 940ms
3:	learn: 0.4912418	total: 39ms	remaining: 936ms
4:	learn: 0.4594249	total:

...

39:	learn: 0.2320804	total: 240ms	remaining: 361ms
40:	learn: 0.2314547	total: 248ms	remaining: 357ms
41:	learn: 0.2311416	total: 256ms	remaining: 353ms
42:	learn: 0.2308092	total: 264ms	remaining: 350ms
43:	learn: 0.2304268	total: 272ms	remaining: 346ms
44:	learn: 0.2300091	total: 279ms	remaining: 341ms
45:	learn: 0.2296542	total: 287ms	remaining: 337ms
46:	learn: 0.2295541	total: 295ms	remaining: 332ms
47:	learn: 0.2292130	total: 302ms	remaining: 328ms
48:	learn: 0.2288754	total: 309ms	remaining: 322ms
49:	learn: 0.2286664	total: 316ms	remaining: 316ms
50:	learn: 0.2285354	total: 323ms	remaining: 311ms
51:	learn: 0.2284800	total: 330ms	remaining: 305ms
52:	learn: 0.2283514	total: 337ms	remaining: 299ms
53:	learn: 0.2281818	total: 344ms	remaining: 293ms
54:	learn: 0.2280764	total: 351ms	remaining: 287ms
55:	learn: 0.2279378	total: 358ms	remaining: 281ms
56:	learn: 0.2277716	total: 365ms	remaining: 275ms
57:	learn: 0.2276617	total: 372ms	remaining: 270ms
58:	learn: 0.2275559	total: 380

0:	learn: 0.6272384	total: 10.5ms	remaining: 1.04s
1:	learn: 0.5731903	total: 20.6ms	remaining: 1.01s
2:	learn: 0.5291257	total: 31.5ms	remaining: 1.02s
3:	learn: 0.4913894	total: 42ms	remaining: 1.01s
4:	learn: 0.4593759	total: 52ms	remaining: 987ms
5:	learn: 0.4318338	total: 62.4ms	remaining: 977ms
6:	learn: 0.4081239	total: 72.7ms	remaining: 966ms
7:	learn: 0.3876194	total: 83.6ms	remaining: 961ms
8:	learn: 0.3699058	total: 94.4ms	remaining: 955ms
9:	learn: 0.3546004	total: 104ms	remaining: 940ms
10:	learn: 0.3410154	total: 115ms	remaining: 926ms
11:	learn: 0.3291688	total: 125ms	remaining: 917ms
12:	learn: 0.3188936	total: 136ms	remaining: 908ms
13:	learn: 0.3098606	total: 147ms	remaining: 903ms
14:	learn: 0.3018899	total: 158ms	remaining: 893ms
15:	learn: 0.2947596	total: 168ms	remaining: 880ms
16:	learn: 0.2884375	total: 178ms	remaining: 867ms
17:	learn: 0.2829692	total: 190ms	remaining: 864ms
18:	learn: 0.2779061	total: 201ms	remaining: 856ms
19:	learn: 0.2735309	total: 212ms	re

72:	learn: 0.2090612	total: 546ms	remaining: 202ms
73:	learn: 0.2090301	total: 553ms	remaining: 194ms
74:	learn: 0.2089683	total: 561ms	remaining: 187ms
75:	learn: 0.2089569	total: 569ms	remaining: 180ms
76:	learn: 0.2088966	total: 576ms	remaining: 172ms
77:	learn: 0.2088355	total: 583ms	remaining: 164ms
78:	learn: 0.2088231	total: 590ms	remaining: 157ms
79:	learn: 0.2087769	total: 597ms	remaining: 149ms
80:	learn: 0.2087444	total: 604ms	remaining: 142ms
81:	learn: 0.2086905	total: 611ms	remaining: 134ms
82:	learn: 0.2086396	total: 618ms	remaining: 127ms
83:	learn: 0.2085965	total: 625ms	remaining: 119ms
84:	learn: 0.2085785	total: 632ms	remaining: 111ms
85:	learn: 0.2085371	total: 639ms	remaining: 104ms
86:	learn: 0.2084885	total: 645ms	remaining: 96.4ms
87:	learn: 0.2084540	total: 652ms	remaining: 88.9ms
88:	learn: 0.2084151	total: 660ms	remaining: 81.5ms
89:	learn: 0.2083721	total: 668ms	remaining: 74.2ms
90:	learn: 0.2083349	total: 675ms	remaining: 66.8ms
91:	learn: 0.2083266	total

49:	learn: 0.2384258	total: 372ms	remaining: 372ms
50:	learn: 0.2382713	total: 380ms	remaining: 365ms
51:	learn: 0.2381239	total: 387ms	remaining: 357ms
52:	learn: 0.2380174	total: 395ms	remaining: 351ms
53:	learn: 0.2378304	total: 403ms	remaining: 344ms
54:	learn: 0.2376324	total: 411ms	remaining: 336ms
55:	learn: 0.2374362	total: 418ms	remaining: 329ms
56:	learn: 0.2372929	total: 426ms	remaining: 322ms
57:	learn: 0.2372613	total: 433ms	remaining: 314ms
58:	learn: 0.2371796	total: 440ms	remaining: 306ms
59:	learn: 0.2370482	total: 448ms	remaining: 299ms
60:	learn: 0.2369405	total: 455ms	remaining: 291ms
61:	learn: 0.2368240	total: 462ms	remaining: 283ms
62:	learn: 0.2366882	total: 469ms	remaining: 275ms
63:	learn: 0.2366259	total: 476ms	remaining: 268ms
64:	learn: 0.2365330	total: 483ms	remaining: 260ms
65:	learn: 0.2364775	total: 490ms	remaining: 252ms
66:	learn: 0.2364044	total: 497ms	remaining: 245ms
67:	learn: 0.2363109	total: 504ms	remaining: 237ms
68:	learn: 0.2362667	total: 511

20:	learn: 0.2667258	total: 198ms	remaining: 743ms
21:	learn: 0.2633730	total: 208ms	remaining: 737ms
22:	learn: 0.2603075	total: 218ms	remaining: 729ms
23:	learn: 0.2575647	total: 227ms	remaining: 720ms
24:	learn: 0.2552719	total: 237ms	remaining: 710ms
25:	learn: 0.2531976	total: 246ms	remaining: 699ms
26:	learn: 0.2518438	total: 255ms	remaining: 689ms
27:	learn: 0.2500865	total: 264ms	remaining: 678ms
28:	learn: 0.2486534	total: 273ms	remaining: 668ms
29:	learn: 0.2472364	total: 282ms	remaining: 658ms
30:	learn: 0.2459910	total: 291ms	remaining: 648ms
31:	learn: 0.2447153	total: 300ms	remaining: 638ms
32:	learn: 0.2440088	total: 309ms	remaining: 628ms
33:	learn: 0.2431118	total: 319ms	remaining: 618ms
34:	learn: 0.2423463	total: 328ms	remaining: 609ms
35:	learn: 0.2416623	total: 337ms	remaining: 600ms
36:	learn: 0.2411160	total: 347ms	remaining: 591ms
37:	learn: 0.2404079	total: 356ms	remaining: 581ms
38:	learn: 0.2397683	total: 365ms	remaining: 572ms
39:	learn: 0.2392092	total: 375

In [48]:
# Sample Entropy

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['SampEn']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6258948	total: 12.8ms	remaining: 1.27s
1:	learn: 0.5710082	total: 24.4ms	remaining: 1.19s
2:	learn: 0.5254600	total: 34.7ms	remaining: 1.12s
3:	learn: 0.4867048	total: 44.8ms	remaining: 1.07s
4:	learn: 0.4537089	total: 56.2ms	remaining: 1.07s
5:	learn: 0.4253330	total: 66.7ms	remaining: 1.04s
6:	learn: 0.4009489	total: 77.2ms	remaining: 1.02s
7:	learn: 0.3802042	total: 87.3ms	remaining: 1s
8:	learn: 0.3618932	total: 97.6ms	remaining: 987ms
9:	learn: 0.3460477	total: 108ms	remaining: 969ms
10:	learn: 0.3322905	total: 118ms	remaining: 956ms
11:	learn: 0.3201348	total: 126ms	remaining: 923ms
12:	learn: 0.3093666	total: 133ms	remaining: 892ms
13:	learn: 0.3006065	total: 140ms	remaining: 861ms
14:	learn: 0.2921771	total: 146ms	remaining: 826ms
15:	learn: 0.2848930	total: 151ms	remaining: 794ms
16:	learn: 0.2783088	total: 157ms	remaining: 766ms
17:	learn: 0.2725334	total: 162ms	remaining: 739ms
18:	learn: 0.2674698	total: 167ms	remaining: 714ms
19:	learn: 0.2628828	total: 174ms	r

77:	learn: 0.2345836	total: 542ms	remaining: 153ms
78:	learn: 0.2345756	total: 550ms	remaining: 146ms
79:	learn: 0.2345585	total: 556ms	remaining: 139ms
80:	learn: 0.2345319	total: 562ms	remaining: 132ms
81:	learn: 0.2345275	total: 567ms	remaining: 125ms
82:	learn: 0.2344662	total: 572ms	remaining: 117ms
83:	learn: 0.2344525	total: 580ms	remaining: 110ms
84:	learn: 0.2344437	total: 588ms	remaining: 104ms
85:	learn: 0.2344368	total: 596ms	remaining: 97ms
86:	learn: 0.2343325	total: 603ms	remaining: 90.1ms
87:	learn: 0.2343139	total: 610ms	remaining: 83.2ms
88:	learn: 0.2342256	total: 618ms	remaining: 76.4ms
89:	learn: 0.2341953	total: 627ms	remaining: 69.6ms
90:	learn: 0.2341942	total: 635ms	remaining: 62.8ms
91:	learn: 0.2341722	total: 644ms	remaining: 56ms
92:	learn: 0.2341456	total: 652ms	remaining: 49.1ms
93:	learn: 0.2341278	total: 660ms	remaining: 42.1ms
94:	learn: 0.2341174	total: 667ms	remaining: 35.1ms
95:	learn: 0.2340974	total: 675ms	remaining: 28.1ms
96:	learn: 0.2340900	tot

60:	learn: 0.2164498	total: 580ms	remaining: 371ms
61:	learn: 0.2163671	total: 589ms	remaining: 361ms
62:	learn: 0.2162917	total: 597ms	remaining: 351ms
63:	learn: 0.2162434	total: 606ms	remaining: 341ms
64:	learn: 0.2161550	total: 615ms	remaining: 331ms
65:	learn: 0.2161323	total: 623ms	remaining: 321ms
66:	learn: 0.2160908	total: 630ms	remaining: 310ms
67:	learn: 0.2160221	total: 638ms	remaining: 300ms
68:	learn: 0.2159881	total: 646ms	remaining: 290ms
69:	learn: 0.2159495	total: 654ms	remaining: 280ms
70:	learn: 0.2159153	total: 662ms	remaining: 271ms
71:	learn: 0.2159004	total: 671ms	remaining: 261ms
72:	learn: 0.2158883	total: 678ms	remaining: 251ms
73:	learn: 0.2157854	total: 686ms	remaining: 241ms
74:	learn: 0.2157002	total: 694ms	remaining: 231ms
75:	learn: 0.2156866	total: 704ms	remaining: 222ms
76:	learn: 0.2156690	total: 712ms	remaining: 213ms
77:	learn: 0.2156415	total: 720ms	remaining: 203ms
78:	learn: 0.2156332	total: 728ms	remaining: 194ms
79:	learn: 0.2155535	total: 735

46:	learn: 0.2377516	total: 390ms	remaining: 440ms
47:	learn: 0.2376079	total: 398ms	remaining: 431ms
48:	learn: 0.2374573	total: 406ms	remaining: 422ms
49:	learn: 0.2373279	total: 413ms	remaining: 413ms
50:	learn: 0.2371972	total: 421ms	remaining: 404ms
51:	learn: 0.2369958	total: 428ms	remaining: 395ms
52:	learn: 0.2369474	total: 435ms	remaining: 386ms
53:	learn: 0.2368462	total: 442ms	remaining: 377ms
54:	learn: 0.2367149	total: 449ms	remaining: 368ms
55:	learn: 0.2366412	total: 456ms	remaining: 358ms
56:	learn: 0.2365899	total: 463ms	remaining: 349ms
57:	learn: 0.2365155	total: 470ms	remaining: 340ms
58:	learn: 0.2364410	total: 477ms	remaining: 331ms
59:	learn: 0.2362963	total: 484ms	remaining: 323ms
60:	learn: 0.2362352	total: 491ms	remaining: 314ms
61:	learn: 0.2362174	total: 499ms	remaining: 306ms
62:	learn: 0.2361813	total: 506ms	remaining: 297ms
63:	learn: 0.2360977	total: 514ms	remaining: 289ms
64:	learn: 0.2360677	total: 521ms	remaining: 280ms
65:	learn: 0.2360115	total: 528

20:	learn: 0.2667693	total: 205ms	remaining: 771ms
21:	learn: 0.2634548	total: 215ms	remaining: 763ms
22:	learn: 0.2604060	total: 225ms	remaining: 754ms
23:	learn: 0.2576672	total: 235ms	remaining: 745ms
24:	learn: 0.2553679	total: 245ms	remaining: 734ms
25:	learn: 0.2533710	total: 254ms	remaining: 724ms
26:	learn: 0.2518810	total: 264ms	remaining: 715ms
27:	learn: 0.2502860	total: 274ms	remaining: 704ms
28:	learn: 0.2489802	total: 283ms	remaining: 692ms
29:	learn: 0.2476117	total: 290ms	remaining: 677ms
30:	learn: 0.2465753	total: 298ms	remaining: 663ms
31:	learn: 0.2455041	total: 305ms	remaining: 648ms
32:	learn: 0.2445563	total: 312ms	remaining: 634ms
33:	learn: 0.2437428	total: 319ms	remaining: 620ms
34:	learn: 0.2430124	total: 326ms	remaining: 606ms
35:	learn: 0.2424389	total: 333ms	remaining: 593ms
36:	learn: 0.2419741	total: 340ms	remaining: 580ms
37:	learn: 0.2414977	total: 347ms	remaining: 567ms
38:	learn: 0.2410291	total: 354ms	remaining: 554ms
39:	learn: 0.2406108	total: 361

...

64:	learn: 0.2286480	total: 573ms	remaining: 308ms
65:	learn: 0.2286374	total: 580ms	remaining: 299ms
66:	learn: 0.2286055	total: 588ms	remaining: 289ms
67:	learn: 0.2284936	total: 595ms	remaining: 280ms
68:	learn: 0.2284418	total: 603ms	remaining: 271ms
69:	learn: 0.2284023	total: 611ms	remaining: 262ms
70:	learn: 0.2282936	total: 619ms	remaining: 253ms
71:	learn: 0.2281905	total: 626ms	remaining: 243ms
72:	learn: 0.2281595	total: 633ms	remaining: 234ms
73:	learn: 0.2281098	total: 641ms	remaining: 225ms
74:	learn: 0.2280239	total: 648ms	remaining: 216ms
75:	learn: 0.2280089	total: 655ms	remaining: 207ms
76:	learn: 0.2279769	total: 662ms	remaining: 198ms
77:	learn: 0.2279649	total: 669ms	remaining: 189ms
78:	learn: 0.2279049	total: 676ms	remaining: 180ms
79:	learn: 0.2278089	total: 684ms	remaining: 171ms
80:	learn: 0.2277763	total: 690ms	remaining: 162ms
81:	learn: 0.2277690	total: 697ms	remaining: 153ms
82:	learn: 0.2277567	total: 705ms	remaining: 144ms
83:	learn: 0.2277350	total: 712

48:	learn: 0.2403225	total: 371ms	remaining: 386ms
49:	learn: 0.2401744	total: 379ms	remaining: 379ms
50:	learn: 0.2400338	total: 388ms	remaining: 373ms
51:	learn: 0.2398125	total: 397ms	remaining: 367ms
52:	learn: 0.2396528	total: 406ms	remaining: 360ms
53:	learn: 0.2395812	total: 415ms	remaining: 354ms
54:	learn: 0.2394490	total: 423ms	remaining: 346ms
55:	learn: 0.2394001	total: 431ms	remaining: 338ms
56:	learn: 0.2393167	total: 440ms	remaining: 332ms
57:	learn: 0.2392378	total: 448ms	remaining: 324ms
58:	learn: 0.2392111	total: 455ms	remaining: 316ms
59:	learn: 0.2392035	total: 462ms	remaining: 308ms
60:	learn: 0.2391495	total: 468ms	remaining: 299ms
61:	learn: 0.2390151	total: 475ms	remaining: 291ms
62:	learn: 0.2389650	total: 481ms	remaining: 283ms
63:	learn: 0.2389445	total: 488ms	remaining: 275ms
64:	learn: 0.2388919	total: 495ms	remaining: 266ms
65:	learn: 0.2388452	total: 502ms	remaining: 258ms
66:	learn: 0.2388133	total: 508ms	remaining: 250ms
67:	learn: 0.2387509	total: 514

16:	learn: 0.2831297	total: 210ms	remaining: 1.02s
17:	learn: 0.2775402	total: 222ms	remaining: 1.01s
18:	learn: 0.2726217	total: 235ms	remaining: 1s
19:	learn: 0.2681469	total: 248ms	remaining: 992ms
20:	learn: 0.2642070	total: 260ms	remaining: 978ms
21:	learn: 0.2612084	total: 272ms	remaining: 963ms
22:	learn: 0.2581439	total: 283ms	remaining: 949ms
23:	learn: 0.2554078	total: 294ms	remaining: 932ms
24:	learn: 0.2530448	total: 306ms	remaining: 917ms
25:	learn: 0.2509097	total: 318ms	remaining: 904ms
26:	learn: 0.2494206	total: 330ms	remaining: 891ms
27:	learn: 0.2478165	total: 341ms	remaining: 877ms
28:	learn: 0.2463556	total: 352ms	remaining: 862ms
29:	learn: 0.2450362	total: 364ms	remaining: 850ms
30:	learn: 0.2439210	total: 376ms	remaining: 837ms
31:	learn: 0.2428165	total: 388ms	remaining: 824ms
32:	learn: 0.2419426	total: 403ms	remaining: 817ms
33:	learn: 0.2411303	total: 415ms	remaining: 805ms
34:	learn: 0.2404151	total: 426ms	remaining: 791ms
35:	learn: 0.2398302	total: 438ms	

93:	learn: 0.2370673	total: 764ms	remaining: 48.8ms
94:	learn: 0.2370551	total: 773ms	remaining: 40.7ms
95:	learn: 0.2370485	total: 783ms	remaining: 32.6ms
96:	learn: 0.2370466	total: 792ms	remaining: 24.5ms
97:	learn: 0.2369977	total: 802ms	remaining: 16.4ms
98:	learn: 0.2369770	total: 810ms	remaining: 8.18ms
99:	learn: 0.2369252	total: 819ms	remaining: 0us
0:	learn: 0.6264245	total: 8.99ms	remaining: 890ms
1:	learn: 0.5715897	total: 18.3ms	remaining: 899ms
2:	learn: 0.5264481	total: 26.3ms	remaining: 851ms
3:	learn: 0.4881328	total: 34.2ms	remaining: 821ms
4:	learn: 0.4553350	total: 41.6ms	remaining: 791ms
5:	learn: 0.4274497	total: 49.1ms	remaining: 769ms
6:	learn: 0.4032442	total: 56.5ms	remaining: 751ms
7:	learn: 0.3826344	total: 64.3ms	remaining: 739ms
8:	learn: 0.3644614	total: 71.6ms	remaining: 724ms
9:	learn: 0.3488294	total: 78.5ms	remaining: 707ms
10:	learn: 0.3352949	total: 86ms	remaining: 696ms
11:	learn: 0.3232935	total: 93.5ms	remaining: 686ms
12:	learn: 0.3125995	total:

75:	learn: 0.2348385	total: 566ms	remaining: 179ms
76:	learn: 0.2348157	total: 573ms	remaining: 171ms
77:	learn: 0.2347712	total: 581ms	remaining: 164ms
78:	learn: 0.2347102	total: 589ms	remaining: 157ms
79:	learn: 0.2346982	total: 596ms	remaining: 149ms
80:	learn: 0.2346730	total: 603ms	remaining: 141ms
81:	learn: 0.2346504	total: 610ms	remaining: 134ms
82:	learn: 0.2346220	total: 618ms	remaining: 127ms
83:	learn: 0.2346055	total: 625ms	remaining: 119ms
84:	learn: 0.2345446	total: 633ms	remaining: 112ms
85:	learn: 0.2345209	total: 640ms	remaining: 104ms
86:	learn: 0.2345027	total: 647ms	remaining: 96.8ms
87:	learn: 0.2344408	total: 654ms	remaining: 89.2ms
88:	learn: 0.2344205	total: 661ms	remaining: 81.7ms
89:	learn: 0.2343871	total: 669ms	remaining: 74.3ms
90:	learn: 0.2343800	total: 675ms	remaining: 66.8ms
91:	learn: 0.2343691	total: 683ms	remaining: 59.4ms
92:	learn: 0.2343552	total: 690ms	remaining: 52ms
93:	learn: 0.2343435	total: 697ms	remaining: 44.5ms
94:	learn: 0.2343402	tota

In [49]:
# Interquartile Range

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['IQR']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6233483	total: 7.67ms	remaining: 759ms
1:	learn: 0.5665198	total: 14.5ms	remaining: 713ms
2:	learn: 0.5193902	total: 21ms	remaining: 679ms
3:	learn: 0.4799520	total: 27.7ms	remaining: 665ms
4:	learn: 0.4463026	total: 34.5ms	remaining: 656ms
5:	learn: 0.4178772	total: 41.4ms	remaining: 649ms
6:	learn: 0.3926420	total: 48.2ms	remaining: 641ms
7:	learn: 0.3709104	total: 56.2ms	remaining: 646ms
8:	learn: 0.3528330	total: 63.4ms	remaining: 641ms
9:	learn: 0.3364961	total: 70.8ms	remaining: 638ms
10:	learn: 0.3224459	total: 77.8ms	remaining: 630ms
11:	learn: 0.3104655	total: 85.1ms	remaining: 624ms
12:	learn: 0.2999661	total: 92.3ms	remaining: 618ms
13:	learn: 0.2911918	total: 99.1ms	remaining: 609ms
14:	learn: 0.2829052	total: 106ms	remaining: 602ms
15:	learn: 0.2757021	total: 113ms	remaining: 594ms
16:	learn: 0.2694387	total: 120ms	remaining: 586ms
17:	learn: 0.2637453	total: 127ms	remaining: 579ms
18:	learn: 0.2587590	total: 134ms	remaining: 571ms
19:	learn: 0.2544701	total: 1

71:	learn: 0.2258062	total: 566ms	remaining: 220ms
72:	learn: 0.2257909	total: 573ms	remaining: 212ms
73:	learn: 0.2257740	total: 580ms	remaining: 204ms
74:	learn: 0.2257597	total: 587ms	remaining: 196ms
75:	learn: 0.2257425	total: 594ms	remaining: 188ms
76:	learn: 0.2257343	total: 601ms	remaining: 179ms
77:	learn: 0.2257298	total: 607ms	remaining: 171ms
78:	learn: 0.2257049	total: 613ms	remaining: 163ms
79:	learn: 0.2256874	total: 620ms	remaining: 155ms
80:	learn: 0.2256662	total: 626ms	remaining: 147ms
81:	learn: 0.2256628	total: 632ms	remaining: 139ms
82:	learn: 0.2256547	total: 639ms	remaining: 131ms
83:	learn: 0.2256486	total: 645ms	remaining: 123ms
84:	learn: 0.2256371	total: 652ms	remaining: 115ms
85:	learn: 0.2256126	total: 658ms	remaining: 107ms
86:	learn: 0.2256064	total: 664ms	remaining: 99.3ms
87:	learn: 0.2255868	total: 670ms	remaining: 91.4ms
88:	learn: 0.2255692	total: 677ms	remaining: 83.6ms
89:	learn: 0.2255560	total: 683ms	remaining: 75.9ms
90:	learn: 0.2255478	total:

46:	learn: 0.2121819	total: 367ms	remaining: 413ms
47:	learn: 0.2121201	total: 373ms	remaining: 404ms
48:	learn: 0.2119721	total: 380ms	remaining: 396ms
49:	learn: 0.2118416	total: 386ms	remaining: 386ms
50:	learn: 0.2117497	total: 392ms	remaining: 376ms
51:	learn: 0.2116537	total: 397ms	remaining: 367ms
52:	learn: 0.2115939	total: 403ms	remaining: 357ms
53:	learn: 0.2115563	total: 408ms	remaining: 348ms
54:	learn: 0.2115209	total: 414ms	remaining: 339ms
55:	learn: 0.2114829	total: 419ms	remaining: 329ms
56:	learn: 0.2114314	total: 425ms	remaining: 320ms
57:	learn: 0.2113956	total: 430ms	remaining: 312ms
58:	learn: 0.2113653	total: 436ms	remaining: 303ms
59:	learn: 0.2113375	total: 441ms	remaining: 294ms
60:	learn: 0.2113309	total: 447ms	remaining: 286ms
61:	learn: 0.2112910	total: 453ms	remaining: 278ms
62:	learn: 0.2112816	total: 459ms	remaining: 270ms
63:	learn: 0.2112600	total: 465ms	remaining: 261ms
64:	learn: 0.2112430	total: 469ms	remaining: 253ms
65:	learn: 0.2112027	total: 475

20:	learn: 0.2566614	total: 205ms	remaining: 773ms
21:	learn: 0.2539250	total: 216ms	remaining: 766ms
22:	learn: 0.2511437	total: 227ms	remaining: 759ms
23:	learn: 0.2485517	total: 238ms	remaining: 753ms
24:	learn: 0.2463442	total: 247ms	remaining: 742ms
25:	learn: 0.2444249	total: 258ms	remaining: 734ms
26:	learn: 0.2427572	total: 267ms	remaining: 723ms
27:	learn: 0.2411226	total: 277ms	remaining: 712ms
28:	learn: 0.2398888	total: 286ms	remaining: 700ms
29:	learn: 0.2386496	total: 295ms	remaining: 689ms
30:	learn: 0.2375128	total: 306ms	remaining: 680ms
31:	learn: 0.2365493	total: 315ms	remaining: 669ms
32:	learn: 0.2360266	total: 325ms	remaining: 659ms
33:	learn: 0.2352867	total: 335ms	remaining: 650ms
34:	learn: 0.2346720	total: 344ms	remaining: 639ms
35:	learn: 0.2341302	total: 354ms	remaining: 630ms
36:	learn: 0.2336889	total: 365ms	remaining: 621ms
37:	learn: 0.2334113	total: 374ms	remaining: 611ms
38:	learn: 0.2329745	total: 384ms	remaining: 600ms
39:	learn: 0.2326395	total: 396

96:	learn: 0.2113857	total: 730ms	remaining: 22.6ms
97:	learn: 0.2113572	total: 738ms	remaining: 15.1ms
98:	learn: 0.2113568	total: 745ms	remaining: 7.52ms
99:	learn: 0.2113471	total: 752ms	remaining: 0us
0:	learn: 0.6242467	total: 8.43ms	remaining: 835ms
1:	learn: 0.5677766	total: 16.1ms	remaining: 788ms
2:	learn: 0.5213012	total: 22.9ms	remaining: 742ms
3:	learn: 0.4824436	total: 30.2ms	remaining: 724ms
4:	learn: 0.4488657	total: 37.1ms	remaining: 706ms
5:	learn: 0.4203187	total: 44.2ms	remaining: 693ms
6:	learn: 0.3955386	total: 51.4ms	remaining: 683ms
7:	learn: 0.3745697	total: 58.7ms	remaining: 675ms
8:	learn: 0.3568245	total: 65.3ms	remaining: 661ms
9:	learn: 0.3406980	total: 72.6ms	remaining: 653ms
10:	learn: 0.3269032	total: 80.2ms	remaining: 649ms
11:	learn: 0.3148311	total: 87.3ms	remaining: 640ms
12:	learn: 0.3044200	total: 94.6ms	remaining: 633ms
13:	learn: 0.2958011	total: 102ms	remaining: 625ms
14:	learn: 0.2876082	total: 109ms	remaining: 617ms
15:	learn: 0.2805164	total:

...

80:	learn: 0.2197120	total: 770ms	remaining: 181ms
81:	learn: 0.2197094	total: 780ms	remaining: 171ms
82:	learn: 0.2196993	total: 790ms	remaining: 162ms
83:	learn: 0.2196945	total: 800ms	remaining: 152ms
84:	learn: 0.2196824	total: 810ms	remaining: 143ms
85:	learn: 0.2196587	total: 819ms	remaining: 133ms
86:	learn: 0.2196528	total: 828ms	remaining: 124ms
87:	learn: 0.2196430	total: 834ms	remaining: 114ms
88:	learn: 0.2196347	total: 841ms	remaining: 104ms
89:	learn: 0.2196268	total: 848ms	remaining: 94.2ms
90:	learn: 0.2196204	total: 854ms	remaining: 84.5ms
91:	learn: 0.2196168	total: 860ms	remaining: 74.8ms
92:	learn: 0.2196075	total: 866ms	remaining: 65.2ms
93:	learn: 0.2196056	total: 873ms	remaining: 55.7ms
94:	learn: 0.2195961	total: 880ms	remaining: 46.3ms
95:	learn: 0.2195876	total: 886ms	remaining: 36.9ms
96:	learn: 0.2195805	total: 892ms	remaining: 27.6ms
97:	learn: 0.2195755	total: 898ms	remaining: 18.3ms
98:	learn: 0.2195744	total: 904ms	remaining: 9.13ms
99:	learn: 0.2195599	

58:	learn: 0.2307028	total: 573ms	remaining: 398ms
59:	learn: 0.2306695	total: 583ms	remaining: 388ms
60:	learn: 0.2306635	total: 590ms	remaining: 377ms
61:	learn: 0.2306265	total: 600ms	remaining: 368ms
62:	learn: 0.2306191	total: 609ms	remaining: 358ms
63:	learn: 0.2305988	total: 618ms	remaining: 348ms
64:	learn: 0.2305614	total: 628ms	remaining: 338ms
65:	learn: 0.2305349	total: 638ms	remaining: 329ms
66:	learn: 0.2305171	total: 647ms	remaining: 319ms
67:	learn: 0.2305018	total: 658ms	remaining: 309ms
68:	learn: 0.2304774	total: 666ms	remaining: 299ms
69:	learn: 0.2304589	total: 676ms	remaining: 290ms
70:	learn: 0.2304399	total: 684ms	remaining: 280ms
71:	learn: 0.2304290	total: 693ms	remaining: 270ms
72:	learn: 0.2304158	total: 703ms	remaining: 260ms
73:	learn: 0.2304102	total: 712ms	remaining: 250ms
74:	learn: 0.2303973	total: 721ms	remaining: 240ms
75:	learn: 0.2303784	total: 731ms	remaining: 231ms
76:	learn: 0.2303709	total: 740ms	remaining: 221ms
77:	learn: 0.2303566	total: 751

20:	learn: 0.2539376	total: 205ms	remaining: 770ms
21:	learn: 0.2505979	total: 215ms	remaining: 762ms
22:	learn: 0.2477487	total: 225ms	remaining: 754ms
23:	learn: 0.2451931	total: 236ms	remaining: 747ms
24:	learn: 0.2429865	total: 247ms	remaining: 741ms
25:	learn: 0.2410337	total: 257ms	remaining: 731ms
26:	learn: 0.2393589	total: 266ms	remaining: 718ms
27:	learn: 0.2377493	total: 276ms	remaining: 709ms
28:	learn: 0.2365166	total: 286ms	remaining: 699ms
29:	learn: 0.2353174	total: 296ms	remaining: 690ms
30:	learn: 0.2341929	total: 305ms	remaining: 679ms
31:	learn: 0.2332332	total: 314ms	remaining: 667ms
32:	learn: 0.2327201	total: 323ms	remaining: 656ms
33:	learn: 0.2320324	total: 333ms	remaining: 646ms
34:	learn: 0.2316580	total: 342ms	remaining: 635ms
35:	learn: 0.2311052	total: 351ms	remaining: 625ms
36:	learn: 0.2305768	total: 360ms	remaining: 614ms
37:	learn: 0.2301212	total: 369ms	remaining: 603ms
38:	learn: 0.2296791	total: 378ms	remaining: 592ms
39:	learn: 0.2293398	total: 388

81:	learn: 0.2306880	total: 749ms	remaining: 164ms
82:	learn: 0.2306824	total: 756ms	remaining: 155ms
83:	learn: 0.2306614	total: 764ms	remaining: 146ms
84:	learn: 0.2306498	total: 771ms	remaining: 136ms
85:	learn: 0.2306402	total: 777ms	remaining: 127ms
86:	learn: 0.2306327	total: 784ms	remaining: 117ms
87:	learn: 0.2306197	total: 791ms	remaining: 108ms
88:	learn: 0.2306126	total: 798ms	remaining: 98.6ms
89:	learn: 0.2305985	total: 804ms	remaining: 89.4ms
90:	learn: 0.2305928	total: 811ms	remaining: 80.2ms
91:	learn: 0.2305854	total: 817ms	remaining: 71.1ms
92:	learn: 0.2305803	total: 824ms	remaining: 62ms
93:	learn: 0.2305751	total: 830ms	remaining: 53ms
94:	learn: 0.2305642	total: 837ms	remaining: 44ms
95:	learn: 0.2305558	total: 842ms	remaining: 35.1ms
96:	learn: 0.2305456	total: 849ms	remaining: 26.3ms
97:	learn: 0.2305284	total: 856ms	remaining: 17.5ms
98:	learn: 0.2305277	total: 863ms	remaining: 8.71ms
99:	learn: 0.2305213	total: 870ms	remaining: 0us
0:	learn: 0.6236619	total: 8

50:	learn: 0.2313219	total: 375ms	remaining: 361ms
51:	learn: 0.2312248	total: 383ms	remaining: 354ms
52:	learn: 0.2311618	total: 391ms	remaining: 347ms
53:	learn: 0.2311250	total: 398ms	remaining: 339ms
54:	learn: 0.2310879	total: 406ms	remaining: 332ms
55:	learn: 0.2310477	total: 413ms	remaining: 325ms
56:	learn: 0.2309989	total: 421ms	remaining: 318ms
57:	learn: 0.2309597	total: 428ms	remaining: 310ms
58:	learn: 0.2309498	total: 436ms	remaining: 303ms
59:	learn: 0.2309136	total: 443ms	remaining: 295ms
60:	learn: 0.2309082	total: 449ms	remaining: 287ms
61:	learn: 0.2308701	total: 455ms	remaining: 279ms
62:	learn: 0.2308572	total: 462ms	remaining: 271ms
63:	learn: 0.2308390	total: 469ms	remaining: 264ms
64:	learn: 0.2308004	total: 476ms	remaining: 256ms
65:	learn: 0.2307673	total: 484ms	remaining: 249ms
66:	learn: 0.2307494	total: 491ms	remaining: 242ms
67:	learn: 0.2307361	total: 499ms	remaining: 235ms
68:	learn: 0.2307122	total: 506ms	remaining: 227ms
69:	learn: 0.2306940	total: 513

In [50]:
# Variance

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['Var']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6438176	total: 8.13ms	remaining: 805ms
1:	learn: 0.6031914	total: 15.6ms	remaining: 765ms
2:	learn: 0.5697684	total: 22.8ms	remaining: 737ms
3:	learn: 0.5413925	total: 30.2ms	remaining: 724ms
4:	learn: 0.5177507	total: 37.3ms	remaining: 708ms
5:	learn: 0.4973403	total: 45ms	remaining: 706ms
6:	learn: 0.4794144	total: 52.5ms	remaining: 698ms
7:	learn: 0.4643749	total: 63ms	remaining: 725ms
8:	learn: 0.4511758	total: 71.3ms	remaining: 721ms
9:	learn: 0.4396864	total: 78.2ms	remaining: 704ms
10:	learn: 0.4300255	total: 86ms	remaining: 696ms
11:	learn: 0.4215801	total: 94.4ms	remaining: 692ms
12:	learn: 0.4139535	total: 102ms	remaining: 684ms
13:	learn: 0.4073867	total: 110ms	remaining: 673ms
14:	learn: 0.4017670	total: 117ms	remaining: 661ms
15:	learn: 0.3967131	total: 124ms	remaining: 649ms
16:	learn: 0.3927393	total: 131ms	remaining: 639ms
17:	learn: 0.3887836	total: 138ms	remaining: 630ms
18:	learn: 0.3851814	total: 145ms	remaining: 619ms
19:	learn: 0.3824361	total: 152ms	r

76:	learn: 0.3657540	total: 776ms	remaining: 232ms
77:	learn: 0.3657500	total: 784ms	remaining: 221ms
78:	learn: 0.3657461	total: 793ms	remaining: 211ms
79:	learn: 0.3657407	total: 801ms	remaining: 200ms
80:	learn: 0.3657346	total: 810ms	remaining: 190ms
81:	learn: 0.3657325	total: 818ms	remaining: 179ms
82:	learn: 0.3657176	total: 826ms	remaining: 169ms
83:	learn: 0.3657120	total: 834ms	remaining: 159ms
84:	learn: 0.3657009	total: 842ms	remaining: 149ms
85:	learn: 0.3656899	total: 849ms	remaining: 138ms
86:	learn: 0.3656809	total: 856ms	remaining: 128ms
87:	learn: 0.3656732	total: 864ms	remaining: 118ms
88:	learn: 0.3656662	total: 872ms	remaining: 108ms
89:	learn: 0.3656613	total: 879ms	remaining: 97.7ms
90:	learn: 0.3656527	total: 887ms	remaining: 87.7ms
91:	learn: 0.3656465	total: 895ms	remaining: 77.8ms
92:	learn: 0.3656351	total: 903ms	remaining: 68ms
93:	learn: 0.3656308	total: 912ms	remaining: 58.2ms
94:	learn: 0.3656275	total: 919ms	remaining: 48.4ms
95:	learn: 0.3656241	total:

49:	learn: 0.3383731	total: 385ms	remaining: 385ms
50:	learn: 0.3383026	total: 393ms	remaining: 378ms
51:	learn: 0.3382413	total: 402ms	remaining: 371ms
52:	learn: 0.3382146	total: 410ms	remaining: 364ms
53:	learn: 0.3381986	total: 419ms	remaining: 357ms
54:	learn: 0.3381439	total: 427ms	remaining: 349ms
55:	learn: 0.3381172	total: 433ms	remaining: 341ms
56:	learn: 0.3380773	total: 439ms	remaining: 331ms
57:	learn: 0.3380634	total: 447ms	remaining: 323ms
58:	learn: 0.3380476	total: 453ms	remaining: 315ms
59:	learn: 0.3380009	total: 459ms	remaining: 306ms
60:	learn: 0.3379703	total: 464ms	remaining: 297ms
61:	learn: 0.3379555	total: 469ms	remaining: 288ms
62:	learn: 0.3379178	total: 475ms	remaining: 279ms
63:	learn: 0.3378918	total: 480ms	remaining: 270ms
64:	learn: 0.3378689	total: 485ms	remaining: 261ms
65:	learn: 0.3378525	total: 491ms	remaining: 253ms
66:	learn: 0.3378458	total: 496ms	remaining: 244ms
67:	learn: 0.3378232	total: 501ms	remaining: 236ms
68:	learn: 0.3378100	total: 507

19:	learn: 0.3893369	total: 198ms	remaining: 791ms
20:	learn: 0.3867803	total: 208ms	remaining: 782ms
21:	learn: 0.3843929	total: 218ms	remaining: 773ms
22:	learn: 0.3825622	total: 228ms	remaining: 763ms
23:	learn: 0.3806406	total: 238ms	remaining: 754ms
24:	learn: 0.3793885	total: 248ms	remaining: 743ms
25:	learn: 0.3779248	total: 257ms	remaining: 732ms
26:	learn: 0.3767289	total: 267ms	remaining: 721ms
27:	learn: 0.3756195	total: 276ms	remaining: 711ms
28:	learn: 0.3745653	total: 286ms	remaining: 701ms
29:	learn: 0.3736321	total: 296ms	remaining: 691ms
30:	learn: 0.3728320	total: 306ms	remaining: 680ms
31:	learn: 0.3721738	total: 315ms	remaining: 669ms
32:	learn: 0.3715249	total: 324ms	remaining: 658ms
33:	learn: 0.3709835	total: 333ms	remaining: 647ms
34:	learn: 0.3704771	total: 343ms	remaining: 637ms
35:	learn: 0.3700336	total: 353ms	remaining: 627ms
36:	learn: 0.3696726	total: 362ms	remaining: 617ms
37:	learn: 0.3693305	total: 372ms	remaining: 607ms
38:	learn: 0.3690654	total: 382

84:	learn: 0.3532641	total: 564ms	remaining: 99.5ms
85:	learn: 0.3532517	total: 570ms	remaining: 92.9ms
86:	learn: 0.3532394	total: 577ms	remaining: 86.2ms
87:	learn: 0.3532335	total: 583ms	remaining: 79.4ms
88:	learn: 0.3532302	total: 590ms	remaining: 72.9ms
89:	learn: 0.3532141	total: 598ms	remaining: 66.4ms
90:	learn: 0.3532030	total: 606ms	remaining: 59.9ms
91:	learn: 0.3531980	total: 613ms	remaining: 53.3ms
92:	learn: 0.3531838	total: 620ms	remaining: 46.7ms
93:	learn: 0.3531783	total: 625ms	remaining: 39.9ms
94:	learn: 0.3531680	total: 630ms	remaining: 33.2ms
95:	learn: 0.3531572	total: 636ms	remaining: 26.5ms
96:	learn: 0.3531425	total: 644ms	remaining: 19.9ms
97:	learn: 0.3531355	total: 649ms	remaining: 13.2ms
98:	learn: 0.3531212	total: 654ms	remaining: 6.61ms
99:	learn: 0.3531177	total: 659ms	remaining: 0us
0:	learn: 0.6441810	total: 12.9ms	remaining: 1.28s
1:	learn: 0.6038816	total: 25.6ms	remaining: 1.25s
2:	learn: 0.5707583	total: 38.6ms	remaining: 1.25s
3:	learn: 0.542765

...

37:	learn: 0.3581786	total: 390ms	remaining: 636ms
38:	learn: 0.3578824	total: 399ms	remaining: 623ms
39:	learn: 0.3575972	total: 409ms	remaining: 613ms
40:	learn: 0.3573775	total: 420ms	remaining: 605ms
41:	learn: 0.3571534	total: 432ms	remaining: 596ms
42:	learn: 0.3569444	total: 443ms	remaining: 588ms
43:	learn: 0.3567691	total: 454ms	remaining: 577ms
44:	learn: 0.3566179	total: 464ms	remaining: 567ms
45:	learn: 0.3564769	total: 474ms	remaining: 557ms
46:	learn: 0.3563515	total: 485ms	remaining: 547ms
47:	learn: 0.3562569	total: 495ms	remaining: 536ms
48:	learn: 0.3561404	total: 505ms	remaining: 525ms
49:	learn: 0.3560653	total: 515ms	remaining: 515ms
50:	learn: 0.3560443	total: 522ms	remaining: 502ms
51:	learn: 0.3559710	total: 529ms	remaining: 488ms
52:	learn: 0.3559176	total: 536ms	remaining: 475ms
53:	learn: 0.3558715	total: 543ms	remaining: 463ms
54:	learn: 0.3558362	total: 550ms	remaining: 450ms
55:	learn: 0.3558274	total: 557ms	remaining: 437ms
56:	learn: 0.3558033	total: 563

98:	learn: 0.3578796	total: 724ms	remaining: 7.31ms
99:	learn: 0.3578763	total: 731ms	remaining: 0us
0:	learn: 0.6443910	total: 7.83ms	remaining: 776ms
1:	learn: 0.6043788	total: 15.4ms	remaining: 755ms
2:	learn: 0.5711572	total: 24.1ms	remaining: 780ms
3:	learn: 0.5433996	total: 32.3ms	remaining: 774ms
4:	learn: 0.5200264	total: 40.1ms	remaining: 762ms
5:	learn: 0.5002806	total: 48ms	remaining: 753ms
6:	learn: 0.4831314	total: 55.6ms	remaining: 739ms
7:	learn: 0.4684001	total: 62.9ms	remaining: 723ms
8:	learn: 0.4555240	total: 70.1ms	remaining: 709ms
9:	learn: 0.4442766	total: 77.5ms	remaining: 697ms
10:	learn: 0.4345363	total: 84.8ms	remaining: 686ms
11:	learn: 0.4259758	total: 92.1ms	remaining: 676ms
12:	learn: 0.4183330	total: 99.2ms	remaining: 664ms
13:	learn: 0.4117426	total: 107ms	remaining: 657ms
14:	learn: 0.4060881	total: 114ms	remaining: 648ms
15:	learn: 0.4010962	total: 122ms	remaining: 640ms
16:	learn: 0.3965861	total: 129ms	remaining: 629ms
17:	learn: 0.3929630	total: 136

63:	learn: 0.3321717	total: 582ms	remaining: 327ms
64:	learn: 0.3321543	total: 587ms	remaining: 316ms
65:	learn: 0.3321329	total: 592ms	remaining: 305ms
66:	learn: 0.3321289	total: 597ms	remaining: 294ms
67:	learn: 0.3321160	total: 602ms	remaining: 283ms
68:	learn: 0.3321000	total: 608ms	remaining: 273ms
69:	learn: 0.3320942	total: 613ms	remaining: 263ms
70:	learn: 0.3320866	total: 621ms	remaining: 254ms
71:	learn: 0.3320791	total: 627ms	remaining: 244ms
72:	learn: 0.3320604	total: 633ms	remaining: 234ms
73:	learn: 0.3320545	total: 638ms	remaining: 224ms
74:	learn: 0.3320468	total: 644ms	remaining: 215ms
75:	learn: 0.3320430	total: 649ms	remaining: 205ms
76:	learn: 0.3320278	total: 654ms	remaining: 195ms
77:	learn: 0.3320127	total: 660ms	remaining: 186ms
78:	learn: 0.3319987	total: 667ms	remaining: 177ms
79:	learn: 0.3319935	total: 674ms	remaining: 169ms
80:	learn: 0.3319788	total: 682ms	remaining: 160ms
81:	learn: 0.3319602	total: 689ms	remaining: 151ms
82:	learn: 0.3319588	total: 697

24:	learn: 0.3764266	total: 197ms	remaining: 590ms
25:	learn: 0.3748815	total: 205ms	remaining: 584ms
26:	learn: 0.3736669	total: 214ms	remaining: 579ms
27:	learn: 0.3725132	total: 222ms	remaining: 571ms
28:	learn: 0.3717113	total: 230ms	remaining: 562ms
29:	learn: 0.3706995	total: 236ms	remaining: 550ms
30:	learn: 0.3698562	total: 241ms	remaining: 537ms
31:	learn: 0.3693729	total: 247ms	remaining: 526ms
32:	learn: 0.3686700	total: 254ms	remaining: 516ms
33:	learn: 0.3680722	total: 260ms	remaining: 505ms
34:	learn: 0.3675154	total: 266ms	remaining: 494ms
35:	learn: 0.3670124	total: 273ms	remaining: 485ms
36:	learn: 0.3666006	total: 279ms	remaining: 475ms
37:	learn: 0.3661858	total: 286ms	remaining: 466ms
38:	learn: 0.3658517	total: 292ms	remaining: 457ms
39:	learn: 0.3655528	total: 298ms	remaining: 447ms
40:	learn: 0.3652820	total: 304ms	remaining: 438ms
41:	learn: 0.3651274	total: 311ms	remaining: 429ms
42:	learn: 0.3649980	total: 317ms	remaining: 420ms
43:	learn: 0.3648250	total: 323

0:	learn: 0.6443802	total: 11.4ms	remaining: 1.13s
1:	learn: 0.6042925	total: 21.4ms	remaining: 1.05s
2:	learn: 0.5711714	total: 31.5ms	remaining: 1.02s
3:	learn: 0.5434100	total: 41.8ms	remaining: 1s
4:	learn: 0.5200067	total: 51.9ms	remaining: 985ms
5:	learn: 0.5001310	total: 62.4ms	remaining: 978ms
6:	learn: 0.4829206	total: 73.4ms	remaining: 976ms
7:	learn: 0.4681352	total: 84.2ms	remaining: 968ms
8:	learn: 0.4552084	total: 94.5ms	remaining: 956ms
9:	learn: 0.4439682	total: 106ms	remaining: 950ms
10:	learn: 0.4342048	total: 117ms	remaining: 947ms
11:	learn: 0.4259492	total: 127ms	remaining: 931ms
12:	learn: 0.4182705	total: 137ms	remaining: 917ms
13:	learn: 0.4116356	total: 148ms	remaining: 908ms
14:	learn: 0.4059784	total: 158ms	remaining: 897ms
15:	learn: 0.4009945	total: 168ms	remaining: 884ms
16:	learn: 0.3966308	total: 179ms	remaining: 872ms
17:	learn: 0.3929491	total: 192ms	remaining: 873ms
18:	learn: 0.3894889	total: 200ms	remaining: 853ms
19:	learn: 0.3862865	total: 207ms	r

In [51]:
# Rmean Variance

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['Rmean_Var']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6556261	total: 10ms	remaining: 995ms
1:	learn: 0.6249663	total: 19.3ms	remaining: 946ms
2:	learn: 0.5995768	total: 28.5ms	remaining: 921ms
3:	learn: 0.5783274	total: 37.8ms	remaining: 907ms
4:	learn: 0.5602810	total: 47.1ms	remaining: 894ms
5:	learn: 0.5451255	total: 56.4ms	remaining: 884ms
6:	learn: 0.5320722	total: 65.1ms	remaining: 866ms
7:	learn: 0.5210425	total: 74.1ms	remaining: 852ms
8:	learn: 0.5112610	total: 83.7ms	remaining: 846ms
9:	learn: 0.5029046	total: 92.2ms	remaining: 830ms
10:	learn: 0.4957558	total: 101ms	remaining: 819ms
11:	learn: 0.4893099	total: 110ms	remaining: 808ms
12:	learn: 0.4838375	total: 119ms	remaining: 797ms
13:	learn: 0.4791708	total: 128ms	remaining: 786ms
14:	learn: 0.4749340	total: 137ms	remaining: 774ms
15:	learn: 0.4714678	total: 146ms	remaining: 764ms
16:	learn: 0.4681616	total: 155ms	remaining: 755ms
17:	learn: 0.4654874	total: 164ms	remaining: 746ms
18:	learn: 0.4628434	total: 173ms	remaining: 738ms
19:	learn: 0.4607971	total: 182ms

75:	learn: 0.4092254	total: 555ms	remaining: 175ms
76:	learn: 0.4092027	total: 562ms	remaining: 168ms
77:	learn: 0.4091773	total: 570ms	remaining: 161ms
78:	learn: 0.4091686	total: 577ms	remaining: 154ms
79:	learn: 0.4091412	total: 584ms	remaining: 146ms
80:	learn: 0.4091098	total: 591ms	remaining: 139ms
81:	learn: 0.4090877	total: 599ms	remaining: 131ms
82:	learn: 0.4090721	total: 606ms	remaining: 124ms
83:	learn: 0.4090532	total: 613ms	remaining: 117ms
84:	learn: 0.4090357	total: 620ms	remaining: 109ms
85:	learn: 0.4090077	total: 627ms	remaining: 102ms
86:	learn: 0.4089838	total: 634ms	remaining: 94.7ms
87:	learn: 0.4089659	total: 641ms	remaining: 87.4ms
88:	learn: 0.4089605	total: 648ms	remaining: 80.1ms
89:	learn: 0.4089405	total: 655ms	remaining: 72.8ms
90:	learn: 0.4089217	total: 662ms	remaining: 65.5ms
91:	learn: 0.4088961	total: 669ms	remaining: 58.1ms
92:	learn: 0.4088874	total: 676ms	remaining: 50.9ms
93:	learn: 0.4088761	total: 683ms	remaining: 43.6ms
94:	learn: 0.4088681	to

49:	learn: 0.4386057	total: 369ms	remaining: 369ms
50:	learn: 0.4384850	total: 376ms	remaining: 362ms
51:	learn: 0.4382595	total: 383ms	remaining: 354ms
52:	learn: 0.4380921	total: 391ms	remaining: 346ms
53:	learn: 0.4380071	total: 398ms	remaining: 339ms
54:	learn: 0.4379429	total: 405ms	remaining: 332ms
55:	learn: 0.4378138	total: 412ms	remaining: 324ms
56:	learn: 0.4377292	total: 419ms	remaining: 316ms
57:	learn: 0.4377052	total: 426ms	remaining: 309ms
58:	learn: 0.4376158	total: 433ms	remaining: 301ms
59:	learn: 0.4375455	total: 440ms	remaining: 294ms
60:	learn: 0.4375230	total: 448ms	remaining: 286ms
61:	learn: 0.4374650	total: 457ms	remaining: 280ms
62:	learn: 0.4374309	total: 464ms	remaining: 273ms
63:	learn: 0.4373655	total: 471ms	remaining: 265ms
64:	learn: 0.4373007	total: 478ms	remaining: 257ms
65:	learn: 0.4372555	total: 485ms	remaining: 250ms
66:	learn: 0.4372052	total: 491ms	remaining: 242ms
67:	learn: 0.4371713	total: 497ms	remaining: 234ms
68:	learn: 0.4371180	total: 504

16:	learn: 0.4330906	total: 189ms	remaining: 924ms
17:	learn: 0.4300535	total: 200ms	remaining: 910ms
18:	learn: 0.4271112	total: 210ms	remaining: 894ms
19:	learn: 0.4245705	total: 220ms	remaining: 881ms
20:	learn: 0.4223668	total: 231ms	remaining: 868ms
21:	learn: 0.4204335	total: 241ms	remaining: 855ms
22:	learn: 0.4184275	total: 251ms	remaining: 839ms
23:	learn: 0.4167550	total: 260ms	remaining: 824ms
24:	learn: 0.4152677	total: 270ms	remaining: 810ms
25:	learn: 0.4144437	total: 280ms	remaining: 796ms
26:	learn: 0.4132978	total: 289ms	remaining: 783ms
27:	learn: 0.4122403	total: 299ms	remaining: 770ms
28:	learn: 0.4112961	total: 309ms	remaining: 757ms
29:	learn: 0.4104598	total: 319ms	remaining: 743ms
30:	learn: 0.4098401	total: 328ms	remaining: 730ms
31:	learn: 0.4091858	total: 338ms	remaining: 718ms
32:	learn: 0.4086447	total: 348ms	remaining: 706ms
33:	learn: 0.4081311	total: 357ms	remaining: 692ms
34:	learn: 0.4078235	total: 368ms	remaining: 684ms
35:	learn: 0.4072928	total: 378

0:	learn: 0.6559454	total: 10.2ms	remaining: 1s
1:	learn: 0.6255664	total: 20.6ms	remaining: 1.01s
2:	learn: 0.6004142	total: 30.4ms	remaining: 982ms
3:	learn: 0.5794147	total: 39.4ms	remaining: 946ms
4:	learn: 0.5615530	total: 48.4ms	remaining: 919ms
5:	learn: 0.5464361	total: 57.1ms	remaining: 895ms
6:	learn: 0.5335962	total: 66.4ms	remaining: 883ms
7:	learn: 0.5227062	total: 75.2ms	remaining: 865ms
8:	learn: 0.5130835	total: 84.1ms	remaining: 851ms
9:	learn: 0.5047252	total: 93.1ms	remaining: 838ms
10:	learn: 0.4976667	total: 102ms	remaining: 825ms
11:	learn: 0.4913357	total: 111ms	remaining: 816ms
12:	learn: 0.4860163	total: 120ms	remaining: 805ms
13:	learn: 0.4812740	total: 129ms	remaining: 795ms
14:	learn: 0.4773867	total: 139ms	remaining: 785ms
15:	learn: 0.4740252	total: 147ms	remaining: 774ms
16:	learn: 0.4707704	total: 156ms	remaining: 764ms
17:	learn: 0.4680881	total: 165ms	remaining: 752ms
18:	learn: 0.4654857	total: 174ms	remaining: 743ms
19:	learn: 0.4634047	total: 183ms	

...

75:	learn: 0.4379311	total: 545ms	remaining: 172ms
76:	learn: 0.4378765	total: 553ms	remaining: 165ms
77:	learn: 0.4378587	total: 561ms	remaining: 158ms
78:	learn: 0.4378286	total: 568ms	remaining: 151ms
79:	learn: 0.4377919	total: 576ms	remaining: 144ms
80:	learn: 0.4377607	total: 583ms	remaining: 137ms
81:	learn: 0.4377384	total: 590ms	remaining: 129ms
82:	learn: 0.4377239	total: 597ms	remaining: 122ms
83:	learn: 0.4377088	total: 603ms	remaining: 115ms
84:	learn: 0.4376928	total: 610ms	remaining: 108ms
85:	learn: 0.4376844	total: 617ms	remaining: 100ms
86:	learn: 0.4376608	total: 624ms	remaining: 93.2ms
87:	learn: 0.4376494	total: 631ms	remaining: 86ms
88:	learn: 0.4376401	total: 638ms	remaining: 78.9ms
89:	learn: 0.4376168	total: 645ms	remaining: 71.7ms
90:	learn: 0.4375943	total: 652ms	remaining: 64.5ms
91:	learn: 0.4375786	total: 659ms	remaining: 57.3ms
92:	learn: 0.4375572	total: 666ms	remaining: 50.1ms
93:	learn: 0.4375452	total: 673ms	remaining: 43ms
94:	learn: 0.4375403	total:

43:	learn: 0.4142895	total: 396ms	remaining: 504ms
44:	learn: 0.4139364	total: 406ms	remaining: 496ms
45:	learn: 0.4137108	total: 415ms	remaining: 487ms
46:	learn: 0.4135778	total: 424ms	remaining: 478ms
47:	learn: 0.4133952	total: 434ms	remaining: 470ms
48:	learn: 0.4132303	total: 442ms	remaining: 460ms
49:	learn: 0.4130392	total: 451ms	remaining: 451ms
50:	learn: 0.4127661	total: 459ms	remaining: 441ms
51:	learn: 0.4126429	total: 468ms	remaining: 432ms
52:	learn: 0.4125186	total: 477ms	remaining: 423ms
53:	learn: 0.4123864	total: 488ms	remaining: 416ms
54:	learn: 0.4122740	total: 498ms	remaining: 407ms
55:	learn: 0.4121812	total: 507ms	remaining: 398ms
56:	learn: 0.4120945	total: 515ms	remaining: 389ms
57:	learn: 0.4120675	total: 524ms	remaining: 379ms
58:	learn: 0.4119237	total: 532ms	remaining: 370ms
59:	learn: 0.4118421	total: 541ms	remaining: 361ms
60:	learn: 0.4117872	total: 549ms	remaining: 351ms
61:	learn: 0.4116951	total: 557ms	remaining: 342ms
62:	learn: 0.4116542	total: 571

25:	learn: 0.4448621	total: 191ms	remaining: 543ms
26:	learn: 0.4437047	total: 199ms	remaining: 537ms
27:	learn: 0.4426919	total: 207ms	remaining: 531ms
28:	learn: 0.4416291	total: 215ms	remaining: 526ms
29:	learn: 0.4407480	total: 223ms	remaining: 519ms
30:	learn: 0.4397964	total: 230ms	remaining: 511ms
31:	learn: 0.4392925	total: 237ms	remaining: 504ms
32:	learn: 0.4385430	total: 244ms	remaining: 496ms
33:	learn: 0.4379709	total: 252ms	remaining: 488ms
34:	learn: 0.4374540	total: 259ms	remaining: 481ms
35:	learn: 0.4370233	total: 266ms	remaining: 473ms
36:	learn: 0.4366920	total: 273ms	remaining: 464ms
37:	learn: 0.4365067	total: 280ms	remaining: 456ms
38:	learn: 0.4361663	total: 287ms	remaining: 449ms
39:	learn: 0.4358617	total: 294ms	remaining: 442ms
40:	learn: 0.4355728	total: 302ms	remaining: 435ms
41:	learn: 0.4352167	total: 310ms	remaining: 428ms
42:	learn: 0.4349599	total: 317ms	remaining: 421ms
43:	learn: 0.4347725	total: 324ms	remaining: 413ms
44:	learn: 0.4346350	total: 332

92:	learn: 0.4370319	total: 766ms	remaining: 57.7ms
93:	learn: 0.4370135	total: 774ms	remaining: 49.4ms
94:	learn: 0.4370039	total: 782ms	remaining: 41.1ms
95:	learn: 0.4369748	total: 789ms	remaining: 32.9ms
96:	learn: 0.4369541	total: 797ms	remaining: 24.6ms
97:	learn: 0.4369424	total: 804ms	remaining: 16.4ms
98:	learn: 0.4369382	total: 811ms	remaining: 8.19ms
99:	learn: 0.4369195	total: 818ms	remaining: 0us
0:	learn: 0.6558811	total: 8.01ms	remaining: 793ms
1:	learn: 0.6254122	total: 15.6ms	remaining: 766ms
2:	learn: 0.6002231	total: 22.8ms	remaining: 736ms
3:	learn: 0.5793207	total: 30.2ms	remaining: 725ms
4:	learn: 0.5613512	total: 37.5ms	remaining: 713ms
5:	learn: 0.5460998	total: 44.7ms	remaining: 700ms
6:	learn: 0.5332180	total: 52.1ms	remaining: 692ms
7:	learn: 0.5222755	total: 59.2ms	remaining: 681ms
8:	learn: 0.5126699	total: 66.7ms	remaining: 675ms
9:	learn: 0.5043221	total: 73.9ms	remaining: 665ms
10:	learn: 0.4972855	total: 81.3ms	remaining: 657ms
11:	learn: 0.4909216	tota

73:	learn: 0.4385158	total: 575ms	remaining: 202ms
74:	learn: 0.4384815	total: 582ms	remaining: 194ms
75:	learn: 0.4384513	total: 589ms	remaining: 186ms
76:	learn: 0.4384106	total: 597ms	remaining: 178ms
77:	learn: 0.4383875	total: 605ms	remaining: 171ms
78:	learn: 0.4383500	total: 611ms	remaining: 163ms
79:	learn: 0.4383368	total: 618ms	remaining: 155ms
80:	learn: 0.4383089	total: 625ms	remaining: 147ms
81:	learn: 0.4382703	total: 632ms	remaining: 139ms
82:	learn: 0.4382574	total: 639ms	remaining: 131ms
83:	learn: 0.4382286	total: 645ms	remaining: 123ms
84:	learn: 0.4381946	total: 652ms	remaining: 115ms
85:	learn: 0.4381859	total: 659ms	remaining: 107ms
86:	learn: 0.4381434	total: 666ms	remaining: 99.4ms
87:	learn: 0.4381261	total: 676ms	remaining: 92.2ms
88:	learn: 0.4381153	total: 686ms	remaining: 84.8ms
89:	learn: 0.4380989	total: 696ms	remaining: 77.3ms
90:	learn: 0.4380924	total: 706ms	remaining: 69.8ms
91:	learn: 0.4380794	total: 716ms	remaining: 62.2ms
92:	learn: 0.4380698	tota

In [52]:
# Mean Absolute Deviation

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

acc_score = [];
Truth = [];
Output = [];
iterations_list = []
max_depth_list = []
x_values = ['Mean_Absolute_Deviation']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf

for idx, subject in enumerate(good_list):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()
    
    model.fit(X_train, y_train)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    #print(model.feature_importances_)
#     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
#                           index = (X_train.columns))
#     Importance.sort_values(by = 'Importance', 
#                        axis = 0, 
#                        ascending = True).plot(kind = 'barh', 
#                                               color = 'r')
#     plt.xlabel('Variable Importance')
#     plt.gca().legend_ = None
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
try: 
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
except: 
    print('hi')
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))
print()
print(acc_score)
print()
print(elapsed)

0:	learn: 0.6354624	total: 8.4ms	remaining: 831ms
1:	learn: 0.5882257	total: 17.4ms	remaining: 854ms
2:	learn: 0.5489579	total: 25.7ms	remaining: 831ms
3:	learn: 0.5160205	total: 34.8ms	remaining: 836ms
4:	learn: 0.4885428	total: 43.4ms	remaining: 824ms
5:	learn: 0.4644688	total: 51.8ms	remaining: 812ms
6:	learn: 0.4437418	total: 59.4ms	remaining: 790ms
7:	learn: 0.4259422	total: 67ms	remaining: 771ms
8:	learn: 0.4103194	total: 74.5ms	remaining: 753ms
9:	learn: 0.3970614	total: 81.7ms	remaining: 736ms
10:	learn: 0.3857947	total: 89.6ms	remaining: 725ms
11:	learn: 0.3756531	total: 97.1ms	remaining: 712ms
12:	learn: 0.3667311	total: 104ms	remaining: 698ms
13:	learn: 0.3589023	total: 112ms	remaining: 687ms
14:	learn: 0.3520102	total: 120ms	remaining: 678ms
15:	learn: 0.3461074	total: 127ms	remaining: 667ms
16:	learn: 0.3410918	total: 134ms	remaining: 657ms
17:	learn: 0.3366207	total: 142ms	remaining: 646ms
18:	learn: 0.3324875	total: 149ms	remaining: 636ms
19:	learn: 0.3287739	total: 157m

61:	learn: 0.3078215	total: 568ms	remaining: 348ms
62:	learn: 0.3077913	total: 576ms	remaining: 338ms
63:	learn: 0.3077753	total: 583ms	remaining: 328ms
64:	learn: 0.3077552	total: 591ms	remaining: 318ms
65:	learn: 0.3077418	total: 599ms	remaining: 308ms
66:	learn: 0.3077223	total: 605ms	remaining: 298ms
67:	learn: 0.3077143	total: 612ms	remaining: 288ms
68:	learn: 0.3076947	total: 619ms	remaining: 278ms
69:	learn: 0.3076783	total: 626ms	remaining: 268ms
70:	learn: 0.3076697	total: 633ms	remaining: 259ms
71:	learn: 0.3076597	total: 640ms	remaining: 249ms
72:	learn: 0.3076488	total: 647ms	remaining: 239ms
73:	learn: 0.3076328	total: 654ms	remaining: 230ms
74:	learn: 0.3076262	total: 661ms	remaining: 220ms
75:	learn: 0.3076201	total: 668ms	remaining: 211ms
76:	learn: 0.3076123	total: 675ms	remaining: 202ms
77:	learn: 0.3075966	total: 682ms	remaining: 192ms
78:	learn: 0.3075887	total: 689ms	remaining: 183ms
79:	learn: 0.3075817	total: 696ms	remaining: 174ms
80:	learn: 0.3075796	total: 702

25:	learn: 0.2963203	total: 198ms	remaining: 563ms
26:	learn: 0.2946620	total: 203ms	remaining: 549ms
27:	learn: 0.2932649	total: 208ms	remaining: 535ms
28:	learn: 0.2922079	total: 214ms	remaining: 523ms
29:	learn: 0.2911850	total: 219ms	remaining: 511ms
30:	learn: 0.2903170	total: 224ms	remaining: 499ms
31:	learn: 0.2895120	total: 229ms	remaining: 487ms
32:	learn: 0.2887376	total: 234ms	remaining: 475ms
33:	learn: 0.2880434	total: 240ms	remaining: 466ms
34:	learn: 0.2874510	total: 248ms	remaining: 460ms
35:	learn: 0.2871379	total: 255ms	remaining: 453ms
36:	learn: 0.2866432	total: 262ms	remaining: 447ms
37:	learn: 0.2862013	total: 270ms	remaining: 440ms
38:	learn: 0.2858277	total: 277ms	remaining: 433ms
39:	learn: 0.2855183	total: 284ms	remaining: 427ms
40:	learn: 0.2852219	total: 292ms	remaining: 420ms
41:	learn: 0.2849798	total: 299ms	remaining: 413ms
42:	learn: 0.2847719	total: 307ms	remaining: 406ms
43:	learn: 0.2845824	total: 314ms	remaining: 400ms
44:	learn: 0.2845006	total: 322

0:	learn: 0.6356751	total: 8.23ms	remaining: 815ms
1:	learn: 0.5889042	total: 16.9ms	remaining: 828ms
2:	learn: 0.5505324	total: 24.8ms	remaining: 802ms
3:	learn: 0.5184887	total: 32.9ms	remaining: 789ms
4:	learn: 0.4915438	total: 41.9ms	remaining: 796ms
5:	learn: 0.4675716	total: 50ms	remaining: 783ms
6:	learn: 0.4472984	total: 58.1ms	remaining: 772ms
7:	learn: 0.4297552	total: 65.6ms	remaining: 755ms
8:	learn: 0.4145464	total: 73.4ms	remaining: 742ms
9:	learn: 0.4015042	total: 81.3ms	remaining: 732ms
10:	learn: 0.3902942	total: 89.3ms	remaining: 723ms
11:	learn: 0.3804568	total: 97ms	remaining: 711ms
12:	learn: 0.3717225	total: 105ms	remaining: 700ms
13:	learn: 0.3642014	total: 112ms	remaining: 689ms
14:	learn: 0.3575485	total: 120ms	remaining: 678ms
15:	learn: 0.3515990	total: 127ms	remaining: 669ms
16:	learn: 0.3464487	total: 135ms	remaining: 659ms
17:	learn: 0.3422724	total: 143ms	remaining: 649ms
18:	learn: 0.3383395	total: 150ms	remaining: 641ms
19:	learn: 0.3348098	total: 159ms

68:	learn: 0.2963246	total: 551ms	remaining: 248ms
69:	learn: 0.2963126	total: 560ms	remaining: 240ms
70:	learn: 0.2962830	total: 568ms	remaining: 232ms
71:	learn: 0.2962713	total: 576ms	remaining: 224ms
72:	learn: 0.2962467	total: 584ms	remaining: 216ms
73:	learn: 0.2962312	total: 591ms	remaining: 208ms
74:	learn: 0.2962058	total: 598ms	remaining: 199ms
75:	learn: 0.2962028	total: 604ms	remaining: 191ms
76:	learn: 0.2961890	total: 611ms	remaining: 183ms
77:	learn: 0.2961665	total: 619ms	remaining: 175ms
78:	learn: 0.2961445	total: 626ms	remaining: 166ms
79:	learn: 0.2961305	total: 633ms	remaining: 158ms
80:	learn: 0.2961201	total: 640ms	remaining: 150ms
81:	learn: 0.2961009	total: 648ms	remaining: 142ms
82:	learn: 0.2960880	total: 654ms	remaining: 134ms
83:	learn: 0.2960820	total: 661ms	remaining: 126ms
84:	learn: 0.2960715	total: 668ms	remaining: 118ms
85:	learn: 0.2960572	total: 675ms	remaining: 110ms
86:	learn: 0.2960494	total: 683ms	remaining: 102ms
87:	learn: 0.2960368	total: 690

...

26:	learn: 0.3104998	total: 200ms	remaining: 541ms
27:	learn: 0.3091448	total: 208ms	remaining: 535ms
28:	learn: 0.3080478	total: 216ms	remaining: 530ms
29:	learn: 0.3069564	total: 225ms	remaining: 524ms
30:	learn: 0.3060795	total: 232ms	remaining: 517ms
31:	learn: 0.3052546	total: 240ms	remaining: 510ms
32:	learn: 0.3044852	total: 248ms	remaining: 503ms
33:	learn: 0.3038337	total: 255ms	remaining: 495ms
34:	learn: 0.3035240	total: 262ms	remaining: 487ms
35:	learn: 0.3031843	total: 270ms	remaining: 480ms
36:	learn: 0.3027201	total: 278ms	remaining: 473ms
37:	learn: 0.3022624	total: 285ms	remaining: 465ms
38:	learn: 0.3019013	total: 292ms	remaining: 457ms
39:	learn: 0.3016373	total: 299ms	remaining: 449ms
40:	learn: 0.3013468	total: 307ms	remaining: 442ms
41:	learn: 0.3011307	total: 314ms	remaining: 434ms
42:	learn: 0.3009228	total: 321ms	remaining: 426ms
43:	learn: 0.3007267	total: 328ms	remaining: 417ms
44:	learn: 0.3006420	total: 335ms	remaining: 409ms
45:	learn: 0.3004678	total: 342

90:	learn: 0.3049980	total: 729ms	remaining: 72.1ms
91:	learn: 0.3049903	total: 737ms	remaining: 64.1ms
92:	learn: 0.3049835	total: 745ms	remaining: 56.1ms
93:	learn: 0.3049788	total: 753ms	remaining: 48ms
94:	learn: 0.3049762	total: 761ms	remaining: 40ms
95:	learn: 0.3049726	total: 770ms	remaining: 32.1ms
96:	learn: 0.3049528	total: 777ms	remaining: 24ms
97:	learn: 0.3049453	total: 784ms	remaining: 16ms
98:	learn: 0.3049401	total: 792ms	remaining: 8ms
99:	learn: 0.3049320	total: 799ms	remaining: 0us
0:	learn: 0.6367948	total: 11ms	remaining: 1.09s
1:	learn: 0.5904358	total: 21.1ms	remaining: 1.04s
2:	learn: 0.5513487	total: 30.8ms	remaining: 997ms
3:	learn: 0.5192939	total: 41.2ms	remaining: 988ms
4:	learn: 0.4918928	total: 51.9ms	remaining: 986ms
5:	learn: 0.4680138	total: 62.3ms	remaining: 976ms
6:	learn: 0.4475479	total: 72.5ms	remaining: 963ms
7:	learn: 0.4304156	total: 82.8ms	remaining: 952ms
8:	learn: 0.4151750	total: 93.1ms	remaining: 942ms
9:	learn: 0.4020679	total: 103ms	rema

53:	learn: 0.2760459	total: 388ms	remaining: 331ms
54:	learn: 0.2759780	total: 394ms	remaining: 322ms
55:	learn: 0.2759549	total: 398ms	remaining: 313ms
56:	learn: 0.2758955	total: 404ms	remaining: 304ms
57:	learn: 0.2758657	total: 409ms	remaining: 297ms
58:	learn: 0.2758275	total: 414ms	remaining: 288ms
59:	learn: 0.2758153	total: 419ms	remaining: 280ms
60:	learn: 0.2757775	total: 425ms	remaining: 272ms
61:	learn: 0.2757489	total: 430ms	remaining: 263ms
62:	learn: 0.2757141	total: 435ms	remaining: 255ms
63:	learn: 0.2756866	total: 441ms	remaining: 248ms
64:	learn: 0.2756608	total: 447ms	remaining: 240ms
65:	learn: 0.2756425	total: 452ms	remaining: 233ms
66:	learn: 0.2756285	total: 457ms	remaining: 225ms
67:	learn: 0.2756178	total: 463ms	remaining: 218ms
68:	learn: 0.2755993	total: 468ms	remaining: 210ms
69:	learn: 0.2755829	total: 473ms	remaining: 203ms
70:	learn: 0.2755772	total: 479ms	remaining: 196ms
71:	learn: 0.2755681	total: 484ms	remaining: 188ms
72:	learn: 0.2755555	total: 490

24:	learn: 0.3231045	total: 197ms	remaining: 590ms
25:	learn: 0.3213825	total: 205ms	remaining: 583ms
26:	learn: 0.3198886	total: 213ms	remaining: 575ms
27:	learn: 0.3185578	total: 221ms	remaining: 569ms
28:	learn: 0.3174770	total: 229ms	remaining: 562ms
29:	learn: 0.3165223	total: 236ms	remaining: 552ms
30:	learn: 0.3156705	total: 244ms	remaining: 544ms
31:	learn: 0.3148668	total: 252ms	remaining: 535ms
32:	learn: 0.3141084	total: 259ms	remaining: 527ms
33:	learn: 0.3134594	total: 267ms	remaining: 517ms
34:	learn: 0.3128929	total: 274ms	remaining: 508ms
35:	learn: 0.3125924	total: 281ms	remaining: 500ms
36:	learn: 0.3121676	total: 289ms	remaining: 491ms
37:	learn: 0.3117471	total: 296ms	remaining: 482ms
38:	learn: 0.3114237	total: 303ms	remaining: 474ms
39:	learn: 0.3111010	total: 311ms	remaining: 466ms
40:	learn: 0.3108238	total: 318ms	remaining: 458ms
41:	learn: 0.3105861	total: 325ms	remaining: 449ms
42:	learn: 0.3104414	total: 332ms	remaining: 440ms
43:	learn: 0.3102824	total: 339

98:	learn: 0.3034628	total: 737ms	remaining: 7.45ms
99:	learn: 0.3034497	total: 745ms	remaining: 0us
0:	learn: 0.6368234	total: 14.5ms	remaining: 1.44s
1:	learn: 0.5904480	total: 27.8ms	remaining: 1.36s
2:	learn: 0.5513910	total: 40.5ms	remaining: 1.31s
3:	learn: 0.5190992	total: 52.9ms	remaining: 1.27s
4:	learn: 0.4917086	total: 65.3ms	remaining: 1.24s
5:	learn: 0.4683175	total: 77.3ms	remaining: 1.21s
6:	learn: 0.4477019	total: 89.8ms	remaining: 1.19s
7:	learn: 0.4304574	total: 101ms	remaining: 1.17s
8:	learn: 0.4155376	total: 113ms	remaining: 1.14s
9:	learn: 0.4022745	total: 125ms	remaining: 1.12s
10:	learn: 0.3910694	total: 137ms	remaining: 1.11s
11:	learn: 0.3808631	total: 149ms	remaining: 1.09s
12:	learn: 0.3722300	total: 161ms	remaining: 1.08s
13:	learn: 0.3645297	total: 173ms	remaining: 1.06s
14:	learn: 0.3576600	total: 186ms	remaining: 1.05s
15:	learn: 0.3520955	total: 198ms	remaining: 1.04s
16:	learn: 0.3473199	total: 212ms	remaining: 1.03s
17:	learn: 0.3428524	total: 225ms	r